# **_CRYSTALS_** (**_CRYptographic SuiTe for Algebraic LatticeS_**): <br> &nbsp;&#x2022; **_Kyber_** (Study and Analysis)
***
***

## Introduction
***

***In development...***

***

## Background
***

***In development...***

***

## Demonstration of the CRYSTALS-Kyber Cryptosystem,<br>using Bouncy Castle library (version 1.73) for Java
***

### Environment setup for the Bouncy Castle Crypto library (version 1.73) for Java

In [1]:
// Import of the JAR file for the basic resources of
// the Bouncy Castle Crypto library (version 1.73) for Java.
%classpath add jar ../../../etc/resources/jar/bcprov-jdk18on-173.jar

In [2]:
// Import of the JAR file for the extension resources of
// the Bouncy Castle Crypto library (version 1.73) for Java.
%classpath add jar ../../../etc/resources/jar/bcprov-ext-jdk18on-173.jar

***

### Demonstration for the extraction of Execution and Security Parameter Sets<br>for the CRYSTALS-Kyber Cryptosystem

In [3]:
// Import of all the required modules and sub-libraries.

// Import of all the required modules and sub-libraries of
// the Bouncy Castle library, for use of basic cryptography.
import org.bouncycastle.crypto.AsymmetricCipherKeyPair;
import org.bouncycastle.crypto.SecretWithEncapsulation;
import org.bouncycastle.crypto.params.AsymmetricKeyParameter;

// Import of all the required modules and sub-libraries of
// the Bouncy Castle library, regarding the use of
// Key Encapsulation Methods/Mechanisms (KEMs).
import org.bouncycastle.jcajce.SecretKeyWithEncapsulation;
import org.bouncycastle.jcajce.spec.KEMExtractSpec;
import org.bouncycastle.jcajce.spec.KEMGenerateSpec;

// Import of all the required modules and sub-libraries of
// the Bouncy Castle library, regarding the provider of
// (Classical) Post-Quantum Cryptography.
import org.bouncycastle.pqc.jcajce.provider.BouncyCastlePQCProvider;

// Import of all the required modules and sub-libraries of
// the Bouncy Castle library, regarding the CRYSTALS-Kyber
// Public-Key Cryptosystem's Parameter Specifications.
import org.bouncycastle.pqc.jcajce.spec.KyberParameterSpec;

// Import of all the required modules and sub-libraries required of
// the Bouncy Castle library, regarding the CRYSTALS-Kyber Cryptosystem.
import org.bouncycastle.pqc.crypto.crystals.kyber.KyberParameters;
import org.bouncycastle.pqc.crypto.crystals.kyber.KyberKeyGenerationParameters;
import org.bouncycastle.pqc.crypto.crystals.kyber.KyberKeyPairGenerator;
import org.bouncycastle.pqc.crypto.crystals.kyber.KyberPrivateKeyParameters;
import org.bouncycastle.pqc.crypto.crystals.kyber.KyberPublicKeyParameters;

// Import of all the required modules and sub-libraries
// required of the built-in security module of Java.
import java.security.PrivateKey;
import java.security.PublicKey;
import java.security.SecureRandom;
import java.security.Security;

import java.nio.ByteBuffer;
import java.nio.ByteOrder;

import java.util.Arrays;


// Definition of all the required constant values.

// Definition of the size of a byte
// with the respective number of bits.
final short BYTE_SIZE_IN_BITS = 8;

// Definition of the number of possible
// binary digits (bits) for a hexadecimal character.
final short NUM_POSSIBLE_BITS_IN_HEXADECIMAL_CHAR = 16;



// Definition of all the required classes.

// Definition of the class of the cryptographic engine's parameters
// for the Public-Key (Asymmetric) Cryptosystem CRYSTALS-Kyber
// (CRYptographic SuiTe for Algebraic LatticeS - Kyber),
// which is a (Classical) Post-Quantum Cryptosystem
// (supposed to be secure against classical and quantum attacks),
// and based on computational hardness of mathematical problems
// derived from the well-known lattices algebraic structures,
// most specifically, in the LWE (Learning With Errors) Problem.
class KyberEngineParams {
    
    // Definition of all the required constant values,
    // regarding the class of the cryptographic engine's
    // parameters for the CRYSTALS-Kyber Public-Key Cryptosystem.
    
    // Definition of the maximum degree n of
    // the polynomials used in the matrix A representing
    // the lattice algebraic structure to be adopted for
    // the CRYSTALS-Kyber Public-Key Cryptosystem.
    public final static int param_n = 256;
    
    // Definition of the numeric modulus q for
    // the polynomials used in the matrix A representing
    // the lattice algebraic structure to be adopted for
    // the CRYSTALS-Kyber Public-Key Cryptosystem.
    public final static int param_q = 3329;
    
    // Definition of the numeric inverse of modulus q
    // for the polynomials used in the matrix A representing
    // the lattice algebraic structure to be adopted for
    // the CRYSTALS-Kyber Public-Key Cryptosystem.
    public final static int param_q_inv = 62209;

    // Definition of the compression factor d_t
    // to reduce/compress the size of the asymmetric keys to be
    // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public final static int compression_asymmetric_keys_factor_d_t = 12;
    
    // Definition of the numeric noise factor eta_2
    // used for the random error vectors e_1 and e_2 extracted
    // from a (Centered) Binomial Probability Distribution,
    // applied during the asymmetric encryption and generation of
    // the KEM (Key Encapsulation Method/Mechanism) procedures
    // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public final static int param_eta_2 = 2;

    // Definition of the size of the symmetric
    // hashes and (pseudo) random seeds, in terms of bytes,
    // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public final static int num_symmetric_bytes = 32;
    
    // Definition of the size of the symmetric
    // shared secret, in terms of bytes, to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem.
    public final static int num_shared_secret_bytes = 32;
    
    // Definition of the size a polynomial, in terms of bytes,
    // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public final static int num_polynomial_bytes = 
            ( ( compression_asymmetric_keys_factor_d_t * param_n ) / 
              BYTE_SIZE_IN_BITS );

    // Definition of the size of an IND-CPA (INDistinguishable
    // under Chosen Plaintext Attack) message, in terms of bytes,
    // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    private final static int num_ind_cpa_message_bytes = 32;


    // Definition of all the required variable values,
    // regarding the class of the cryptographic engine's
    // parameters for the CRYSTALS-Kyber Public-Key Cryptosystem.
    
    // Definition of the number of polynomials k
    // per vector used in the matrix A representing
    // the lattice algebraic structure, representing as well,
    // the size factor for that same lattice algebraic structure,
    // and thus, being the main security parameter in terms of scaling
    // the security strength of the CRYSTALS-Kyber Public-Key Cryptosystem.
    private final int param_k;
    
    // Definition of the size, in terms of bytes,
    // of a polynomial compressed in an array of bytes, to be
    // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    private final int num_polynomial_compressed_bytes;
    
    // Definition of the size, in terms of bytes,
    // of a polynomials vector in a matrix of bytes,
    // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    private final int num_polynomials_vector_bytes;
    
    // Definition of the size, in terms of bytes,
    // of a polynomials vector compressed in a matrix of bytes,
    // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    private final int num_polynomials_vector_compressed_bytes;
    
    // Definition of the numeric noise factor eta_1
    // used in the secret random coins vector r
    // extracted from a Binomial Probability Distribution,
    // dependent on the public key k_pub, and also
    // used for the error vector e and secret vector s,
    // regarding the polynomials representing a lattice
    // algebraic structure for the mathematical equation
    // A x s + e = t), representing a lattice-based problem in
    // the form of a MLWE (Module-Learning With Errors), to be
    // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    private final int param_eta_1;
    
    // Definition of the compression factor d_u
    // to reduce/compress the size of the ciphertext
    // generated from the asymmetric encryption and to be
    // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    private final int compression_asymmetric_encryption_factor_d_u;

    // Definition of the compression factor d_v
    // to reduce/compress the size of the ciphertext
    // generated from the asymmetric encryption and to be
    // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    private final int compression_asymmetric_encryption_factor_d_v;
    
    // Definition of the size, in terms of bytes,
    // of the asymmetric public key to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem
    // with the IND-CPA (INDistinguishability
    // under Chosen Plaintext Attack) property.
    private final int num_ind_cpa_public_key_bytes;
    
    // Definition of the size, in terms of bytes,
    // of the asymmetric private key to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem
    // with the IND-CPA (INDistinguishability
    // under Chosen Plaintext Attack) property.
    private final int num_ind_cpa_secret_key_bytes;
    
    // Definition of the size, in terms of bytes,
    // of the numeric polynomial, which is related to
    // the asymmetric encryption to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem
    // with the IND-CPA (INDistinguishability
    // under Chosen Plaintext Attack) property.
    private final int num_ind_cpa_polynomial_bytes;
    
    // Definition of the size, in terms of bytes,
    // of the asymmetric public key to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem's
    // KEM (Key Encapsulation Method/Mechanism)
    // with the IND-CCA2 (INDistinguishability
    // under adaptive Chosen Ciphertext Attack) property.
    private final int num_ind_cca_kem_public_key_bytes;
    
    // Definition of the size, in terms of bytes,
    // of the asymmetric private key to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem's
    // KEM (Key Encapsulation Method/Mechanism)
    // with the IND-CCA2 (INDistinguishability
    // under adaptive Chosen Ciphertext Attack) property.
    private final int num_ind_cca_kem_secret_key_bytes;
    
    // Definition of the size, in terms of bytes,
    // of the ciphertext generated and to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem's
    // KEM (Key Encapsulation Method/Mechanism)
    // with the IND-CCA2 (INDistinguishability
    // under adaptive Chosen Ciphertext Attack) property.
    private final int num_ind_cca_kem_ciphertext_bytes;
    
    // Definition of the size of the symmetric session key, in terms of bytes,
    // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    private final int num_session_key_bytes;
    
    
    // Constructors of the class of the cryptographic engine's
    // parameters for the CRYSTALS-Kyber Public-Key Cryptosystem.
    
    // Constructor of the class of the cryptographic engine's
    // parameters for the CRYSTALS-Kyber Public-Key Cryptosystem,
    // and the initialization of the initial required parameters.
    //    Parameters:
    //    @param param_k: The number of polynomials k
    //                    per vector used in the matrix A
    //                    representing the lattice algebraic
    //                    structure, representing as well,
    //                    the size factor for that same
    //                    lattice algebraic structure.
    public KyberEngineParams( int param_k ) {

        // Setting of the number of polynomials k
        // per vector used in the matrix A representing
        // the lattice algebraic structure, representing as well,
        // the size factor for that same lattice algebraic structure,
        // and thus, being the main security parameter in terms of scaling
        // the security strength of the CRYSTALS-Kyber Public-Key Cryptosystem.
        this.param_k = param_k;
    
        
        // Setting a switch-case code block for all the possible values
        // for the number of polynomials k per vector used in the matrix A,
        // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        switch( this.param_k ) {
        
            // If the number of polynomials k per vector used in the matrix A,
            // used by the CRYSTALS-Kyber Public-Key Cryptosystem, is set as 2.
            case 2:
                
                // Setting of the numeric noise factor eta_1
                // used in the secret random coins vector r
                // extracted from a Binomial Probability Distribution,
                // dependent on the public key k_pub, and also
                // used for the error vector e and secret vector s,
                // regarding the polynomials representing a lattice
                // algebraic structure for the mathematical equation
                // A x s + e = t), representing a lattice-based problem in
                // the form of a MLWE (Module-Learning With Errors), to be
                // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
                this.param_eta_1 = 3;
                
                // Setting of the compression factor d_u
                // to reduce/compress the size of the ciphertext
                // generated from the asymmetric encryption and to be
                // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
                this.compression_asymmetric_encryption_factor_d_u = 10;

                // Setting of the compression factor d_v
                // to reduce/compress the size of the ciphertext
                // generated from the asymmetric encryption and to be
                // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
                this.compression_asymmetric_encryption_factor_d_v = 4;
                
                // Setting of the size of the symmetric session key, in terms of bytes,
                // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
                this.num_session_key_bytes = 16;
                
                // Breaking of the case code block.
                break;
            
            // If the number of polynomials k per vector used in the matrix A,
            // used by the CRYSTALS-Kyber Public-Key Cryptosystem, is set as 3.
            case 3:
                
                // Setting of the numeric noise factor eta_1
                // used in the secret random coins vector r
                // extracted from a Binomial Probability Distribution,
                // dependent on the public key k_pub, and also
                // used for the error vector e and secret vector s,
                // regarding the polynomials representing a lattice
                // algebraic structure for the mathematical equation
                // A x s + e = t), representing a lattice-based problem in
                // the form of a MLWE (Module-Learning With Errors), to be
                // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
                this.param_eta_1 = 2;
                
                // Setting of the compression factor d_u
                // to reduce/compress the size of the ciphertext
                // generated from the asymmetric encryption and to be
                // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
                this.compression_asymmetric_encryption_factor_d_u = 10;

                // Setting of the compression factor d_v
                // to reduce/compress the size of the ciphertext
                // generated from the asymmetric encryption and to be
                // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
                this.compression_asymmetric_encryption_factor_d_v = 4;
                
                // Setting of the size of the symmetric session key, in terms of bytes,
                // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
                this.num_session_key_bytes = 24;
                
                // Breaking of the case code block.
                break;
            
            // If the number of polynomials k per vector used in the matrix A,
            // used by the CRYSTALS-Kyber Public-Key Cryptosystem, is set as 4.
            case 4:
                
                // Setting of the numeric noise factor eta_1
                // used in the secret random coins vector r
                // extracted from a Binomial Probability Distribution,
                // dependent on the public key k_pub, and also
                // used for the error vector e and secret vector s,
                // regarding the polynomials representing a lattice
                // algebraic structure for the mathematical equation
                // A x s + e = t), representing a lattice-based problem in
                // the form of a MLWE (Module-Learning With Errors), to be
                // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
                this.param_eta_1 = 2;
                
                // Setting of the compression factor d_u
                // to reduce/compress the size of the ciphertext
                // generated from the asymmetric encryption and to be
                // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
                this.compression_asymmetric_encryption_factor_d_u = 11;

                // Setting of the compression factor d_v
                // to reduce/compress the size of the ciphertext
                // generated from the asymmetric encryption and to be
                // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
                this.compression_asymmetric_encryption_factor_d_v = 5;
                
                // Setting of the size of the symmetric session key, in terms of bytes,
                // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
                this.num_session_key_bytes = 32;
                
                // Breaking of the case code block.
                break;
            
            // If the number of polynomials k per vector used in the matrix A,
            // used by the CRYSTALS-Kyber Public-Key Cryptosystem, has an invalid value.
            default:
                
                // Throwing of an IllegalArgumentException, regarding an invalid value
                // for the number of polynomials k per vector used in the matrix A,
                // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
                throw new IllegalArgumentException("Error: k=" + this.param_k + " " +
                                                   "is not supported for CRYSTALS-Kyber (Asymmetric) " +
                                                   "Public-Key Cryptosystem!");
        
        }
        
        
        // (Calculation) and setting of the size, in terms of bytes,
        // of a polynomial compressed in an array of bytes, to be
        // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        this.num_polynomial_compressed_bytes = 
            ( ( this.compression_asymmetric_encryption_factor_d_v * 
                this.param_k * this.param_n ) / BYTE_SIZE_IN_BITS );
        
        // (Calculation) and setting of the size, in terms of bytes,
        // of a polynomials vector in a matrix of bytes,
        // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        this.num_polynomials_vector_bytes = 
            ( this.param_k * this.num_polynomial_bytes );
        
        // (Calculation) and setting of the size, in terms of bytes,
        // of a polynomials vector compressed in a matrix of bytes,
        // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        this.num_polynomials_vector_compressed_bytes = 
            ( ( this.compression_asymmetric_encryption_factor_d_u * 
                this.param_k * this.param_n ) / BYTE_SIZE_IN_BITS );
        
        // (Calculation) and setting of the size,
        // in terms of bytes, of the asymmetric public key
        // to be used by the CRYSTALS-Kyber Public-Key
        // Cryptosystem with the IND-CPA (INDistinguishability
        // under Chosen Plaintext Attack) property.
        this.num_ind_cpa_public_key_bytes = 
            ( this.num_polynomials_vector_bytes + this.num_symmetric_bytes );
        
        // (Calculation) and setting of the size,
        // in terms of bytes, of the asymmetric private key
        // to be used by the CRYSTALS-Kyber Public-Key
        // Cryptosystem with the IND-CPA (INDistinguishability
        // under Chosen Plaintext Attack) property.
        this.num_ind_cpa_secret_key_bytes = this.num_polynomials_vector_bytes;
        
        // (Calculation) and setting of the size, in terms of bytes,
        // of the numeric polynomial, which is related to the asymmetric
        // encryption to be used by the CRYSTALS-Kyber Public-Key
        // Cryptosystem with the IND-CPA (INDistinguishability
        // under Chosen Plaintext Attack) property.
        this.num_ind_cpa_polynomial_bytes = 
            ( this.num_polynomials_vector_compressed_bytes + 
              this.num_polynomial_compressed_bytes );

        // (Calculation) and setting of the size,
        // in terms of bytes, of the asymmetric public key
        // to be used by the CRYSTALS-Kyber Public-Key
        // Cryptosystem's KEM (Key Encapsulation Method/Mechanism)
        // with the IND-CCA2 (INDistinguishability under
        // adaptive Chosen Ciphertext Attack) property.
        this.num_ind_cca_kem_public_key_bytes = this.num_ind_cpa_public_key_bytes;
        
        // (Calculation) and setting of the size,
        // in terms of bytes, of the asymmetric private key
        // to be used by the CRYSTALS-Kyber Public-Key
        // Cryptosystem's KEM (Key Encapsulation Method/Mechanism)
        // with the IND-CCA2 (INDistinguishability under
        // adaptive Chosen Ciphertext Attack) property.
        this.num_ind_cca_kem_secret_key_bytes = 
            ( this.num_ind_cpa_secret_key_bytes + this.num_ind_cpa_public_key_bytes + 
              ( 2 * this.num_symmetric_bytes ) );
        
        // (Calculation) and setting of the size,
        // in terms of bytes, of the ciphertext generated
        // and to be used by the CRYSTALS-Kyber Public-Key
        // Cryptosystem's KEM (Key Encapsulation Method/Mechanism)
        // with the IND-CCA2 (INDistinguishability under
        // adaptive Chosen Ciphertext Attack) property.
        this.num_ind_cca_kem_ciphertext_bytes = this.num_ind_cpa_polynomial_bytes;
    
    }
    
    
    // Methods of the class of the cryptographic engine's
    // parameters for the CRYSTALS-Kyber Public-Key Cryptosystem.
    
    // Method to obtain the maximum degree n of
    // the polynomials used in the matrix A representing
    // the lattice algebraic structure to be adopted for
    // the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_param_n() {
        
        // Return of the maximum degree n of
        // the polynomials used in the matrix A representing
        // the lattice algebraic structure to be adopted for
        // the CRYSTALS-Kyber Public-Key Cryptosystem.
        return this.param_n;
        
    }
    
    // Method to obtain the numeric modulus q for
    // the polynomials used in the matrix A representing
    // the lattice algebraic structure to be adopted for
    // the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_param_q() {
        
        // Return of the numeric modulus q for
        // the polynomials used in the matrix A representing
        // the lattice algebraic structure to be adopted for
        // the CRYSTALS-Kyber Public-Key Cryptosystem.
        return this.param_q;
        
    }
    
    // Method to obtain the numeric inverse of modulus q
    // for the polynomials used in the matrix A representing
    // the lattice algebraic structure to be adopted for
    // the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_param_q_inv() {
        
        // Return of the numeric inverse of modulus q
        // for the polynomials used in the matrix A representing
        // the lattice algebraic structure to be adopted for
        // the CRYSTALS-Kyber Public-Key Cryptosystem.
        return this.param_q_inv;
        
    }
    
    // Method to obtain the compression factor d_t
    // to reduce/compress the size of the asymmetric keys to be
    // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_compression_asymmetric_keys_factor_d_t() {
        
        // Return of the compression factor d_t
        // to reduce/compress the size of the asymmetric keys to be
        // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        return this.compression_asymmetric_keys_factor_d_t;
        
    }
    
    // Method to obtain the numeric noise factor eta_1
    // used in the secret random coins vector r
    // extracted from a Binomial Probability Distribution,
    // dependent on the public key k_pub, and also
    // used for the error vector e and secret vector s,
    // regarding the polynomials representing a lattice
    // algebraic structure for the mathematical equation
    // A x s + e = t), representing a lattice-based problem in
    // the form of a MLWE (Module-Learning With Errors), to be
    // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_param_eta_1() {
        
        // Return of the numeric noise factor eta_1
        // used in the secret random coins vector r
        // extracted from a Binomial Probability Distribution,
        // dependent on the public key k_pub, and also
        // used for the error vector e and secret vector s,
        // regarding the polynomials representing a lattice
        // algebraic structure for the mathematical equation
        // A x s + e = t), representing a lattice-based problem in
        // the form of a MLWE (Module-Learning With Errors), to be
        // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        return this.param_eta_1;
        
    }

    // Method to obtain the numeric noise factor eta_2
    // used for the random error vectors e_1 and e_2 extracted
    // from a (Centered) Binomial Probability Distribution,
    // applied during the asymmetric encryption and generation of
    // the KEM (Key Encapsulation Method/Mechanism) procedures
    // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_param_eta_2() {
        
        // Return of the numeric noise factor eta_2
        // used for the random error vectors e_1 and e_2 extracted
        // from a (Centered) Binomial Probability Distribution,
        // applied during the asymmetric encryption and generation of
        // the KEM (Key Encapsulation Method/Mechanism) procedures
        // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        return this.param_eta_2;
        
    }
    
    // Method to obtain the size of the symmetric
    // hashes and (pseudo) random seeds, in terms of bytes,
    // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_num_symmetric_bytes() {
        
        // Return of the size of the symmetric
        // hashes and (pseudo) random seeds, in terms of bytes,
        // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        return this.num_symmetric_bytes;
        
    }
    
    // Method to obtain the size of the symmetric
    // hashes and (pseudo) random seeds, in terms of bits,
    // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_num_symmetric_bits() {
        
        // Return of the size of the symmetric
        // hashes and (pseudo) random seeds, in terms of bits,
        // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        return ( this.get_num_symmetric_bytes() *
                 BYTE_SIZE_IN_BITS );
        
    }
    
    // Method to obtain the size of the symmetric
    // shared secret, in terms of bytes, to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_num_shared_secret_bytes() {
        
        // Return of the size of the symmetric
        // shared secret, in terms of bytes, to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem.
        return this.num_shared_secret_bytes;
        
    }
    
    // Method to obtain the size of the symmetric
    // shared secret, in terms of bits, to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_num_shared_secret_bits() {
        
        // Return of the size of the symmetric
        // shared secret, in terms of bits, to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem.
        return ( this.get_num_shared_secret_bytes() *
                 BYTE_SIZE_IN_BITS );
        
    }
    
    // Method to obtain the size a polynomial, in terms of bytes,
    // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_num_polynomial_bytes() {
        
        // Return of the size a polynomial, in terms of bytes,
        // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        return this.num_polynomial_bytes;
        
    }
    
    // Method to obtain the size a polynomial, in terms of bits,
    // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_num_polynomial_bits() {
        
        // Return of the size a polynomial, in terms of bits,
        // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        return ( this.get_num_polynomial_bytes() *
                 BYTE_SIZE_IN_BITS );
        
    }
    
    // Method to obtain the size of an IND-CPA (INDistinguishable
    // under Chosen Plaintext Attack) message, in terms of bytes,
    // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_num_ind_cpa_message_bytes() {
        
        // Return of the size of an IND-CPA (INDistinguishable
        // under Chosen Plaintext Attack) message, in terms of bytes,
        // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        return this.num_ind_cpa_message_bytes;
        
    }
    
    // Method to obtain the size of an IND-CPA (INDistinguishable
    // under Chosen Plaintext Attack) message, in terms of bits,
    // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_num_ind_cpa_message_bits() {
        
        // Return of the size of an IND-CPA (INDistinguishable
        // under Chosen Plaintext Attack) message, in terms of bits,
        // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        return ( this.get_num_ind_cpa_message_bytes() *
                 BYTE_SIZE_IN_BITS );
        
    }
    
    // Method to obtain the number of polynomials k
    // per vector used in the matrix A representing
    // the lattice algebraic structure, representing
    // as well, the size factor for that same lattice
    // algebraic structure, and thus, being the main
    // security parameter in terms of scaling the security
    // strength of the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_param_k() {
        
        // Return of the number of polynomials k
        // per vector used in the matrix A representing
        // the lattice algebraic structure, representing
        // as well, the size factor for that same lattice
        // algebraic structure, and thus, being the main
        // security parameter in terms of scaling the security
        // strength of the CRYSTALS-Kyber Public-Key Cryptosystem.
        return this.param_k;
        
    }
    
    // Method to obtain the size, in terms of bytes,
    // of a polynomial compressed in an array of bytes, to be
    // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_num_polynomial_compressed_bytes() {
        
        // Return of the size, in terms of bytes,
        // of a polynomial compressed in an array of bytes, to be
        // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        return this.num_polynomial_compressed_bytes;
        
    }
    
    // Method to obtain the size, in terms of bits,
    // of a polynomial compressed in an array of bytes, to be
    // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_num_polynomial_compressed_bits() {
        
        // Return of the size, in terms of bits,
        // of a polynomial compressed in an array of bytes, to be
        // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        return ( this.num_polynomial_compressed_bytes *
                 BYTE_SIZE_IN_BITS );
        
    }
    
    // Method to obtain the size, in terms of bytes,
    // of a polynomials vector in a matrix of bytes,
    // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_num_polynomials_vector_bytes() {
        
        // Return of the size, in terms of bytes,
        // of a polynomials vector in a matrix of bytes,
        // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        return this.num_polynomials_vector_bytes;
        
    }
    
    // Method to obtain the size, in terms of bits,
    // of a polynomials vector in a matrix of bytes,
    // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_num_polynomials_vector_bits() {
        
        // Return of the size, in terms of bits,
        // of a polynomials vector in a matrix of bytes,
        // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        return ( this.get_num_polynomials_vector_bytes() * 
                 BYTE_SIZE_IN_BITS );
        
    }
    
    // Method to obtain the size, in terms of bytes,
    // of a polynomials vector compressed in a matrix of bytes,
    // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem. 
    public int get_num_polynomials_vector_compressed_bytes() {
        
        // Return of the size, in terms of bytes,
        // of a polynomials vector compressed in a matrix of bytes,
        // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        return this.num_polynomials_vector_compressed_bytes;
        
    }
    
    // Method to obtain the size, in terms of bits,
    // of a polynomials vector compressed in a matrix of bytes,
    // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem. 
    public int get_num_polynomials_vector_compressed_bits() {
        
        // Return of the size, in terms of bits,
        // of a polynomials vector compressed in a matrix of bytes,
        // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        return ( this.get_num_polynomials_vector_compressed_bytes() *
                 BYTE_SIZE_IN_BITS );
        
    }
    
    // Method to obtain the compression factor d_u
    // to reduce/compress the size of the ciphertext
    // generated from the asymmetric encryption and to be
    // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_compression_asymmetric_encryption_factor_d_u() {
        
        // Return of the compression factor d_u
        // to reduce/compress the size of the ciphertext
        // generated from the asymmetric encryption and to be
        // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        return this.compression_asymmetric_encryption_factor_d_u;
        
    }

    // Method to obtain the compression factor d_v
    // to reduce/compress the size of the ciphertext
    // generated from the asymmetric encryption and to be
    // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_compression_asymmetric_encryption_factor_d_v() {
        
        // Return of the compression factor d_v
        // to reduce/compress the size of the ciphertext
        // generated from the asymmetric encryption and to be
        // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        return this.compression_asymmetric_encryption_factor_d_v;
        
    }
    
    // Method to obtain the size, in terms of bytes,
    // of the asymmetric public key to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem
    // with the IND-CPA (INDistinguishability
    // under Chosen Plaintext Attack) property.
    public int get_num_ind_cpa_public_key_bytes() {
        
        // Return of the size, in terms of bytes,
        // of the asymmetric public key to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem
        // with the IND-CPA (INDistinguishability
        // under Chosen Plaintext Attack) property.
        return this.num_ind_cpa_public_key_bytes;
        
    }
    
    // Method to obtain the size, in terms of bits,
    // of the asymmetric public key to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem
    // with the IND-CPA (INDistinguishability
    // under Chosen Plaintext Attack) property.
    public int get_num_ind_cpa_public_key_bits() {
        
        // Return of the size, in terms of bits,
        // of the asymmetric public key to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem
        // with the IND-CPA (INDistinguishability
        // under Chosen Plaintext Attack) property.
        return ( this.get_num_ind_cpa_public_key_bytes() * 
                 BYTE_SIZE_IN_BITS );
        
    }
    
    // Method to obtain the size, in terms of bytes,
    // of the asymmetric private key to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem
    // with the IND-CPA (INDistinguishability
    // under Chosen Plaintext Attack) property.
    public int get_num_ind_cpa_secret_key_bytes() {
        
        // Return of the size, in terms of bytes,
        // of the asymmetric private key to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem
        // with the IND-CPA (INDistinguishability
        // under Chosen Plaintext Attack) property.
        return this.num_ind_cpa_secret_key_bytes;
        
    }
    
    // Method to obtain the size, in terms of bits,
    // of the asymmetric private key to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem
    // with the IND-CPA (INDistinguishability
    // under Chosen Plaintext Attack) property.
    public int get_num_ind_cpa_secret_key_bits() {
        
        // Return of the size, in terms of bits,
        // of the asymmetric private key to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem
        // with the IND-CPA (INDistinguishability
        // under Chosen Plaintext Attack) property.
        return ( this.num_ind_cpa_secret_key_bytes * 
                 BYTE_SIZE_IN_BITS );
        
    }
    
    // Method to obtain the size, in terms of bytes,
    // of the numeric polynomial, which is related to
    // the asymmetric encryption to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem
    // with the IND-CPA (INDistinguishability
    // under Chosen Plaintext Attack) property.
    public int get_num_ind_cpa_polynomial_bytes() {
        
        // Return of the size, in terms of bytes,
        // of the numeric polynomial, which is related to
        // the asymmetric encryption to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem
        // with the IND-CPA (INDistinguishability
        // under Chosen Plaintext Attack) property.
        return this.num_ind_cpa_polynomial_bytes;
        
    }
    
    // Method to obtain the size, in terms of bits,
    // of the numeric polynomial, which is related to
    // the asymmetric encryption to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem
    // with the IND-CPA (INDistinguishability
    // under Chosen Plaintext Attack) property.
    public int get_num_ind_cpa_polynomial_bits() {
        
        // Return of the size, in terms of bits,
        // of the numeric polynomial, which is related to
        // the asymmetric encryption to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem
        // with the IND-CPA (INDistinguishability
        // under Chosen Plaintext Attack) property.
        return ( this.get_num_ind_cpa_polynomial_bytes() *
                 BYTE_SIZE_IN_BITS );
        
    }
    
    // Method to obtain the size, in terms of bytes,
    // of the asymmetric public key to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem's
    // KEM (Key Encapsulation Method/Mechanism)
    // with the IND-CCA2 (INDistinguishability
    // under adaptive Chosen Ciphertext Attack) property.
    public int get_num_ind_cca_kem_public_key_bytes() {
        
        // Return of the size, in terms of bytes,
        // of the asymmetric public key to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem's
        // KEM (Key Encapsulation Method/Mechanism)
        // with the IND-CCA2 (INDistinguishability
        // under adaptive Chosen Ciphertext Attack) property.
        return this.num_ind_cca_kem_public_key_bytes;
        
    }
    
    // Method to obtain the size, in terms of bits,
    // of the asymmetric public key to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem's
    // KEM (Key Encapsulation Method/Mechanism)
    // with the IND-CCA2 (INDistinguishability
    // under adaptive Chosen Ciphertext Attack) property.
    public int get_num_ind_cca_kem_public_key_bits() {
        
        // Return of the size, in terms of bits,
        // of the asymmetric public key to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem's
        // KEM (Key Encapsulation Method/Mechanism)
        // with the IND-CCA2 (INDistinguishability
        // under adaptive Chosen Ciphertext Attack) property.
        return ( this.get_num_ind_cca_kem_public_key_bytes() *
                 BYTE_SIZE_IN_BITS );
        
    }
    
    // Method to obtain the size, in terms of bytes,
    // of the asymmetric private key to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem's
    // KEM (Key Encapsulation Method/Mechanism)
    // with the IND-CCA2 (INDistinguishability
    // under adaptive Chosen Ciphertext Attack) property.
    public int get_num_ind_cca_kem_secret_key_bytes() {
        
        // Return of the size, in terms of bytes,
        // of the asymmetric private key to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem's
        // KEM (Key Encapsulation Method/Mechanism)
        // with the IND-CCA2 (INDistinguishability
        // under adaptive Chosen Ciphertext Attack) property.
        return this.num_ind_cca_kem_secret_key_bytes;
        
    }
    
    // Method to obtain the size, in terms of bits,
    // of the asymmetric private key to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem's
    // KEM (Key Encapsulation Method/Mechanism)
    // with the IND-CCA2 (INDistinguishability
    // under adaptive Chosen Ciphertext Attack) property.
    public int get_num_ind_cca_kem_secret_key_bits() {
        
        // Return of the size, in terms of bits,
        // of the asymmetric private key to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem's
        // KEM (Key Encapsulation Method/Mechanism)
        // with the IND-CCA2 (INDistinguishability
        // under adaptive Chosen Ciphertext Attack) property.
        return ( this.get_num_ind_cca_kem_secret_key_bytes() *
                 BYTE_SIZE_IN_BITS );
        
    }
    
    // Method to obtain the size, in terms of bytes,
    // of the ciphertext generated and to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem's
    // KEM (Key Encapsulation Method/Mechanism)
    // with the IND-CCA2 (INDistinguishability
    // under adaptive Chosen Ciphertext Attack) property.
    public int get_num_ind_cca_kem_ciphertext_bytes() {
        
        // Return of the size, in terms of bytes,
        // of the ciphertext generated and to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem's
        // KEM (Key Encapsulation Method/Mechanism)
        // with the IND-CCA2 (INDistinguishability
        // under adaptive Chosen Ciphertext Attack) property.
        return this.num_ind_cca_kem_ciphertext_bytes;
        
    }
    
    // Method to obtain the size, in terms of bits,
    // of the ciphertext generated and to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem's
    // KEM (Key Encapsulation Method/Mechanism)
    // with the IND-CCA2 (INDistinguishability
    // under adaptive Chosen Ciphertext Attack) property.
    public int get_num_ind_cca_kem_ciphertext_bits() {
        
        // Return of the size, in terms of bits,
        // of the ciphertext generated and to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem's
        // KEM (Key Encapsulation Method/Mechanism)
        // with the IND-CCA2 (INDistinguishability
        // under adaptive Chosen Ciphertext Attack) property.
        return ( this.get_num_ind_cca_kem_ciphertext_bytes() *
                 BYTE_SIZE_IN_BITS );
        
    }
    
    // Method to obtain the size of the symmetric session key, in terms of bytes,
    // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_num_session_key_bytes() {
        
        // Return of the size of the symmetric session key, in terms of bytes,
        // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        return this.num_session_key_bytes;
        
    }
    
    // Method to obtain the size of the symmetric session key, in terms of bits,
    // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_num_session_key_bits() {
        
        // Return of the size of the symmetric session key, in terms of bits,
        // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        return ( this.get_num_session_key_bytes() *
                 BYTE_SIZE_IN_BITS );
        
    }
        
}


// Definition of class of Public-Key (Asymmetric) Cryptosystem
// CRYSTALS-Kyber (CRYptographic SuiTe for Algebraic LatticeS - Kyber),
// which is a (Classical) Post-Quantum Cryptosystem
// (supposed to be secure against classical and quantum attacks),
// and based on computational hardness of mathematical problems
// derived from the well-known lattices algebraic structures,
// most specifically, in the LWE (Learning With Errors) Problem.
class CrystalsKyberCryptosystem {
    
    // Definition of all the required constant values,
    // regarding the class of the CRYSTALS-Kyber Public-Key Cryptosystem.
    
    // Setting of the string array with the names of
    // the CRYSTALS-Kyber Public-Key Cryptosystem's
    // Security Parameters for the key pair generation
    // (consisting on private and public keys), as well as
    // the respective Key Encapsulation Method/Mechanism (KEM).
    public final String[]
        security_parameters_names = {
    
        // 1) Security Parameters Set with a factor of 2 for
        //    the size of the lattice algebraic structure, which provides
        //    a (post-quantum) security strength of TODO bits, designed for
        //    the distribution of symmetric session keys of 128 bits.
        "Kyber512",
        
        // 2) Security Parameters Set with a factor of 3 for
        //    the size of the lattice algebraic structure, which provides
        //    a (post-quantum) security strength of TODO bits, designed for
        //    the distribution of symmetric session keys of 192 bits.
        "Kyber768",
        
        // 3) Security Parameters Set with a factor of 4 for
        //    the size of the lattice algebraic structure, which provides
        //    a (post-quantum) security strength of TODO bits, designed for
        //    the distribution of symmetric session keys of 256 bits.
        "Kyber1024"
    
    };
    
    // Setting of the string array with the names and
    // definitions of the CRYSTALS-Kyber Public-Key Cryptosystem's
    // Security Parameters for the key pair generation
    // (consisting on private and public keys), as well as
    // the respective Key Encapsulation Method/Mechanism (KEM).
    public final String[] 
        security_parameters_names_and_descriptions = {
    
        // 1) Security Parameters Set with a factor of k = 2 ( k x n = 2 x 256 = 512 )
        //    for the size of the lattice algebraic structure, which provides
        //    a (post-quantum) security strength of 64 bits, designed for
        //    the distribution of symmetric session keys of 128 bits.
        //    NOTE: The Kyber512 has a NIST (Classical) Post-Quantum
        //          Security Level of 1 (≈ exhaustive search on AES-128).
        "=> Kyber512: Security Parameters Set with a factor of k = 2 ( k x n = 2 x 256 = 512 )\n   " +
        "             for the size of the lattice algebraic structure, which provides\n   " +
        "             a (post-quantum) security strength of 64 bits, designed for\n   " +
        "             the distribution of symmetric session keys of 128 bits.\n\n   " +
        "             NOTE: The Kyber512 Cryptosystem has a NIST (Classical)\n   " +
        "                   Post-Quantum Security Level of 1 (≈ exhaustive search on AES-128).",
        
        // 2) Security Parameters Set with a factor of k = 3 ( k x n = 3 x 256 = 768 )
        //    for the size of the lattice algebraic structure, which provides
        //    a (post-quantum) security strength of 96 bits, designed for
        //    the distribution of symmetric session keys of 192 bits.
        //    NOTE: The Kyber512 has a NIST (Classical) Post-Quantum
        //          Security Level of 1 (≈ exhaustive search on AES-192).
        "=> Kyber768: Security Parameters Set with a factor of k = 3 ( k x n = 3 x 256 = 768 )\n   " +
        "             for the size of the lattice algebraic structure, which provides\n   " +
        "             a (post-quantum) security strength of 96 bits, designed for\n   " +
        "             the distribution of symmetric session keys of 192 bits.\n\n   " +
        "             NOTE: The Kyber768 Cryptosystem has a NIST (Classical)\n   " +
        "                   Post-Quantum Security Level of 3 (≈ exhaustive search on AES-192).",
        
        // 3) Security Parameters Set with a factor of k = 4 ( k x n = 4 x 256 = 1024 )
        //    for the size of the lattice algebraic structure, which provides
        //    a (post-quantum) security strength of 128 bits, designed for
        //    the distribution of symmetric session keys of 256 bits.
        //    NOTE: The Kyber512 has a NIST (Classical) Post-Quantum
        //          Security Level of 1 (≈ exhaustive search on AES-256).
        "=> Kyber1024: Security Parameters Set with a factor of k = 4 ( k x n = 4 x 256 = 1024 )\n   " +
        "              for the size of the lattice algebraic structure, which provides\n   " +
        "              a (post-quantum) security strength of 128 bits, designed for\n   " +
        "              the distribution of symmetric session keys of 256 bits.\n\n   " +
        "              NOTE: The Kyber1024 Cryptosystem has a NIST (Classical)\n   " +
        "                    Post-Quantum Security Level of 5 (≈ exhaustive search on AES-256)."
        
    };
    
    
    // Setting of the CRYSTALS-Kyber Public-Key Cryptosystem's
    // Security Parameters for the key pair generation
    // (consisting on private and public keys), as well as
    // the respective Key Encapsulation Method/Mechanism (KEM).
    public final KyberParameterSpec[] 
        security_parameters_specifications = {
        
        // 1) Security Parameters Set with a factor of 2 for
        //    the size of the lattice algebraic structure, which provides
        //    a (post-quantum) security strength of TODO bits, designed for
        //    the distribution of symmetric session keys of 128 bits.
        KyberParameterSpec.kyber512,
        
        // 2) Security Parameters Set with a factor of 3 for
        //    the size of the lattice algebraic structure, which provides
        //    a (post-quantum) security strength of TODO bits, designed for
        //    the distribution of symmetric session keys of 192 bits.
        KyberParameterSpec.kyber768,
        
        // 3) Security Parameters Set with a factor of 4 for
        //    the size of the lattice algebraic structure, which provides
        //    a (post-quantum) security strength of TODO bits, designed for
        //    the distribution of symmetric session keys of 256 bits.
        KyberParameterSpec.kyber1024
        
    };
    
    
    // Setting of the array of the parameters
    // regarding the number of polynomials k
    // per vector used in the matrix A representing
    // the lattice algebraic structure, representing as well,
    // the size factor for that same lattice algebraic structure,
    // and thus, being the main security parameter in terms of scaling
    // the security strength of the CRYSTALS-Kyber Public-Key Cryptosystem.
    public final int[] engine_k_parameters = {
        
        // 1) Parameter k=2 to build the 'Kyber512' Security Parameter Set
        //    as the parameter choice for the general parameters to be
        //    used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        2,
        
        // 2) Parameters k=3 to build the 'Kyber768' Security Parameter Set
        //    as the parameter choice for the general parameters to be
        //    used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        3,
        
        // 3) Parameters k=4 to build the 'Kyber1024' Security Parameter Set
        //    as the parameter choice for the general parameters to be
        //    used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        4
        
    };
    
    
    // Definition of all the required variable values,
    // regarding the class of the CRYSTALS-Kyber Public-Key Cryptosystem.
    
    // Definition of the parameter choice index for
    // the generation of asymmetric keys to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem.
    private short parameter_choice_index;
    
    // Definition of the general parameters to be
    // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    private KyberParameters parameters;
    
    // Definition of the cryptographic engine's parameters to be
    // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    private KyberEngineParams engine_parameters;
    
    // Definition of the Java's Secure Random to act as
    // Pseudo-Random Generator (PRG) for the generation of
    // the instance of the Key Pair to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem.
    private SecureRandom secure_pseudo_random_generator_key_pair_generation;
    
    // Definition of the key generation parameters to be
    // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    private KyberKeyGenerationParameters key_generation_parameters;

    // Definition of the pair of asymmetric keys to be
    // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    private AsymmetricCipherKeyPair asymmetric_key_pair;
            
    
    
    // Constructor of the class of the CRYSTALS-Kyber Public-Key Cryptosystem,
    // and the initialization of the initial required parameters.
    //    Parameters:
    //    @param parameter_choice_index: The parameter choice index for
    //                                   the generation of the pair of
    //                                   asymmetric keys to be used.
    public CrystalsKyberCryptosystem( short parameter_choice_index ) {
        
        // Addition of the (Classical) Post-Quantum Cryptography
        // provider of the Bouncy Castle library for Java.
        this.add_classical_post_quantum_cryptography_bouncy_castle_provider();
        
        // Initialization of the parameter choice index for
        // the generation of the pair of asymmetric keys to be used.
        this.parameter_choice_index = parameter_choice_index;
        
        // Initialization of the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem, as a null object.
        this.parameters = null;
        
        // Initialization of the cryptographic engine's parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem, as a null object.
        this.engine_parameters = null;
        
        // Initialization of the Java's Secure Random
        // to act as Pseudo-Random Generator (PRG) for
        // the generation of the instance of the Key Pair
        // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        this.secure_pseudo_random_generator_key_pair_generation = 
            new SecureRandom();
        
        // Initialization of the key generation parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem, as a null object.
        this.key_generation_parameters = null;
        
        // Initialization of the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem, as a null object.
        this.asymmetric_key_pair = null;
        
    }
    
    
    // Methods of the class of the Public-Key (Asymmetric) Cryptosystem
    // CRYSTALS-Kyber (CRYptographic SuiTe for Algebraic LatticeS - Kyber).
    
    // Method to add the (Classical) Post-Quantum Cryptography
    // provider of the Bouncy Castle library for Java,
    // to the built-in security module of Java.
    public void add_classical_post_quantum_cryptography_bouncy_castle_provider() {
        
        // If the built-in security module of Java does not find or
        // not have the (Classical) Post-Quantum Cryptography
        // provider of the Bouncy Castle library for Java,
        // denoted as BCPQC (Bouncy Castle - Post-Quantum Cryptography).
        if( Security.getProvider( "BCPQC" ) == null ) {
         
            // Addition of the (Classical) Post-Quantum Cryptography
            // provider of the Bouncy Castle library for Java,
            // to the built-in security module of Java.
            Security.addProvider( new BouncyCastlePQCProvider() );
        
        }
        
    }
    
    
    // Method to obtain the parameter choice index defined for
    // the generation of the pair of asymmetric keys to be used.
    public short get_parameter_choice_index() {
        
        // Return of the parameter choice index defined for
        // the generation of the pair of asymmetric keys to be used.
        return this.parameter_choice_index;
        
    }
    
    
    // Method to obtain the string with the name of
    // the CRYSTALS-Kyber Public-Key Cryptosystem's
    // Security Parameters defined for the key pair generation
    // (consisting on private and public keys), as well as
    // the respective Key Encapsulation Method/Mechanism (KEM).
    public String get_security_parameters_name() {
        
        // Return of the string with the name of
        // the CRYSTALS-Kyber Public-Key Cryptosystem's
        // Security Parameters defined for the key pair generation
        // (consisting on private and public keys), as well as
        // the respective Key Encapsulation Method/Mechanism (KEM).
        return security_parameters_names[ this.parameter_choice_index ];
        
    }
    
    
    // Method to obtain the string with the name and
    // description of the CRYSTALS-Kyber Public-Key Cryptosystem's
    // Security Parameters defined for the key pair generation
    // (consisting on private and public keys), as well as
    // the respective Key Encapsulation Method/Mechanism (KEM).
    public String get_security_parameters_name_and_description() {
        
        // Return of the string with the name and description of
        // the CRYSTALS-Kyber Public-Key Cryptosystem's
        // Security Parameters defined for the key pair generation
        // (consisting on private and public keys), as well as
        // the respective Key Encapsulation Method/Mechanism (KEM).
        return security_parameters_names_and_descriptions[ this.parameter_choice_index ];
        
    }
       
        
    // Method to obtain the CRYSTALS-Kyber Public-Key
    // Cryptosystem's Security Parameters Specifications.
    public KyberParameterSpec get_security_parameters_specifications() {
        
        // Return of the CRYSTALS-Kyber Public-Key
        // Cryptosystem's Security Parameters Specifications.
        return security_parameters_specifications[ this.parameter_choice_index ];
        
    }
    
    
    // Method to obtain the parameter value regarding
    // the number of polynomials k per vector used in
    // the matrix A representing the lattice algebraic
    // structure, representing as well, the size factor
    // for that same lattice algebraic structure, and thus,
    // being the main security parameter in terms of scaling
    // the security strength of the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_engine_k_parameter() {
        
        // Return of the parameter value regarding
        // the number of polynomials k per vector used in
        // the matrix A representing the lattice algebraic
        // structure, representing as well, the size factor
        // for that same lattice algebraic structure, and thus,
        // being the main security parameter in terms of scaling
        // the security strength of the CRYSTALS-Kyber Public-Key Cryptosystem.
        return engine_k_parameters[ this.parameter_choice_index ];
        
    }
    
    
    // Method to initialize the general parameters to be
    // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public void initialize_parameters() {
        
        // If the parameter choice index for
        // the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // corresponds to the 'Kyber512' Security Parameter Set.
        if( this.parameter_choice_index == 0 ) {
            
            // Setting the 'Kyber512' Security Parameter Set as
            // the parameter choice for the general parameters
            // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
            this.parameters = KyberParameters.kyber512;
            
        }
        // If the parameter choice index for
        // the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // corresponds to the 'Kyber768' Security Parameter Set.
        else if( this.parameter_choice_index == 1 ) {
            
            // Setting the 'Kyber768' Security Parameter Set as
            // the parameter choice for the general parameters
            // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
            this.parameters = KyberParameters.kyber768;
            
        }
        // If the parameter choice index for
        // the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // corresponds to the 'Kyber1024' Security Parameter Set.
        else if( this.parameter_choice_index == 2 ) {
            
            // Setting the 'Kyber1024' Security Parameter Set as
            // the parameter choice for the general parameters
            // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
            this.parameters = KyberParameters.kyber1024;
            
        }
        
    }
    
    
    // Method to obtain the general parameters to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem.
    public KyberParameters get_parameters() {
        
        // Return of the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem.
        return this.parameters;
        
    }
    
    
    // Method to obtain the size of the symmetric session key, in terms of bytes,
    // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_session_key_bytes_size() {
        
        // If the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is not null (i.e., already initialized).
        if( this.get_parameters() != null ) {
            
            // Return of the size of the symmetric session key, in terms of bytes,
            // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
            return ( this.get_session_key_bits_size() / 
                     BYTE_SIZE_IN_BITS );
            
        }
        // If the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is null (i.e., not initialized yet).
        else {
            
            // Return of an invalid size of the symmetric session key, in terms of bytes,
            // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
            return 0;

        }
        
    }
    
    
    // Method to obtain the size of the symmetric session key, in terms of bits,
    // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_session_key_bits_size() {
        
        // If the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is not null (i.e., already initialized).
        if( this.get_parameters() != null ) {
            
            // Return of the size of the symmetric session key, in terms of bits,
            // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
            return this.get_parameters().getSessionKeySize();
            
        }
        // If the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is null (i.e., not initialized yet).
        else {
            
            // Return of an invalid size of the symmetric session key, in terms of bits,
            // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
            return 0;

        }
        
    }
    
    
    // Method to initialize the cryptographic engine's parameters
    // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public void initialize_engine_parameters() {
        
        // Retrieval of the parameters required to build
        // the CRYSTALS-Kyber Public-Key Cryptosystem's cryptographic engine.
        int kyber_engine_k_parameter = this.get_engine_k_parameter();

        // Initialization of the cryptographic engine's parameters
        // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        this.engine_parameters = 
            new KyberEngineParams( kyber_engine_k_parameter );
        
    }
        
    
    // Method to obtain the cryptographic engine's parameters
    // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public KyberEngineParams get_engine_parameters() {
        
        // Return of the cryptographic engine's parameters
        // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        return this.engine_parameters;
        
    }
    
    
    // Method to obtain the Java's Secure Random to act as
    // Pseudo-Random Generator (PRG) for the key pair generation to
    // be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public SecureRandom get_secure_pseudo_random_generator_key_pair_generation() {
            
        // Return of the Java's Secure Random to act as
        // Pseudo-Random Generator (PRG) for the key pair generation to
        // be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        return this.secure_pseudo_random_generator_key_pair_generation;
    
    }
    
    
    // Method to initialize the key generation parameters to be
    // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public void initialize_key_generation_parameters() {
        
        // If the the general parameters to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem, are not null.
        if( this.get_parameters() != null ) {
        
            // Initialization of the key generation parameters to be
            // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
            this.key_generation_parameters = 
                new KyberKeyGenerationParameters
                    ( this.secure_pseudo_random_generator_key_pair_generation,
                      this.parameters );
        
        }
        
    }
    
    
    // Method to obtain the key generation parameters to be\
    // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public KyberKeyGenerationParameters get_key_generation_parameters() {
        
        // Return of the key generation parameters to be
        // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        return this.key_generation_parameters;
        
    }
    
    
    // Method to initialize the generation of
    // the pair of asymmetric keys to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem. 
    public void initialize_asymmetric_key_pair_generation() {
       
        // Creation of the generator of the pair of asymmetric keys
        // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem. 
        KyberKeyPairGenerator crystals_kyber_key_pair_generator = 
            new KyberKeyPairGenerator();

        // Initialization of the generator of the pair of asymmetric keys
        // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem,
        // using the Security Parameters set defined before.
        crystals_kyber_key_pair_generator
            .init( this.key_generation_parameters );

        // Generation of the pair of asymmetric keys to be
        // used by the CRYSTALS-Kyber Public-Key Cryptosystem. 
        this.asymmetric_key_pair = 
            crystals_kyber_key_pair_generator.generateKeyPair();
            
    }
    
    
    // Method to obtain the pair of asymmetric keys to be
    // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public AsymmetricCipherKeyPair get_asymmetric_key_pair() {
        
        // Return of the pair of asymmetric keys to be
        // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        return this.asymmetric_key_pair;
        
    }
    
    
    // Method to obtain the public key from
    // the pair of asymmetric keys to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem.
    public AsymmetricKeyParameter get_public_key() {
        
        // If the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is not defined by a null object.
        if( this.get_asymmetric_key_pair() != null ) {
            
            // Return of the public key from
            // the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem.
            return this.get_asymmetric_key_pair().getPublic();
            
        }
        // If the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is defined by a null object.
        else {
           
            // Return of an invalid public key from
            // the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem,
            // as a null object.
            return null;
            
        }
        
    }
    
    
    // Method to obtain the public key parameters of
    // the pair of asymmetric keys to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem.
    public KyberPublicKeyParameters get_public_key_parameters() {
        
        // If the public key from
        // the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is not defined by a null object.
        if( this.get_public_key() != null ) {
            
            // Return of the public key parameters of
            // the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem.
            return ( ( KyberPublicKeyParameters ) this.get_public_key() );
            
        }
        // If the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is defined by a null object.
        else {
           
            // Return of an invalid public key parameters of
            // the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem,
            // as a null object.
            return null;
            
        }
        
    }
    
    
    // Method to obtain the target/solution t
    // (i.e. the target/solution vector for
    // the mathematical equation A x s + e = t),
    // encoded (and compressed) in bytes,
    // representing a lattice-based problem in
    // the form of a LWE (Learning With Errors),
    // retrieved from the public key parameters of
    // the pair of asymmetric keys to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem.
    public byte[] get_target_solution_t_from_public_key_parameters() {
        
        // If the public key parameters of
        // the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is not defined by a null object.
        if( this.get_public_key_parameters() != null ) {
            
            // Return of the target/solution t
            // (i.e. the target/solution vector for
            // the mathematical equation A x s + e = t),
            // encoded (and compressed) in bytes,
            // representing a lattice-based problem in
            // the form of a LWE (Learning With Errors),
            // retrieved from the public key parameters of
            // the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem.
            return this.get_public_key_parameters().getT();
            
        }
        // If the public key parameters of
        // the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is defined by a null object.
        else {
           
            // Return of an invalid target/solution t
            // (i.e. the target/solution vector for
            // the mathematical equation A x s + e = t),
            // encoded (and compressed) in bytes,
            // representing a lattice-based problem in
            // the form of a LWE (Learning With Errors),
            // retrieved from the public key parameters of
            // the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem,
            // as a null object.
            return null;
            
        }
        
    }
    
    
    // Method to obtain the size,
    // in terms of bytes, of the target/solution t
    // (i.e. the target/solution vector for
    // the mathematical equation A x s + e = t),
    // encoded (and compressed) in bytes,
    // representing a lattice-based problem in
    // the form of a LWE (Learning With Errors),
    // retrieved from the public key parameters of
    // the pair of asymmetric keys to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_target_solution_t_size_bytes_from_public_key_parameters() {
        
        // If the target/solution t
        // (i.e. the target/solution vector for
        // the mathematical equation A x s + e = t),
        // encoded (and compressed) in bytes,
        // representing a lattice-based problem in
        // the form of a LWE (Learning With Errors),
        // retrieved from the public key parameters of
        // the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is not defined by a null object.
        if( this.get_target_solution_t_from_public_key_parameters() != null ) {
        
            // Return of the size,
            // in terms of bytes, of the target/solution t
            // (i.e. the target/solution vector for
            // the mathematical equation A x s + e = t),
            // encoded (and compressed) in bytes,
            // representing a lattice-based problem in
            // the form of a LWE (Learning With Errors),
            // retrieved from the public key parameters of
            // the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem.
            return this.get_target_solution_t_from_public_key_parameters().length;
        
        }
        // If the target/solution t
        // (i.e. the target/solution vector for
        // the mathematical equation A x s + e = t),
        // encoded (and compressed) in bytes,
        // representing a lattice-based problem in
        // the form of a LWE (Learning With Errors),
        // retrieved from the public key parameters of
        // the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is defined by a null object.
        else {
            
            // Return of an invalid size,
            // in terms of bytes, of the target/solution t
            // (i.e. the target/solution vector for
            // the mathematical equation A x s + e = t),
            // encoded (and compressed) in bytes,
            // representing a lattice-based problem in
            // the form of a LWE (Learning With Errors),
            // retrieved from the public key parameters of
            // the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem.
            return 0;
            
        }
        
    }
    
    
    // Method to obtain the size,
    // in terms of bits, of the target/solution t
    // (i.e. the target/solution vector for
    // the mathematical equation A x s + e = t),
    // encoded (and compressed) in bytes,
    // representing a lattice-based problem in
    // the form of a LWE (Learning With Errors),
    // retrieved from the public key parameters of
    // the pair of asymmetric keys to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_target_solution_t_size_bits_from_public_key_parameters() {
        
        // If the target/solution t
        // (i.e. the target/solution vector for
        // the mathematical equation A x s + e = t),
        // encoded (and compressed) in bytes,
        // representing a lattice-based problem in
        // the form of a LWE (Learning With Errors),
        // retrieved from the public key parameters of
        // the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is not defined by a null object.
        if( this.get_target_solution_t_from_public_key_parameters() != null ) {
        
            // Return of the size,
            // in terms of bits, of the target/solution t
            // (i.e. the target/solution vector for
            // the mathematical equation A x s + e = t),
            // encoded (and compressed) in bytes,
            // representing a lattice-based problem in
            // the form of a LWE (Learning With Errors),
            // retrieved from the public key parameters of
            // the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem.
            return ( this.get_target_solution_t_size_bytes_from_public_key_parameters() * 
                     BYTE_SIZE_IN_BITS );
        
        }
        // If the target/solution t
        // (i.e. the target/solution vector for
        // the mathematical equation A x s + e = t),
        // encoded (and compressed) in bytes,
        // representing a lattice-based problem in
        // the form of a LWE (Learning With Errors),
        // retrieved from the public key parameters of
        // the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is defined by a null object.
        else {
            
            // Return of an invalid size,
            // in terms of bits, of the target/solution t
            // (i.e. the target/solution vector for
            // the mathematical equation A x s + e = t),
            // encoded (and compressed) in bytes,
            // representing a lattice-based problem in
            // the form of a LWE (Learning With Errors),
            // retrieved from the public key parameters of
            // the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem.
            return 0;
            
        }
        
    }
    
    
    // Method to obtain the (pseudo) random seed ρ (rho)
    // (i.e. the random seed from which will be expanded and
    // obtained the uniformly-random matrix A consisting of
    // polynomials representing a lattice algebraic structure for
    // the mathematical equation A x s + e = t), encoded in bytes,
    // representing a lattice-based problem in the form of a
    // MLWE (Module-Learning With Errors), retrieved from
    // the public key parameters of the pair of asymmetric keys
    // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public byte[] get_random_seed_rho_from_public_key_parameters() {
        
        // If the public key parameters of
        // the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is not defined by a null object.
        if( this.get_public_key_parameters() != null ) {
            
            // Return of the (pseudo) random seed ρ (rho)
            // (i.e. the random seed from which will be expanded and
            // obtained the uniformly-random matrix A consisting of
            // polynomials representing a lattice algebraic structure for
            // the mathematical equation A x s + e = t), encoded in bytes,
            // representing a lattice-based problem in the form of a
            // MLWE (Module-Learning With Errors), retrieved from
            // the public key parameters of the pair of asymmetric keys
            // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
            return this.get_public_key_parameters().getRho();
            
        }
        // If the public key parameters of
        // the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is defined by a null object.
        else {
           
            // Return of the (pseudo) random seed ρ (rho)
            // (i.e. the random seed from which will be expanded and
            // obtained the uniformly-random matrix A consisting of
            // polynomials representing a lattice algebraic structure for
            // the mathematical equation A x s + e = t), encoded in bytes,
            // representing a lattice-based problem in the form of a
            // MLWE (Module-Learning With Errors), retrieved from
            // the public key parameters of the pair of asymmetric keys
            // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem,
            // as a null object.
            return null;
            
        }
        
    }
    
    
    // Method to obtain the size, in terms of bytes,
    // of the (pseudo) random seed ρ (rho)
    // (i.e. the random seed from which will be expanded and
    // obtained the uniformly-random matrix A consisting of
    // polynomials representing a lattice algebraic structure for
    // the mathematical equation A x s + e = t), encoded in bytes,
    // representing a lattice-based problem in the form of a
    // MLWE (Module-Learning With Errors), retrieved from
    // the public key parameters of the pair of asymmetric keys
    // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_random_seed_rho_size_bytes_from_public_key_parameters() {
        
        // If the (pseudo) random seed ρ (rho)
        // (i.e. the random seed from which will be expanded and
        // obtained the uniformly-random matrix A consisting of
        // polynomials representing a lattice algebraic structure for
        // the mathematical equation A x s + e = t), encoded in bytes,
        // representing a lattice-based problem in the form of a
        // MLWE (Module-Learning With Errors), retrieved from
        // the public key parameters of the pair of asymmetric keys
        // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is not defined by a null object.
        if( this.get_random_seed_rho_from_public_key_parameters() != null ) {
            
            // Return of the size, in terms of bytes,
            // of the (pseudo) random seed ρ (rho)
            // (i.e. the random seed from which will be expanded and
            // obtained the uniformly-random matrix A consisting of
            // polynomials representing a lattice algebraic structure for
            // the mathematical equation A x s + e = t), encoded in bytes,
            // representing a lattice-based problem in the form of a
            // MLWE (Module-Learning With Errors), retrieved from
            // the public key parameters of the pair of asymmetric keys
            // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
            return this.get_random_seed_rho_from_public_key_parameters().length;
        
        }
        // If the (pseudo) random seed ρ (rho)
        // (i.e. the random seed from which will be expanded and
        // obtained the uniformly-random matrix A consisting of
        // polynomials representing a lattice algebraic structure for
        // the mathematical equation A x s + e = t), encoded in bytes,
        // representing a lattice-based problem in the form of a
        // MLWE (Module-Learning With Errors), retrieved from
        // the public key parameters of the pair of asymmetric keys
        // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is defined by a null object.
        else {
            
            // Return of an invalid size, in terms of bytes,
            // of the (pseudo) random seed ρ (rho)
            // (i.e. the random seed from which will be expanded and
            // obtained the uniformly-random matrix A consisting of
            // polynomials representing a lattice algebraic structure for
            // the mathematical equation A x s + e = t), encoded in bytes,
            // representing a lattice-based problem in the form of a
            // MLWE (Module-Learning With Errors), retrieved from
            // the public key parameters of the pair of asymmetric keys
            // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
            return 0;
            
        }
        
    }
    
    
    // Method to obtain the size, in terms of bits,
    // of the (pseudo) random seed ρ (rho)
    // (i.e. the random seed from which will be expanded and
    // obtained the uniformly-random matrix A consisting of
    // polynomials representing a lattice algebraic structure for
    // the mathematical equation A x s + e = t), encoded in bytes,
    // representing a lattice-based problem in the form of a
    // MLWE (Module-Learning With Errors), retrieved from
    // the public key parameters of the pair of asymmetric keys
    // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_random_seed_rho_size_bits_from_public_key_parameters() {
        
        // If the (pseudo) random seed ρ (rho)
        // (i.e. the random seed from which will be expanded and
        // obtained the uniformly-random matrix A consisting of
        // polynomials representing a lattice algebraic structure for
        // the mathematical equation A x s + e = t), encoded in bytes,
        // representing a lattice-based problem in the form of a
        // MLWE (Module-Learning With Errors), retrieved from
        // the public key parameters of the pair of asymmetric keys
        // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is not defined by a null object.
        if( this.get_random_seed_rho_from_public_key_parameters() != null ) {
        
            // Return of the size, in terms of bits,
            // of the (pseudo) random seed ρ (rho)
            // (i.e. the random seed from which will be expanded and
            // obtained the uniformly-random matrix A consisting of
            // polynomials representing a lattice algebraic structure for
            // the mathematical equation A x s + e = t), encoded in bytes,
            // representing a lattice-based problem in the form of a
            // MLWE (Module-Learning With Errors), retrieved from
            // the public key parameters of the pair of asymmetric keys
            // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
            return ( this.get_random_seed_rho_size_bytes_from_public_key_parameters() * 
                     BYTE_SIZE_IN_BITS );
            
        }
        // If the (pseudo) random seed ρ (rho)
        // (i.e. the random seed from which will be expanded and
        // obtained the uniformly-random matrix A consisting of
        // polynomials representing a lattice algebraic structure for
        // the mathematical equation A x s + e = t), encoded in bytes,
        // representing a lattice-based problem in the form of a
        // MLWE (Module-Learning With Errors), retrieved from
        // the public key parameters of the pair of asymmetric keys
        // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is defined by a null object.
        else {
            
            // Return of an invalid size, in terms of bits,
            // of the (pseudo) random seed ρ (rho)
            // (i.e. the random seed from which will be expanded and
            // obtained the uniformly-random matrix A consisting of
            // polynomials representing a lattice algebraic structure for
            // the mathematical equation A x s + e = t), encoded in bytes,
            // representing a lattice-based problem in the form of a
            // MLWE (Module-Learning With Errors), retrieved from
            // the public key parameters of the pair of asymmetric keys
            // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
            return 0;
            
        }
        
    }
    
    
    // Method to obtain the public key from
    // the pair of asymmetric keys to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem,
    // being encoded as an array of bytes.
    public byte[] get_public_key_encoded_bytes() {
        
        // If the public key from
        // the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is not defined by a null object.
        if( this.get_public_key() != null ) {

            // Retrieval of the public key from
            // the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem,
            // being encoded as an array of bytes.
            byte[] public_key_encoded_bytes = 
                 ( ( KyberPublicKeyParameters ) this.get_public_key() ).getEncoded();

            // Return of the public key from
            // the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem,
            // being encoded as an array of bytes.
            return public_key_encoded_bytes;
        
        }
        // If the public key from
        // the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is defined by a null object.
        else {
        
            // Return of an invalid public key from
            // the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem,
            // being encoded as an array of bytes.
            return null;
            
        }
        
    }
    
    
    // Method to obtain the size of the public key
    // from the pair of asymmetric keys to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem,
    // as being encoded as an array of bytes.
    public int get_public_key_size_bytes() {
        
        // If the public key from
        // the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // being encoded as an array of bytes,
        // is not defined by a null object.
        if( this.get_public_key_encoded_bytes() != null ) {

            // Return of the size of the public key
            // from the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem,
            // as being encoded as an array of bytes.
            return this.get_public_key_encoded_bytes().length;
        
        }
        // If the public key from
        // the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // being encoded as an array of bytes,
        // is defined by a null object.
        else {
    
            // Return of an invalid size of the public key
            // from the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem,
            // as being encoded as an array of bytes.
            return 0;
            
        }
 
    }
    
    
    // Method to obtain the size of the public key
    // from the pair of asymmetric keys to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem,
    // as being encoded as an array of bits.
    public int get_public_key_size_bits() {
        
        // If the public key from
        // the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // being encoded as an array of bytes,
        // is not defined by a null object.
        if( this.get_public_key_encoded_bytes() != null ) {

            // Return of the size of the public key
            // from the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem,
            // as being encoded as an array of bits.
            return ( this.get_public_key_size_bytes() *
                     BYTE_SIZE_IN_BITS );
        
        }
        // If the public key from
        // the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // being encoded as an array of bytes,
        // is defined by a null object.
        else {
    
            // Return of an invalid size of the public key
            // from the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem,
            // as being encoded as an array of bytes.
            return 0;
            
        }
        
    }
    
    
    // Method to obtain the payload of the public key
    // from the pair of asymmetric keys to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem,
    // as a proper object of public key of the cryptosystem,
    // using the associated Security Parameters.
    public KyberPublicKeyParameters get_public_key_payload_from_parameters() {
        
        // If the key generation parameters to be
        // used by the CRYSTALS-Kyber Public-Key Cryptosystem,
        // are not defined by a null object.
        if( this.get_key_generation_parameters() != null ) {
        
            // Definition of the payload of the public key
            // from the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem,
            // as a proper object of public key of the cryptosystem.
            KyberPublicKeyParameters public_key_payload;

            // Retrieval of the cipher parameters from
            // the key generation parameters to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem.
            KyberParameters crystals_kyber_cryptosystem_parameters = 
                this.get_key_generation_parameters().getParameters();


            // Call of the required methods to
            // obtain all the values, encoded in bytes,
            // retrieved from the public key parameters of
            // the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem,
            // which will be used to build public key payload.

            // Retrieval of the target/solution t
            // (i.e. the target/solution vector for
            // the mathematical equation A x s + e = t),
            // encoded (and compressed) in bytes.
            byte[] target_solution_t = 
                this.get_target_solution_t_from_public_key_parameters();

            // Retrieval of the (pseudo) random seed ρ (rho)
            // (i.e. the random seed from which will be expanded and
            // obtained the uniformly-random matrix A consisting of
            // polynomials representing a lattice algebraic structure for
            // the mathematical equation A x s + e = t), encoded in bytes.
            byte[] random_seed_rho = 
                this.get_random_seed_rho_from_public_key_parameters();


            // Return of the payload of the public key
            // from the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem,
            // as a proper object of public key of the cryptosystem.
            return new KyberPublicKeyParameters( crystals_kyber_cryptosystem_parameters,
                                                 target_solution_t,
                                                 random_seed_rho );
        
        }
        // If the key generation parameters to be
        // used by the CRYSTALS-Kyber Public-Key Cryptosystem,
        // are defined by a null object.
        else {
            
            // Return of an invalid payload of the public key
            // from the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem,
            // as a proper object of public key of
            // the cryptosystem, as a null object.
            return null;
            
        }
        
    }
    
    
    // Method to obtain the payload of the public key
    // from the pair of asymmetric keys to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem,
    // as a proper object of public key of the cryptosystem,
    // using the associated encoding in an array of bytes.
    public KyberPublicKeyParameters get_public_key_payload_from_encoding() {
        
        // If the key generation parameters to be
        // used by the CRYSTALS-Kyber Public-Key Cryptosystem,
        // are not defined by a null object.
        if( this.get_key_generation_parameters() != null ) {
        
            // Definition of the payload of the public key
            // from the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem,
            // as a proper object of public key of the cryptosystem.
            KyberPublicKeyParameters public_key_payload;

            // Retrieval of the cipher parameters from
            // the key generation parameters to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem.
            KyberParameters crystals_kyber_cryptosystem_parameters = 
                this.get_key_generation_parameters().getParameters();


            // Return of the payload of the public key
            // from the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem,
            // as a proper object of public key of the cryptosystem.
            return new KyberPublicKeyParameters( crystals_kyber_cryptosystem_parameters,
                                                 this.get_public_key_encoded_bytes() );
            
        }
        // If the key generation parameters to be
        // used by the CRYSTALS-Kyber Public-Key Cryptosystem,
        // are defined by a null object.
        else {
            
            // Return of an invalid payload of the public key
            // from the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem,
            // as a proper object of public key of
            // the cryptosystem, as a null object.
            return null;
            
        }
        
    }
    
    
    // Method to obtain the payload of the public key
    // from the pair of asymmetric keys to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem,
    // as a proper object of public key of the cryptosystem.
    public KyberPublicKeyParameters get_public_key_payload() {
    
        // Return of the payload of the public key
        // from the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // as a proper object of public key of the cryptosystem.
        return this.get_public_key_payload_from_parameters();
        
    }
    
    
    // Method to obtain the private key from
    // the pair of asymmetric keys to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem.
    public AsymmetricKeyParameter get_private_key() {
        
        // If the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is not defined by a null object.
        if( this.get_asymmetric_key_pair() != null ) {
            
            // Return of the private key from
            // the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem.
            return this.get_asymmetric_key_pair().getPrivate();
            
        }
        // If the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is defined by a null object.
        else {
           
            // Return of an invalid private key from
            // the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem,
            // as a null object.
            return null;
            
        }
        
    }
    
    
    // Method to obtain the private key parameters of
    // the pair of asymmetric keys to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem.
    public KyberPrivateKeyParameters get_private_key_parameters() {
        
        // If the private key from
        // the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is not defined by a null object.
        if( this.get_private_key() != null ) {
            
            // Return of the private key parameters of
            // the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem.
            return ( ( KyberPrivateKeyParameters ) this.get_private_key() );
            
        }
        // If the private key from
        // the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is defined by a null object.
        else {
           
            // Return of an invalid private key from
            // the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem,
            // as a null object.
            return null;
            
        }
        
    }
    
    
    // Method to obtain the random secret s
    // (i.e. the secret vector in the lattice
    // algebraic structure), encoded in bytes,
    // with the IND-CPA (INDistinguishability
    // under Chosen Plaintext Attack) property,
    // retrieved from the private key parameters of
    // the pair of asymmetric keys to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem.
    public byte[] get_random_secret_s_from_private_key_parameters() {
        
        // If the private key parameters of
        // the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem
        // are not defined by a null object.
        if( this.get_private_key_parameters() != null ) {
            
            // Return of the random secret s
            // (i.e. the secret vector in the lattice
            // algebraic structure), encoded in bytes,
            // with the IND-CPA (INDistinguishability
            // under Chosen Plaintext Attack) property,
            // retrieved from the private key parameters of
            // the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem.
            return this.get_private_key_parameters().getS();
            
        }
        // If the private key parameters of
        // the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem
        // are defined by a null object.
        else {
           
            // Return of an invalid random secret s
            // (i.e. the secret vector in the lattice
            // algebraic structure), encoded in bytes,
            // with the IND-CPA (INDistinguishability
            // under Chosen Plaintext Attack) property,
            // retrieved from the private key parameters of
            // the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem,
            // as a null object.
            return null;
            
        }
        
    }
    
    
    // Method to obtain the size,
    // in terms of bytes, of the random secret s
    // (i.e. the secret vector in the lattice
    // algebraic structure), encoded in bytes,
    // with the IND-CPA (INDistinguishability
    // under Chosen Plaintext Attack) property,
    // retrieved from the private key parameters of
    // the pair of asymmetric keys to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_random_secret_s_size_bytes_from_private_key_parameters() {
        
        // If the random secret s
        // (i.e. the secret vector in the lattice
        // algebraic structure), encoded in bytes,
        // with the IND-CPA (INDistinguishability
        // under Chosen Plaintext Attack) property,
        // retrieved from the private key parameters of
        // the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is not defined by a null object.
        if( this.get_random_secret_s_from_private_key_parameters() != null ) {
        
            // Return of the size,
            // in terms of bytes, of the random secret s
            // (i.e. the secret vector in the lattice
            // algebraic structure), encoded in bytes,
            // with the IND-CPA (INDistinguishability
            // under Chosen Plaintext Attack) property,
            // retrieved from the private key parameters of
            // the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem.
            return this.get_random_secret_s_from_private_key_parameters().length;

        }
        // If the random secret s
        // (i.e. the secret vector in the lattice
        // algebraic structure), encoded in bytes,
        // with the IND-CPA (INDistinguishability
        // under Chosen Plaintext Attack) property,
        // retrieved from the private key parameters of
        // the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is defined by a null object.
        else {
            
            // Return of an invalid size,
            // in terms of bytes, of the random secret s
            // (i.e. the secret vector in the lattice
            // algebraic structure), encoded in bytes,
            // with the IND-CPA (INDistinguishability
            // under Chosen Plaintext Attack) property,
            // retrieved from the private key parameters of
            // the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem.
            return 0;
            
        }
        
    }
    
    
    // Method to obtain the size,
    // in terms of bits, of the random secret s
    // (i.e. the secret vector in the lattice
    // algebraic structure), encoded in bytes,
    // with the IND-CPA (INDistinguishability
    // under Chosen Plaintext Attack) property,
    // retrieved from the private key parameters of
    // the pair of asymmetric keys to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_random_secret_s_size_bits_from_private_key_parameters() {
        
        // If the random secret s
        // (i.e. the secret vector in the lattice
        // algebraic structure), encoded in bytes,
        // with the IND-CPA (INDistinguishability
        // under Chosen Plaintext Attack) property,
        // retrieved from the private key parameters of
        // the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is not defined by a null object.
        if( this.get_random_secret_s_from_private_key_parameters() != null ) {
        
            // Return of the size,
            // in terms of bits, of the random secret s
            // (i.e. the secret vector in the lattice
            // algebraic structure), encoded in bytes,
            // with the IND-CPA (INDistinguishability
            // under Chosen Plaintext Attack) property,
            // retrieved from the private key parameters of
            // the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem.
            return ( this.get_random_secret_s_size_bytes_from_private_key_parameters() *
                     BYTE_SIZE_IN_BITS );

        }
        // If the random secret s
        // (i.e. the secret vector in the lattice
        // algebraic structure), encoded in bytes,
        // with the IND-CPA (INDistinguishability
        // under Chosen Plaintext Attack) property,
        // retrieved from the private key parameters of
        // the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is defined by a null object.
        else {
            
            // Return of an invalid size,
            // in terms of bits, of the random secret s
            // (i.e. the secret vector in the lattice
            // algebraic structure), encoded in bytes,
            // with the IND-CPA (INDistinguishability
            // under Chosen Plaintext Attack) property,
            // retrieved from the private key parameters of
            // the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem.
            return 0;
            
        }
        
    }
    
    
    // Method to obtain the hashed public key,
    // denoted as H(pub_key), encoded in bytes,
    // with the IND-CPA (INDistinguishability
    // under Chosen Plaintext Attack) property,
    // retrieved from the private key parameters of
    // the pair of asymmetric keys to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem.
    public byte[] get_hashed_public_key_h_pub_k_from_private_key_parameters() {
        
        // If the private key parameters of
        // the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem
        // are not defined by a null object.
        if( this.get_private_key_parameters() != null ) {
            
            // Return of the hashed public key,
            // denoted as H(pub_key), encoded in bytes,
            // with the IND-CPA (INDistinguishability
            // under Chosen Plaintext Attack) property,
            // retrieved from the private key parameters of
            // the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem.
            return this.get_private_key_parameters().getHPK();
            
        }
        // If the private key parameters of
        // the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem
        // are defined by a null object.
        else {
           
            // Return of an invalid hashed public key,
            // denoted as H(pub_key), encoded in bytes,
            // with the IND-CPA (INDistinguishability
            // under Chosen Plaintext Attack) property,
            // retrieved from the private key parameters of
            // the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem,
            // as a null object.
            return null;
            
        }
        
    }
    
    
    // Method to obtain the size, in terms of bytes,
    // of the hashed public key, denoted as
    // H(pub_key), encoded in bytes, with
    // the IND-CPA (INDistinguishability
    // under Chosen Plaintext Attack) property,
    // retrieved from the private key parameters of
    // the pair of asymmetric keys to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_hashed_public_key_h_pub_k_size_bytes_from_private_key_parameters() {
        
        // If the hashed public key,
        // denoted as H(pub_key), encoded in bytes,
        // with the IND-CPA (INDistinguishability
        // under Chosen Plaintext Attack) property,
        // retrieved from the private key parameters of
        // the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is not defined by a null object.
        if( this.get_hashed_public_key_h_pub_k_from_private_key_parameters() != null ) {

            // Return of the size, in terms of bytes,
            // of the hashed public key, denoted as
            // H(pub_key), encoded in bytes, with
            // the IND-CPA (INDistinguishability
            // under Chosen Plaintext Attack) property,
            // retrieved from the private key parameters of
            // the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem.
            return this.get_hashed_public_key_h_pub_k_from_private_key_parameters().length;
        
        }
        // If the hashed public key,
        // denoted as H(pub_key), encoded in bytes,
        // with the IND-CPA (INDistinguishability
        // under Chosen Plaintext Attack) property,
        // retrieved from the private key parameters of
        // the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is defined by a null object.
        else {
        
            // Return of an invalid size, in terms of bytes,
            // of the hashed public key, denoted as
            // H(pub_key), encoded in bytes, with
            // the IND-CPA (INDistinguishability
            // under Chosen Plaintext Attack) property,
            // retrieved from the private key parameters of
            // the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem.
            return 0;
            
        }
        
    }
    
    
    // Method to obtain the size, in terms of bits,
    // of the hashed public key, denoted as
    // H(pub_key), encoded in bytes, with
    // the IND-CPA (INDistinguishability
    // under Chosen Plaintext Attack) property,
    // retrieved from the private key parameters of
    // the pair of asymmetric keys to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_hashed_public_key_h_pub_k_size_bits_from_private_key_parameters() {
        
        // If the hashed public key,
        // denoted as H(pub_key), encoded in bytes,
        // with the IND-CPA (INDistinguishability
        // under Chosen Plaintext Attack) property,
        // retrieved from the private key parameters of
        // the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is not defined by a null object.
        if( this.get_hashed_public_key_h_pub_k_from_private_key_parameters() != null ) {

            // Return of the size, in terms of bits,
            // of the hashed public key, denoted as
            // H(pub_key), encoded in bytes, with
            // the IND-CPA (INDistinguishability
            // under Chosen Plaintext Attack) property,
            // retrieved from the private key parameters of
            // the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem.
            return ( this.get_hashed_public_key_h_pub_k_size_bytes_from_private_key_parameters() *
                     BYTE_SIZE_IN_BITS );
        
        }
        // If the hashed public key,
        // denoted as H(pub_key), encoded in bytes,
        // with the IND-CPA (INDistinguishability
        // under Chosen Plaintext Attack) property,
        // retrieved from the private key parameters of
        // the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is defined by a null object.
        else {
        
            // Return of an invalid size, in terms of bits,
            // of the hashed public key, denoted as
            // H(pub_key), encoded in bytes, with
            // the IND-CPA (INDistinguishability
            // under Chosen Plaintext Attack) property,
            // retrieved from the private key parameters of
            // the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem.
            return 0;
            
        }
        
    }
    
    
    // Method to obtain the random secret nonce z
    // (i.e., to avoid practical fault attacks),
    // encoded in bytes, with the IND-CPA
    // (INDistinguishability under Chosen Plaintext
    // Attack) property for the Asymmetric Encryption and
    // with the (INDistinguishability under Chosen Ciphertext
    // Attack) property for the KEM (Key Encapsulation
    // Method/Mechanism), retrieved from the private key
    // parameters of the pair of asymmetric keys to be
    // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public byte[] get_random_secret_nonce_z_from_private_key_parameters() {
        
        // If the private key parameters of
        // the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem
        // are not defined by a null object.
        if( this.get_private_key_parameters() != null ) {
            
            // Return of the random secret nonce z
            // (i.e., to avoid practical fault attacks),
            // encoded in bytes, with the IND-CPA
            // (INDistinguishability under Chosen Plaintext
            // Attack) property for the Asymmetric Encryption and
            // with the (INDistinguishability under Chosen Ciphertext
            // Attack) property for the KEM (Key Encapsulation
            // Method/Mechanism), retrieved from the private key
            // parameters of the pair of asymmetric keys to be
            // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
            return this.get_private_key_parameters().getNonce();
            
        }
        // If the private key parameters of
        // the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem
        // are defined by a null object.
        else {
           
            // Return of an invalid random secret nonce z
            // (i.e., to avoid practical fault attacks),
            // encoded in bytes, with the IND-CPA
            // (INDistinguishability under Chosen Plaintext
            // Attack) property for the Asymmetric Encryption and
            // with the (INDistinguishability under Chosen Ciphertext
            // Attack) property for the KEM (Key Encapsulation
            // Method/Mechanism), retrieved from the private key
            // parameters of the pair of asymmetric keys to be
            // used by the CRYSTALS-Kyber Public-Key Cryptosystem,
            // as a null object.
            return null;
            
        }
        
    }
    
    
    // Method to obtain the size in terms of
    // bytes of the random secret nonce z
    // (i.e., to avoid practical fault attacks),
    // encoded in bytes, with the IND-CPA
    // (INDistinguishability under Chosen Plaintext
    // Attack) property for the Asymmetric Encryption and
    // with the (INDistinguishability under Chosen Ciphertext
    // Attack) property for the KEM (Key Encapsulation
    // Method/Mechanism), retrieved from the private key
    // parameters of the pair of asymmetric keys to be
    // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_random_secret_nonce_z_size_bytes_from_private_key_parameters() {
        
        // If the private key parameters of
        // the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem
        // are not defined by a null object.
        if( this.get_private_key_parameters() != null ) {

            // Return of the size, in terms of bytes,
            // of the random secret nonce z
            // (i.e., to avoid practical fault attacks),
            // encoded in bytes, with the IND-CPA
            // (INDistinguishability under Chosen Plaintext
            // Attack) property for the Asymmetric Encryption and
            // with the (INDistinguishability under Chosen Ciphertext
            // Attack) property for the KEM (Key Encapsulation
            // Method/Mechanism), retrieved from the private key
            // parameters of the pair of asymmetric keys to be
            // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
            return this.get_random_secret_nonce_z_from_private_key_parameters().length;
    
        }
        // If the private key parameters of
        // the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem
        // are defined by a null object.
        else {
            
            // Return of an invalid size, in terms of bytes,
            // of the random secret nonce z
            // (i.e., to avoid practical fault attacks),
            // encoded in bytes, with the IND-CPA
            // (INDistinguishability under Chosen Plaintext
            // Attack) property for the Asymmetric Encryption and
            // with the (INDistinguishability under Chosen Ciphertext
            // Attack) property for the KEM (Key Encapsulation
            // Method/Mechanism), retrieved from the private key
            // parameters of the pair of asymmetric keys to be
            // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
            return 0;
            
        }
        
    }
    
    
    // Method to obtain the size in terms of
    // bits of the random secret nonce z
    // (i.e., to avoid practical fault attacks),
    // encoded in bytes, with the IND-CPA
    // (INDistinguishability under Chosen Plaintext
    // Attack) property for the Asymmetric Encryption and
    // with the (INDistinguishability under Chosen Ciphertext
    // Attack) property for the KEM (Key Encapsulation
    // Method/Mechanism), retrieved from the private key
    // parameters of the pair of asymmetric keys to be
    // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    public int get_random_secret_nonce_z_size_bits_from_private_key_parameters() {
        
        // If the private key parameters of
        // the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem
        // are not defined by a null object.
        if( this.get_private_key_parameters() != null ) {
        
            // Return of the size, in terms of bits,
            // of the random secret nonce z
            // (i.e., to avoid practical fault attacks),
            // encoded in bytes, with the IND-CPA
            // (INDistinguishability under Chosen Plaintext
            // Attack) property for the Asymmetric Encryption and
            // with the (INDistinguishability under Chosen Ciphertext
            // Attack) property for the KEM (Key Encapsulation
            // Method/Mechanism), retrieved from the private key
            // parameters of the pair of asymmetric keys to be
            // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
            return ( this.get_random_secret_nonce_z_size_bytes_from_private_key_parameters() * 
                     BYTE_SIZE_IN_BITS );
        
        }
        // If the private key parameters of
        // the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem
        // are defined by a null object.
        else {
            
            // Return of an invalid size, in terms of bits,
            // of the random secret nonce z
            // (i.e., to avoid practical fault attacks),
            // encoded in bytes, with the IND-CPA
            // (INDistinguishability under Chosen Plaintext
            // Attack) property for the Asymmetric Encryption and
            // with the (INDistinguishability under Chosen Ciphertext
            // Attack) property for the KEM (Key Encapsulation
            // Method/Mechanism), retrieved from the private key
            // parameters of the pair of asymmetric keys to be
            // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
            return 0;
            
        }
        
    }
    
    
    // Method to obtain the private key from
    // the pair of asymmetric keys to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem,
    // being encoded as an array of bytes.
    public byte[] get_private_key_encoded_bytes() {
        
        // If the private key from
        // the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is not defined by a null object.
        if( this.get_private_key() != null ) {
        
            // Retrieval of the private key from
            // the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem,
            // being encoded as an array of bytes.
            byte[] private_key_encoded_bytes = 
                 ( ( KyberPrivateKeyParameters ) this.get_private_key() ).getEncoded();

            // Return of the private key from
            // the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem,
            // being encoded as an array of bytes.
            return private_key_encoded_bytes;
            
        }
        // If the private key from
        // the pair of asymmetric keys to be used by
        // the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is defined by a null object.
        else {
            
            // Return of an invalid private key from
            // the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem,
            // being encoded as an array of bytes,
            // as a null object.
            return null;
            
        }
        
    }
    
    
    // Method to obtain the size of the private key
    // from the pair of asymmetric keys to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem,
    // as being encoded as an array of bytes.
    public int get_private_key_size_bytes() {
        
        // If the private key from the pair of
        // asymmetric keys to be used by the CRYSTALS-Kyber
        // Public-Key Cryptosystem, being encoded as
        // an array of bytes, is not defined as a null object.
        if( this.get_private_key_encoded_bytes() != null ) {
        
            // Return of the size of the private key
            // from the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem,
            // as being encoded as an array of bytes.
            return this.get_private_key_encoded_bytes().length;

        }
        // If the private key from the pair of
        // asymmetric keys to be used by the CRYSTALS-Kyber
        // Public-Key Cryptosystem, being encoded as
        // an array of bytes, is defined as a null object.
        else {
            
            // Return of an invalid size of the private key
            // from the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem,
            // as being encoded as an array of bytes.
            return 0;
            
        }
        
    }
    
    
    // Method to obtain the size of the private key
    // from the pair of asymmetric keys to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem,
    // as being encoded as an array of bits.
    public int get_private_key_size_bits() {
        
        // If the private key from the pair of
        // asymmetric keys to be used by the CRYSTALS-Kyber
        // Public-Key Cryptosystem, being encoded as
        // an array of bytes, is not defined as a null object.
        if( this.get_private_key_encoded_bytes() != null ) {
        
            // Return of the size of the private key
            // from the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem,
            // as being encoded as an array of bits.
            return ( this.get_private_key_size_bytes() * 
                     BYTE_SIZE_IN_BITS );

        }
        // If the private key from the pair of
        // asymmetric keys to be used by the CRYSTALS-Kyber
        // Public-Key Cryptosystem, being encoded as
        // an array of bytes, is defined as a null object.
        else {
            
            // Return of an invalid size of the private key
            // from the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem,
            // as being encoded as an array of bytes.
            return 0;
            
        }
        
    }
    
    
    // Method to obtain the payload of the private key
    // from the pair of asymmetric keys to be used by
    // the CRYSTALS-Kyber Public-Key Cryptosystem,
    // as a proper object of private key of the cryptosystem.
    public KyberPrivateKeyParameters get_private_key_payload() {
        
        // If the key generation parameters to be
        // used by the CRYSTALS-Kyber Public-Key Cryptosystem,
        // are not defined as a null object.
        if( this.get_key_generation_parameters() != null ) {
        
            // Definition of the payload of the private key
            // from the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem,
            // as a proper object of private key of the cryptosystem.
            KyberPrivateKeyParameters private_key_payload;

            // Retrieval of the cipher parameters from
            // the key generation parameters to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem.
            KyberParameters crystals_kyber_cryptosystem_parameters = 
                this.get_key_generation_parameters().getParameters();


            // Call of the required methods to
            // obtain all the values, encoded in bytes,
            // retrieved from the public key parameters of
            // the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem,
            // which will be used to build private key payload.

            // Retrieval of the random secret s
            // (i.e. the secret vector in the lattice
            // algebraic structure), encoded in bytes.
            byte[] random_secret_s = 
                this.get_random_secret_s_from_private_key_parameters();

            // Retrieval of the hashed public key,
            // denoted as H(pub_key), encoded in bytes.
            byte[] hashed_public_key_h_pub_k = 
                this.get_hashed_public_key_h_pub_k_from_private_key_parameters();

            // Retrieval of the random secret
            // nonce z (i.e., to avoid practical
            // fault attacks), encoded in bytes.
            byte[] random_secret_nonce_z = 
                this.get_random_secret_nonce_z_from_private_key_parameters();


            // Retrieval of the target/solution t
            // (i.e. the target/solution vector for
            // the mathematical equation A x s + e = t),
            // encoded (and compressed) in bytes.
            byte[] target_solution_t = 
                this.get_target_solution_t_from_public_key_parameters();

            // Retrieval of the (pseudo) random seed ρ (rho)
            // (i.e. the random seed from which will be expanded and
            // obtained the uniformly-random matrix A consisting of
            // polynomials representing a lattice algebraic structure for
            // the mathematical equation A x s + e = t), encoded in bytes.
            byte[] random_seed_rho = 
                this.get_random_seed_rho_from_public_key_parameters();


            // Return of the payload of the private key
            // from the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem,
            // as a proper object of private key of the cryptosystem.
            return new KyberPrivateKeyParameters( crystals_kyber_cryptosystem_parameters,
                                                  random_secret_s,
                                                  hashed_public_key_h_pub_k,
                                                  random_secret_nonce_z,
                                                  target_solution_t,
                                                  random_seed_rho );
        
        }
        // If the key generation parameters to be
        // used by the CRYSTALS-Kyber Public-Key Cryptosystem,
        // are defined as a null object.
        else {
            
            // Return of an invalid payload of the private key
            // from the pair of asymmetric keys to be used by
            // the CRYSTALS-Kyber Public-Key Cryptosystem,
            // as a proper object of private key of
            // the cryptosystem, as a null object.
            return null;
            
        }
            
    }
    
}


// Setting of the parameter choice for the generation of
// the asymmetric key pair to be used internally by 
// the CRYSTALS-Kyber Public-Key Cryptosystem.
final short PARAMETERS_SET_CHOICE_INDEX = 0;


// Creation of a CRYSTALS-Kyber Public-Key Cryptosystem,
// using a set of security parameters pre-defined.
CrystalsKyberCryptosystem crystals_kyber_cryptosystem = 
    new CrystalsKyberCryptosystem( PARAMETERS_SET_CHOICE_INDEX );


// Retrieval of the string with the name and
// description of the CRYSTALS-Kyber Public-Key Cryptosystem's
// Security Parameters defined for the key pair generation
// (consisting on private and public keys), as well as
// the respective Key Encapsulation Method/Mechanism (KEM).
String crystals_kyber_security_parameters_name_and_description = 
    crystals_kyber_cryptosystem.get_security_parameters_name_and_description();


// Retrieval of the CRYSTALS-Kyber Public-Key
// Cryptosystem's Security Parameters Specifications.
KyberParameterSpec crystals_kyber_security_parameters_specification =
    crystals_kyber_cryptosystem.get_security_parameters_specifications();



// Print of the relevant information for the use of
// the CRYSTALS-Kyber Public-Key Cryptosystem.

// Print of two blank lines, as separators.
System.out.println("");
System.out.println("");


// Print of the header of the example of a possible
// execution of the CRYSTALS-Kyber Public-Key Cryptosystem.
System.out.println(" ---------------- EXAMPLE OF AN EXECUTION OF THE CRYSTALS-KYBER " + 
                                      "PUBLIC-KEY CRYPTOSYSTEM ---------------- ");


// Print of two blank lines, as separators.
System.out.println("");
System.out.println("");


// Print of the string with the name and
// description of the CRYSTALS-Kyber Public-Key Cryptosystem's
// Security Parameters defined for the key pair generation
// (consisting on private and public keys), as well as
// the respective Key Encapsulion Method/Mechanism (KEM).
System.out.println( "  NAME, DESCRIPTION, AND EXTRACTION OF THE EXECUTION AND SECURITY PARAMETERS ADOPTED:\n\n   " +
                       crystals_kyber_security_parameters_name_and_description );


// Print of one blank line, as a separator.
System.out.println("");


// Initialization of the general parameters to be used by
// the CRYSTALS-Kyber Public-Key Cryptosystem.
crystals_kyber_cryptosystem.initialize_parameters();


// Retrieval of the size of the symmetric session key, in terms of bytes,
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
int crystals_kyber_cryptosystem_session_key_bytes_size = 
    crystals_kyber_cryptosystem.get_session_key_bytes_size();

// Retrieval of the size of the symmetric session key, in terms of bits,
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
int crystals_kyber_cryptosystem_session_key_bits_size = 
    crystals_kyber_cryptosystem.get_session_key_bits_size();

// Print of the string with the size of
// the Symmetric Session Key, in terms of bytes and bits,
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
System.out.println( "     * Size of the Symmetric Session Secret Key to be generated and exchanged:\n" +
                    "       - " + crystals_kyber_cryptosystem_session_key_bytes_size + " bytes | " 
                                + crystals_kyber_cryptosystem_session_key_bits_size + " bits" );


// Print of one blank line, as a separator.
System.out.println("");


// Initialization of the cryptographic engine's parameters
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
crystals_kyber_cryptosystem.initialize_engine_parameters();

// Retrieval of the cryptographic engine's parameters
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
KyberEngineParams crystals_kyber_cryptosystem_engine_parameters = 
    crystals_kyber_cryptosystem.get_engine_parameters();


// Retrieval of the number of polynomials k
// per vector used in the matrix A representing
// the lattice algebraic structure, representing
// as well, the size factor for that same lattice
// algebraic structure, and thus, being the main
// security parameter in terms of scaling the security
// strength of the CRYSTALS-Kyber Public-Key Cryptosystem. 
int param_k = crystals_kyber_cryptosystem_engine_parameters.get_param_k();

// Retrieval of the maximum degree n of
// the polynomials used in the matrix A representing
// the lattice algebraic structure to be adopted for
// the CRYSTALS-Kyber Public-Key Cryptosystem.
int param_n = crystals_kyber_cryptosystem_engine_parameters.get_param_n();

// Retrieval of the numeric modulus q for
// the polynomials used in the matrix A representing
// the lattice algebraic structure to be adopted for
// the CRYSTALS-Kyber Public-Key Cryptosystem.
int param_q = crystals_kyber_cryptosystem_engine_parameters.get_param_q();

// Retrieval of the numeric inverse of modulus q
// for the polynomials used in the matrix A representing
// the lattice algebraic structure to be adopted for
// the CRYSTALS-Kyber Public-Key Cryptosystem.
int param_q_inv = crystals_kyber_cryptosystem_engine_parameters.get_param_q_inv();

// Retrieval of the numeric noise factor eta_1
// used in the secret random coins vector r
// extracted from a Binomial Probability Distribution,
// dependent on the public key k_pub, and also
// used for the error vector e and secret vector s,
// regarding the polynomials representing a lattice
// algebraic structure for the mathematical equation
// A x s + e = t), representing a lattice-based problem in
// the form of a MLWE (Module-Learning With Errors), to be
// used by the CRYSTALS-Kyber Public-Key Cryptosystem.
int param_eta_1 = crystals_kyber_cryptosystem_engine_parameters.get_param_eta_1();

// Retrieval of the numeric noise factor eta_2
// used for the random error vectors e_1 and e_2 extracted
// from a (Centered) Binomial Probability Distribution,
// applied during the asymmetric encryption and generation of
// the KEM (Key Encapsulation Method/Mechanism) procedures
// used by the CRYSTALS-Kyber Public-Key Cryptosystem.
int param_eta_2 = crystals_kyber_cryptosystem_engine_parameters.get_param_eta_2();

// Retrieval of the compression factor d_t
// to reduce/compress the size of the asymmetric keys to be
// used by the CRYSTALS-Kyber Public-Key Cryptosystem.
int compression_asymmetric_keys_factor_d_t = 
    crystals_kyber_cryptosystem_engine_parameters
    .get_compression_asymmetric_keys_factor_d_t();

// Retrieval of the compression factor d_u
// to reduce/compress the size of the ciphertext
// generated from the asymmetric encryption and to be
// used by the CRYSTALS-Kyber Public-Key Cryptosystem.
int compression_asymmetric_encryption_factor_d_u = 
    crystals_kyber_cryptosystem_engine_parameters
    .get_compression_asymmetric_encryption_factor_d_u();

// Retrieval of the compression factor d_v
// to reduce/compress the size of the ciphertext
// generated from the asymmetric encryption and to be
// used by the CRYSTALS-Kyber Public-Key Cryptosystem.
int compression_asymmetric_encryption_factor_d_v = 
    crystals_kyber_cryptosystem_engine_parameters
    .get_compression_asymmetric_encryption_factor_d_v();

// Print of the string with the size of
// the Security Parameter Set to be used by
// the CRYSTALS-Kyber Public-Key Cryptosystem.
System.out.println( "     * Security Parameter Set to be adopted:\n" +
                    "       - " + "k = " + param_k + " | " + "n = " + param_n + " | "
                                + "q = " + param_q + " | " + "q⁻¹ = " + param_q_inv + " | "
                                + "η₁ = " + param_eta_1 + " | " + "η₂ = " + param_eta_2 + "\n" +
                    "       - " + "dₜ = " + compression_asymmetric_keys_factor_d_t + " | "
                                + "dᵤ = " + compression_asymmetric_encryption_factor_d_u + " | "
                                + "dᵥ = " + compression_asymmetric_encryption_factor_d_v );


// Print of one blank line, as a separator.
System.out.println("");


// Retrieval of the size of the symmetric
// hashes and (pseudo) random seeds, in terms of bytes,
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
int num_symmetric_bytes = 
    crystals_kyber_cryptosystem_engine_parameters.get_num_symmetric_bytes();

// Retrieval of the size of the symmetric
// hashes and (pseudo) random seeds, in terms of bits,
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
int num_symmetric_bits = 
    crystals_kyber_cryptosystem_engine_parameters.get_num_symmetric_bits();

// Retrieval of the size of the symmetric
// shared secret, in terms of bytes, to be used by,
// the CRYSTALS-Kyber Public-Key Cryptosystem.
int num_shared_secret_bytes = 
    crystals_kyber_cryptosystem_engine_parameters.get_num_shared_secret_bytes();

// Retrieval of the size of the symmetric
// shared secret, in terms of bits, to be used by,
// the CRYSTALS-Kyber Public-Key Cryptosystem.
int num_shared_secret_bits = 
    crystals_kyber_cryptosystem_engine_parameters.get_num_shared_secret_bits();


// Print of the string with the size of the Symmetric Hashes,
// (Pseudo) Random Seeds, and Shared Secrets/Keys, in terms of
// bytes and bits, to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
System.out.println( "     * Size of the Symmetric Hashes, (Pseudo) Random Seeds and\n" +
                    "       Shared Secrets/Keys to be generated and exchanged:\n" +
                    "       " + "- Symmetric Hashes and (Pseudo) Random Seeds: " 
                              + num_symmetric_bytes + " bytes | " + num_symmetric_bits + " bits\n" +
                    "       " + "- Symmetric Shared Secret: " 
                              + num_shared_secret_bytes + " bytes | " + num_shared_secret_bits + " bits ");

// Print of one blank line, as a separator.
System.out.println("");


// Retrieval of the size of a polynomial, in terms of bytes,
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
int num_polynomial_bytes =
    crystals_kyber_cryptosystem_engine_parameters.get_num_polynomial_bytes();

// Retrieval of the size of a polynomial, in terms of bits,
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
int num_polynomial_bits =
    crystals_kyber_cryptosystem_engine_parameters.get_num_polynomial_bits();

// Retrieval of the size, in terms of bytes,
// of a polynomials vector in a matrix of bytes,
// to used by the CRYSTALS-Kyber Public-Key Cryptosystem.
int num_polynomials_vector_bytes = 
    crystals_kyber_cryptosystem_engine_parameters
    .get_num_polynomials_vector_bytes();

// Retrieval of the size, in terms of bits,
// of a polynomials vector in a matrix of bytes,
// to used by the CRYSTALS-Kyber Public-Key Cryptosystem.
int num_polynomials_vector_bits = 
    crystals_kyber_cryptosystem_engine_parameters
    .get_num_polynomials_vector_bits();

// Retrieval of the size, in terms of bytes,
// of a polynomial compressed in an array of bytes, to be
// used by the CRYSTALS-Kyber Public-Key Cryptosystem.
int num_polynomial_compressed_bytes = 
    crystals_kyber_cryptosystem_engine_parameters
    .get_num_polynomial_compressed_bytes();

// Retrieval of the size, in terms of bits,
// of a polynomial compressed in an array of bytes, to be
// used by the CRYSTALS-Kyber Public-Key Cryptosystem.
int num_polynomial_compressed_bits = 
    crystals_kyber_cryptosystem_engine_parameters
    .get_num_polynomial_compressed_bits();

// Retrieval of the size, in terms of bytes,
// of a polynomials vector compressed in a matrix of bytes,
// to used by the CRYSTALS-Kyber Public-Key Cryptosystem.
int num_polynomials_vector_compressed_bytes = 
    crystals_kyber_cryptosystem_engine_parameters
    .get_num_polynomials_vector_compressed_bytes();

// Retrieval of the size, in terms of bits,
// of a polynomials vector compressed in a matrix of bytes,
// to used by the CRYSTALS-Kyber Public-Key Cryptosystem.
int num_polynomials_vector_compressed_bits = 
    crystals_kyber_cryptosystem_engine_parameters
    .get_num_polynomials_vector_compressed_bits();


// Print of the string with the size of a Polynomial X
// and Matrix A (i.e., a vector of Polynomials X), in terms of 
// bytes and bits, to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
System.out.println( "     * Size of a Polynomial X and Matrix A (i.e., a vector of Polynomials X):\n" +
                    "       " + "- Size of a Polynomial X: " 
                              + num_polynomial_bytes + " bytes | " + num_polynomial_bits + " bits\n" +
                    "       " + "- Size of each variable/coefficient xⱼ of the Polynomial X,\n" +
                    "       " + "  with [ 0 ≤ j ≤ (n - 1) ]: "
                              + "≈ " + (double) ( (double) num_polynomial_bytes / (double) param_n ) + " bytes | " 
                              + ( num_polynomial_bits / param_n ) + " bits\n" +
                    "       " + "- Size of a vector composed of k Polynomials X\n" +
                    "       " + "  (i.e., a Matrix A with variables/coefficients A₍ᵢ,ⱼ₎)\n" +
                    "       " + "  for an algebraic linear system of equations,\n" +
                    "       " + "  with [ 1 ≤ i ≤ k ] ∧ [ 0 ≤ j ≤ (n - 1) ]: " 
                              + num_polynomials_vector_bytes + " bytes | " + num_polynomials_vector_bits + " bits\n" +
                    "       " + "- Size of a compressed Polynomial X (scaled by the factor dᵥ): " 
                              + num_polynomial_compressed_bytes + " bytes | " + num_polynomial_compressed_bits + " bits\n" +
                    "       " + "- Size of a compressed vector composed of k Polynomials X\n" +
                    "       " + "  (i.e., a Matrix A with variables/coefficients A₍ᵢ,ⱼ₎)\n" +
                    "       " + "  for an algebraic linear system of equations,\n" +
                    "       " + "  with [ 1 ≤ i ≤ k ] ∧ [ 0 ≤ j ≤ (n - 1) ] (scaled by the factor dᵤ): " 
                              + num_polynomials_vector_compressed_bytes + " bytes | " + num_polynomials_vector_compressed_bits + " bits\n" );

// Print of one blank line, as a separator.
System.out.println("");


// Retrieval of the size, in terms of bytes,
// of the asymmetric public key to be used by
// the CRYSTALS-Kyber Public-Key Cryptosystem
// with the IND-CPA (INDistinguishability
// under Chosen Plaintext Attack) property.
int num_ind_cpa_public_key_bytes = 
    crystals_kyber_cryptosystem_engine_parameters
    .get_num_ind_cpa_public_key_bytes();

// Retrieval of the size, in terms of bits,
// of the asymmetric public key to be used by
// the CRYSTALS-Kyber Public-Key Cryptosystem
// with the IND-CPA (INDistinguishability
// under Chosen Plaintext Attack) property.
int num_ind_cpa_public_key_bits = 
    crystals_kyber_cryptosystem_engine_parameters
    .get_num_ind_cpa_public_key_bits();

// Retrieval of the size, in terms of bytes,
// of the asymmetric private key to be used by
// the CRYSTALS-Kyber Public-Key Cryptosystem
// with the IND-CPA (INDistinguishability
// under Chosen Plaintext Attack) property.
int num_ind_cpa_secret_key_bytes = 
    crystals_kyber_cryptosystem_engine_parameters
    .get_num_ind_cpa_secret_key_bytes();

// Retrieval of the size, in terms of bits,
// of the asymmetric private key to be used by
// the CRYSTALS-Kyber Public-Key Cryptosystem
// with the IND-CPA (INDistinguishability
// under Chosen Plaintext Attack) property.
int num_ind_cpa_secret_key_bits = 
    crystals_kyber_cryptosystem_engine_parameters
    .get_num_ind_cpa_secret_key_bits();

// Retrieval of the size, in terms of bytes,
// of the numeric polynomial, which is related to
// the asymmetric encryption to be used by
// the CRYSTALS-Kyber Public-Key Cryptosystem
// with the IND-CPA (INDistinguishability
// under Chosen Plaintext Attack) property.
int num_ind_cpa_polynomial_bytes = 
    crystals_kyber_cryptosystem_engine_parameters
    .get_num_ind_cpa_polynomial_bytes();

// Retrieval of the size, in terms of bits,
// of the numeric polynomial, which is related to
// the asymmetric encryption to be used by
// the CRYSTALS-Kyber Public-Key Cryptosystem
// with the IND-CPA (INDistinguishability
// under Chosen Plaintext Attack) property.
int num_ind_cpa_polynomial_bits = 
    crystals_kyber_cryptosystem_engine_parameters
    .get_num_ind_cpa_polynomial_bits();

// Retrieval of the size of an IND-CPA (INDistinguishable
// under Chosen Plaintext Attack) message, in terms of bytes,
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
int num_ind_cpa_message_bytes =
    crystals_kyber_cryptosystem_engine_parameters.get_num_ind_cpa_message_bytes();

// Retrieval of the size of an IND-CPA (INDistinguishable
// under Chosen Plaintext Attack) message, in terms of bits,
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
int num_ind_cpa_message_bits =
    crystals_kyber_cryptosystem_engine_parameters.get_num_ind_cpa_message_bits();


// Print of the string with the size of IND-CPA (INDistinguishable
// under Chosen Plaintext Attack) Pair of Asymmetric Keys, Polynomial and Message,
// in terms of bytes and bits, to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
System.out.println( "     * Size of IND-CPA (INDistinguishable under Chosen Plaintext Attack)\n" +
                    "       Pair of Asymmetric Keys, Polynomial and Message:\n" +
                    "       " + "- Size of an IND-CPA Public Key Kₚ: " 
                              + num_ind_cpa_public_key_bytes + " bytes | " + num_ind_cpa_public_key_bits + " bits\n" +
                    "       " + "- Size of an IND-CPA (Secret) Private Key Kₛ: "
                              + num_ind_cpa_secret_key_bytes + " bytes | " + num_ind_cpa_secret_key_bits + " bits\n" +
                    "       " + "- Size of an IND-CPA Polynomial X: "
                              + num_ind_cpa_polynomial_bytes + " bytes | " + num_ind_cpa_polynomial_bits + " bits\n" +
                    "       " + "- Size of an IND-CPA Message: "
                              + num_ind_cpa_message_bytes + " bytes | " + num_ind_cpa_message_bits + " bits\n" );
                   
// Print of one blank line, as a separator.
System.out.println("");


// Retrieval of the size, in terms of bytes,
// of the asymmetric public key to be used by
// the CRYSTALS-Kyber Public-Key Cryptosystem's
// KEM (Key Encapsulation Method/Mechanism)
// with the IND-CCA2 (INDistinguishability
// under adaptive Chosen Ciphertext Attack) property.
int num_ind_cca_kem_public_key_bytes = 
    crystals_kyber_cryptosystem_engine_parameters
    .get_num_ind_cca_kem_public_key_bytes();

// Retrieval of the size, in terms of bits,
// of the asymmetric public key to be used by
// the CRYSTALS-Kyber Public-Key Cryptosystem's
// KEM (Key Encapsulation Method/Mechanism)
// with the IND-CCA2 (INDistinguishability
// under adaptive Chosen Ciphertext Attack) property.
int num_ind_cca_kem_public_key_bits = 
    crystals_kyber_cryptosystem_engine_parameters
    .get_num_ind_cca_kem_public_key_bits();

// Retrieval of the size, in terms of bytes,
// of the asymmetric private key to be used by
// the CRYSTALS-Kyber Public-Key Cryptosystem's
// KEM (Key Encapsulation Method/Mechanism)
// with the IND-CCA2 (INDistinguishability
// under adaptive Chosen Ciphertext Attack) property.
int num_ind_cca_kem_secret_key_bytes = 
    crystals_kyber_cryptosystem_engine_parameters
    .get_num_ind_cca_kem_secret_key_bytes();

// Retrieval of the size, in terms of bits,
// of the asymmetric private key to be used by
// the CRYSTALS-Kyber Public-Key Cryptosystem's
// KEM (Key Encapsulation Method/Mechanism)
// with the IND-CCA2 (INDistinguishability
// under adaptive Chosen Ciphertext Attack) property.
int num_ind_cca_kem_secret_key_bits = 
    crystals_kyber_cryptosystem_engine_parameters
    .get_num_ind_cca_kem_secret_key_bits();

// Retrieval of the size, in terms of bytes,
// of the ciphertext generated and to be used by
// the CRYSTALS-Kyber Public-Key Cryptosystem's
// KEM (Key Encapsulation Method/Mechanism)
// with the IND-CCA2 (INDistinguishability
// under adaptive Chosen Ciphertext Attack) property.
int num_ind_cca_kem_ciphertext_bytes = 
    crystals_kyber_cryptosystem_engine_parameters
    .get_num_ind_cca_kem_ciphertext_bytes();

// Retrieval of the size, in terms of bits,
// of the ciphertext generated and to be used by
// the CRYSTALS-Kyber Public-Key Cryptosystem's
// KEM (Key Encapsulation Method/Mechanism)
// with the IND-CCA2 (INDistinguishability
// under adaptive Chosen Ciphertext Attack) property.
int num_ind_cca_kem_ciphertext_bits = 
    crystals_kyber_cryptosystem_engine_parameters
    .get_num_ind_cca_kem_ciphertext_bits();


// Print of the string with the size of IND-CCA2
// (INDistinguishable under adaptive Chosen Ciphertext Attack)
// Pair of Asymmetric Keys and Ciphertext, in terms of bytes and bits, to be used by
// the CRYSTALS-Kyber Public-Key Cryptosystem's KEM (Key Encapsulation Method/Mechanism).
System.out.println( "     * Size of IND-CCA2 (INDistinguishable under adaptive Chosen Ciphertext Attack)\n" +
                    "       Pair of Asymmetric Keys and Ciphertext, to be used by the internal KEM\n" +
                    "       (Key Encapsulation Method/Mechanism), allowing the distribution of the symmetric key:\n" +
                    "       " + "- Size of an IND-CCA2 Public Key Kₚ: " 
                              + num_ind_cca_kem_public_key_bytes + " bytes | " + num_ind_cca_kem_public_key_bits + " bits\n" +
                    "       " + "- Size of an IND-CCA2 (Secret) Private Key Kₛ: "
                              + num_ind_cpa_secret_key_bytes + " bytes | " + num_ind_cpa_secret_key_bits + " bits\n" +
                    "       " + "- Size of an IND-CCA2 Ciphertext: "
                              + num_ind_cca_kem_ciphertext_bytes + " bytes | " + num_ind_cca_kem_ciphertext_bits + " bits\n" );

// Print of one blank line, as a separator.
System.out.println("");


// Retrieval of the size of the symmetric session key, in terms of bytes,
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
int num_session_key_bytes = 
    crystals_kyber_cryptosystem_engine_parameters
    .get_num_session_key_bytes();

// Retrieval of the size of the symmetric session key, in terms of bits,
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
int num_session_key_bits = 
    crystals_kyber_cryptosystem_engine_parameters
    .get_num_session_key_bits();


// Validation for the size of the symmetric session key, in terms of bytes,
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
if( crystals_kyber_cryptosystem_session_key_bytes_size != num_session_key_bytes ) {
  
    // Print of an error message, regarding an invalid size of
    // the symmetric session key, in terms of bytes,
    // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    System.err.println("Invalid size of the Symmetric Session Key!");
    
}

// Initialization of the key generation parameters to be
// used by the CRYSTALS-Kyber Public-Key Cryptosystem.
crystals_kyber_cryptosystem.initialize_key_generation_parameters();

// Initialization of the generation of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
crystals_kyber_cryptosystem.initialize_asymmetric_key_pair_generation();


// Print of one bold line, as a separator.
System.out.println( "     ===============================================" +
                         "===============================================\n" );


// Print of a header for the parameters and
// information about the asymmetric public key to be
// used by the CRYSTALS-Kyber Public-Key Cryptosystem.
System.out.println( "       PARAMETERS AND INFORMATION ABOUT THE ASYMMETRIC PUBLIC KEY:\n\n" );


// Retrieval of the size, in terms of bytes, of the target/solution t
// (i.e. the target/solution vector for the mathematical equation A x s + e = t),
// encoded (and compressed) in bytes, representing a lattice-based problem in
// the form of a LWE (Learning With Errors), retrieved from the public key parameters of
// the pair of asymmetric keys to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
int target_solution_t_size_bytes_from_public_key_parameters =
    crystals_kyber_cryptosystem.get_target_solution_t_size_bytes_from_public_key_parameters();

// Retrieval of the size, in terms of bits, of the target/solution t
// (i.e. the target/solution vector for the mathematical equation A x s + e = t),
// encoded (and compressed) in bytes, representing a lattice-based problem in
// the form of a LWE (Learning With Errors), retrieved from the public key parameters of
// the pair of asymmetric keys to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
int target_solution_t_size_bits_from_public_key_parameters =
    crystals_kyber_cryptosystem.get_target_solution_t_size_bits_from_public_key_parameters();


// Retrieval of the target/solution t (i.e. the target/solution vector for
// the mathematical equation A x s + e = t), encoded (and compressed) in bytes,
// representing a lattice-based problem in the form of a LWE (Learning With Errors),
// retrieved from the public key parameters of the pair of asymmetric keys to be
// used by the CRYSTALS-Kyber Public-Key Cryptosystem.
byte[] target_solution_t_from_public_key_parameters_bytes = 
    crystals_kyber_cryptosystem.get_target_solution_t_from_public_key_parameters();


// Initialization of a String Buffer for the string representation
// in hexadecimal of the target/solution t (i.e. the target/solution vector for
// the mathematical equation A x s + e = t), encoded (and compressed) in bytes,
// representing a lattice-based problem in the form of a LWE (Learning With Errors),
// retrieved from the public key parameters of the pair of asymmetric keys to be
// used by the CRYSTALS-Kyber Public-Key Cryptosystem. 
StringBuffer target_solution_t_from_public_key_parameters_hexadecimal_buffer = new StringBuffer();


// Initialization of the index for the target/solution t
// (i.e. the target/solution vector for the mathematical equation A x s + e = t),
// encoded (and compressed) in bytes, representing a lattice-based problem in
// the form of a LWE (Learning With Errors), retrieved from the public key parameters of
// the pair of asymmetric keys to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.  
int target_solution_t_from_public_key_parameters_curr_byte_idx = 0;

// For each byte of the target/solution t (i.e. the target/solution vector
// for the mathematical equation A x s + e = t), encoded (and compressed) in bytes,
// representing a lattice-based problem in the form of a LWE (Learning With Errors),
// retrieved from the public key parameters of the pair of asymmetric keys to be
// used by the CRYSTALS-Kyber Public-Key Cryptosystem.  
for( byte target_solution_t_from_public_key_parameters_byte :
        target_solution_t_from_public_key_parameters_bytes ) {
    
    // Conversion of the current byte of the target/solution t (i.e. the target/solution vector
    // for the mathematical equation A x s + e = t), encoded (and compressed) in bytes,
    // representing a lattice-based problem in the form of a LWE (Learning With Errors),
    // retrieved from the public key parameters of the pair of asymmetric keys to be
    // used by the CRYSTALS-Kyber Public-Key Cryptosystem, to a hexadecimal character.
    target_solution_t_from_public_key_parameters_hexadecimal_buffer
        .append( Integer.toString( ( target_solution_t_from_public_key_parameters_byte & 0xff ) +
                                   0x100, NUM_POSSIBLE_BITS_IN_HEXADECIMAL_CHAR ).substring(1) );
    
    // If the current index of the byte of the target/solution t (i.e. the target/solution vector
    // for the mathematical equation A x s + e = t), encoded (and compressed) in bytes,
    // representing a lattice-based problem in the form of a LWE (Learning With Errors),
    // retrieved from the public key parameters of the pair of asymmetric keys to be
    // used by the CRYSTALS-Kyber Public-Key Cryptosystem, is a multiple of two.
    if( target_solution_t_from_public_key_parameters_curr_byte_idx % 2 == 0 ) {
        
        // Concatenation of a 'blank space' to the String Buffer for the string representation
        // in hexadecimal of the target/solution t (i.e. the target/solution vector for
        // the mathematical equation A x s + e = t), encoded (and compressed) in bytes,
        // representing a lattice-based problem in the form of a LWE (Learning With Errors),
        // retrieved from the public key parameters of the pair of asymmetric keys to be
        // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        target_solution_t_from_public_key_parameters_hexadecimal_buffer.append(" ");
        
    }
    // If the current index of the byte of the target/solution t (i.e. the target/solution vector
    // for the mathematical equation A x s + e = t), encoded (and compressed) in bytes,
    // representing a lattice-based problem in the form of a LWE (Learning With Errors),
    // retrieved from the public key parameters of the pair of asymmetric keys to be
    // used by the CRYSTALS-Kyber Public-Key Cryptosystem, is not a multiple of two.
    else {
        
        // If the current index of the byte of the target/solution t (i.e. the target/solution vector
        // for the mathematical equation A x s + e = t), encoded (and compressed) in bytes,
        // representing a lattice-based problem in the form of a LWE (Learning With Errors),
        // retrieved from the public key parameters of the pair of asymmetric keys to be
        // used by the CRYSTALS-Kyber Public-Key Cryptosystem, is not the last one.
        if( target_solution_t_from_public_key_parameters_curr_byte_idx < 
            ( target_solution_t_from_public_key_parameters_bytes.length - 1 ) ) {
        
            // Concatenation of a 'delimeter' ( ; ) to the String Buffer for the string representation
            // in hexadecimal of the target/solution t (i.e. the target/solution vector for
            // the mathematical equation A x s + e = t), encoded (and compressed) in bytes,
            // representing a lattice-based problem in the form of a LWE (Learning With Errors),
            // retrieved from the public key parameters of the pair of asymmetric keys to be
            // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
            target_solution_t_from_public_key_parameters_hexadecimal_buffer.append(";");
        
        }
        
    }
    
    // Incrementation of the index for the target/solution t
    // (i.e. the target/solution vector for the mathematical equation A x s + e = t),
    // encoded (and compressed) in bytes, representing a lattice-based problem in
    // the form of a LWE (Learning With Errors), retrieved from the public key parameters of
    // the pair of asymmetric keys to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.  
    target_solution_t_from_public_key_parameters_curr_byte_idx++;
    
}


// Initialization of the string representation in hexadecimal of
// the target/solution t (i.e. the target/solution vector for
// the mathematical equation A x s + e = t), encoded (and compressed) in bytes,
// representing a lattice-based problem in the form of a LWE (Learning With Errors),
// retrieved from the public key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem,
// from the upper-case values of the respective String Buffer.
String target_solution_t_from_public_key_parameters_hexadecimal_string = 
    target_solution_t_from_public_key_parameters_hexadecimal_buffer.toString().toUpperCase();

// Initialization of the string array representation of the several
// hexadecimal characters of the target/solution t (i.e. the target/solution vector
// for the mathematical equation A x s + e = t), encoded (and compressed) in bytes,
// representing a lattice-based problem in the form of a LWE (Learning With Errors),
// retrieved from the public key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem,
// from the upper-case values of the respective String Buffer.
String[] target_solution_t_from_public_key_parameters_hexadecimal_chars = 
    target_solution_t_from_public_key_parameters_hexadecimal_string
        .split( ";", target_solution_t_from_public_key_parameters_hexadecimal_string.length() );

// Re-Initialization of the string representation in hexadecimal of
// the target/solution t (i.e. the target/solution vector for
// the mathematical equation A x s + e = t), encoded (and compressed) in bytes,
// representing a lattice-based problem in the form of a LWE (Learning With Errors),
// retrieved from the public key parameters of the pair of asymmetric keys to be
// used by the CRYSTALS-Kyber Public-Key Cryptosystem. 
target_solution_t_from_public_key_parameters_hexadecimal_string = "";

// Concatenation of one bold line, as a separator, to the string representation
// in hexadecimal of the target/solution t (i.e. the target/solution vector for
// the mathematical equation A x s + e = t), encoded (and compressed) in bytes,
// representing a lattice-based problem in the form of a LWE (Learning With Errors),
// retrieved from the public key parameters of the pair of asymmetric keys to be
// used by the CRYSTALS-Kyber Public-Key Cryptosystem. 
target_solution_t_from_public_key_parameters_hexadecimal_string += 
            "=====================================" +
            "=====================================" + 
            "\n          ";


// For each index of the numerical values of the target/solution t
// (i.e. the target/solution vector for the mathematical equation A x s + e = t),
// encoded (and compressed) in bytes, representing a lattice-based problem in
// the form of a LWE (Learning With Errors), retrieved from the public key parameters of
// the pair of asymmetric keys to be used by the CRYSTALS-Kyber Public-Key Cryptosystem. 
for( int t_i = 0; t_i < target_solution_t_from_public_key_parameters_hexadecimal_chars.length; t_i++ ) {
    
    // If the current index of the numerical value is not
    // the first one, and it is not a multiple of four.
    if( ( t_i != 0 ) && ( t_i % 4 == 0 ) ) {
        
        // Concatenation of one bold line, as a separator, to the string representation
        // in hexadecimal of the target/solution t (i.e. the target/solution vector for
        // the mathematical equation A x s + e = t), encoded (and compressed) in bytes,
        // representing a lattice-based problem in the form of a LWE (Learning With Errors),
        // retrieved from the public key parameters of the pair of asymmetric keys to be
        // used by the CRYSTALS-Kyber Public-Key Cryptosystem. 
        target_solution_t_from_public_key_parameters_hexadecimal_string += 
            "\n          " +
            "=====================================" +
            "=====================================" + 
            "\n          ";
        
    }
    
    // Concatenation of the current hexadecimal characters, to the string representation
    // in hexadecimal of the target/solution t (i.e. the target/solution vector for
    // the mathematical equation A x s + e = t), encoded (and compressed) in bytes,
    // representing a lattice-based problem in the form of a LWE (Learning With Errors),
    // retrieved from the public key parameters of the pair of asymmetric keys to be
    // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    target_solution_t_from_public_key_parameters_hexadecimal_string += 
        " t" + "[" + t_i + "]" + " = " + 
                target_solution_t_from_public_key_parameters_hexadecimal_chars[t_i] + "; ";
    
}

// Concatenation of one bold line, as a separator, to the string representation
// in hexadecimal of the target/solution t (i.e. the target/solution vector for
// the mathematical equation A x s + e = t), encoded (and compressed) in bytes,
// representing a lattice-based problem in the form of a LWE (Learning With Errors),
// retrieved from the public key parameters of the pair of asymmetric keys to be
// used by the CRYSTALS-Kyber Public-Key Cryptosystem.
target_solution_t_from_public_key_parameters_hexadecimal_string += 
            "\n          " +
            "=====================================" +
            "=====================================";


// Initialization of a (short) integer values array of
// the target/solution t (i.e. the target/solution vector for
// the mathematical equation A x s + e = t), encoded (and compressed) in bytes,
// representing a lattice-based problem in the form of a LWE (Learning With Errors),
// retrieved from the public key parameters of the pair of asymmetric keys to be
// used by the CRYSTALS-Kyber Public-Key Cryptosystem. 
short[] target_solution_t_from_public_key_parameters_values =
    new short[ ( target_solution_t_size_bytes_from_public_key_parameters / 2 ) ];
    
// Conversion of the target/solution t (i.e. the target/solution vector for
// the mathematical equation A x s + e = t), encoded (and compressed) in bytes,
// representing a lattice-based problem in the form of a LWE (Learning With Errors),
// retrieved from the public key parameters of the pair of asymmetric keys to be
// used by the CRYSTALS-Kyber Public-Key Cryptosystem,
// from a byte array to a (short) integer array.
ByteBuffer.wrap( target_solution_t_from_public_key_parameters_bytes )
    .order( ByteOrder.LITTLE_ENDIAN ).asShortBuffer()
    .get( target_solution_t_from_public_key_parameters_values );


// Initialization of the string representation in (short)
// integer values of the target/solution t (i.e. the target/solution vector for
// the mathematical equation A x s + e = t), encoded (and compressed) in bytes,
// representing a lattice-based problem in the form of a LWE (Learning With Errors),
// retrieved from the public key parameters of the pair of asymmetric keys to be
// used by the CRYSTALS-Kyber Public-Key Cryptosystem. 
String target_solution_t_from_public_key_parameters_values_string = "";

// Concatenation of one bold line, as a separator, to the string representation
// in (short) integer values of the target/solution t (i.e. the target/solution vector for
// the mathematical equation A x s + e = t), encoded (and compressed) in bytes,
// representing a lattice-based problem in the form of a LWE (Learning With Errors),
// retrieved from the public key parameters of the pair of asymmetric keys to be
// used by the CRYSTALS-Kyber Public-Key Cryptosystem.
target_solution_t_from_public_key_parameters_values_string += 
            "=====================================" +
            "=====================================" + 
            "\n          ";


// For each index of the numerical values of the target/solution t
// (i.e. the target/solution vector for the mathematical equation A x s + e = t),
// encoded (and compressed) in bytes, representing a lattice-based problem in
// the form of a LWE (Learning With Errors), retrieved from the public key parameters of
// the pair of asymmetric keys to be used by the CRYSTALS-Kyber Public-Key Cryptosystem. 
for( int t_i = 0; t_i < target_solution_t_from_public_key_parameters_values.length; t_i++ ) {
    
    // If the current index of the numerical value is not
    // the first one, and it is not a multiple of four.
    if( ( t_i != 0 ) && ( t_i % 4 == 0 ) ) {
        
        // Concatenation of one bold line, as a separator, to the string representation
        // in (short) integer values of the target/solution t (i.e. the target/solution vector for
        // the mathematical equation A x s + e = t), encoded (and compressed) in bytes,
        // representing a lattice-based problem in the form of a LWE (Learning With Errors),
        // retrieved from the public key parameters of the pair of asymmetric keys to be
        // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        target_solution_t_from_public_key_parameters_values_string += 
            "\n          " +
            "=====================================" +
            "=====================================" + 
            "\n          ";
        
    }
    
    // Concatenation of the current numerical value, to the string representation
    // in (short) integer values of the target/solution t (i.e. the target/solution vector for
    // the mathematical equation A x s + e = t), encoded (and compressed) in bytes,
    // representing a lattice-based problem in the form of a LWE (Learning With Errors),
    // retrieved from the public key parameters of the pair of asymmetric keys to be
    // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    target_solution_t_from_public_key_parameters_values_string += 
        " t" + "[" + t_i + "]" + " = " + 
                target_solution_t_from_public_key_parameters_values[t_i] + "; ";
    
}

// Concatenation of one bold line, as a separator, to the string representation
// in (short) integer values of the target/solution t (i.e. the target/solution vector for
// the mathematical equation A x s + e = t), encoded (and compressed) in bytes,
// representing a lattice-based problem in the form of a LWE (Learning With Errors),
// retrieved from the public key parameters of the pair of asymmetric keys to be
// used by the CRYSTALS-Kyber Public-Key Cryptosystem.
target_solution_t_from_public_key_parameters_values_string += 
            "\n          " +
            "=====================================" +
            "=====================================";


// Print of a header for the information about
// the target/solution t (i.e. the target/solution vector for
// the mathematical equation A x s + e = t), encoded (and compressed) in bytes,
// representing a lattice-based problem in the form of a LWE (Learning With Errors),
// retrieved from the public key parameters of the pair of asymmetric keys to be
// used by the CRYSTALS-Kyber Public-Key Cryptosystem.
System.out.println( "        " + "+ t (Target/Solution Vector) " + 
                           "[" +   target_solution_t_size_bytes_from_public_key_parameters + " bytes" + " | " + 
                                 ( target_solution_t_size_bytes_from_public_key_parameters / 2 ) + " short integers" + " | " + 
                                   target_solution_t_size_bits_from_public_key_parameters + " bits" + "]:" );

// Print of one blank line, as a separator.
System.out.println("\n");

// Print of the string representation in hexadecimal of
// the target/solution t (i.e. the target/solution vector for
// the mathematical equation A x s + e = t), encoded (and compressed) in bytes,
// representing a lattice-based problem in the form of a LWE (Learning With Errors),
// retrieved from the public key parameters of the pair of asymmetric keys to be
// used by the CRYSTALS-Kyber Public-Key Cryptosystem.
System.out.println( "        " + "  Hexadecimal Values: \n  " +
                    "        " + target_solution_t_from_public_key_parameters_hexadecimal_string );

// Print of two blank lines, as separators.
System.out.println("");
System.out.println("");

// Print of the string representation in (short) integer
// values of the target/solution t (i.e. the target/solution vector for
// the mathematical equation A x s + e = t), encoded (and compressed) in bytes,
// representing a lattice-based problem in the form of a LWE (Learning With Errors),
// retrieved from the public key parameters of the pair of asymmetric keys to be
// used by the CRYSTALS-Kyber Public-Key Cryptosystem.
System.out.println( "        " + "  (Short) Integer Values: \n  " +
                    "        " + target_solution_t_from_public_key_parameters_values_string );


// Print of one blank line, as a separator.
System.out.println("");

// Print of one bold line, as a separator.
System.out.println( "       " + "^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^"
                              + "^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^" );

// Print of one blank line, as a separator.
System.out.println("");


// Retrieval of the size, in terms of bytes, of the (pseudo) random seed ρ (rho)
// (i.e. the random seed from which will be expanded and obtained the uniformly-random matrix A
// consisting of polynomials representing a lattice algebraic structure for
// the mathematical equation A x s + e = t), encoded in bytes, representing
// a lattice-based problem in the form of a MLWE (Module-Learning With Errors),
// retrieved from the public key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
int random_seed_rho_size_bytes_from_public_key_parameters =
    crystals_kyber_cryptosystem.get_random_seed_rho_size_bytes_from_public_key_parameters();

// Retrieval of the size, in terms of bits, of the (pseudo) random seed ρ (rho)
// (i.e. the random seed from which will be expanded and obtained the uniformly-random matrix A
// consisting of polynomials representing a lattice algebraic structure for
// the mathematical equation A x s + e = t), encoded in bytes, representing
// a lattice-based problem in the form of a MLWE (Module-Learning With Errors),
// retrieved from the public key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
int random_seed_rho_size_bits_from_public_key_parameters =
    crystals_kyber_cryptosystem.get_random_seed_rho_size_bits_from_public_key_parameters();


// Retrieval of the (pseudo) random seed ρ (rho) (i.e. the random seed
// from which will be expanded and obtained the uniformly-random matrix A
// consisting of polynomials representing a lattice algebraic structure for
// the mathematical equation A x s + e = t), encoded in bytes, representing
// a lattice-based problem in the form of a MLWE (Module-Learning With Errors),
// retrieved from the public key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
byte[] random_seed_rho_from_public_key_parameters_bytes = 
    crystals_kyber_cryptosystem.get_random_seed_rho_from_public_key_parameters();


// Initialization of a String Buffer for the string representation
// in hexadecimal of the (pseudo) random seed ρ (rho) (i.e. the random seed
// from which will be expanded and obtained the uniformly-random matrix A
// consisting of polynomials representing a lattice algebraic structure for
// the mathematical equation A x s + e = t), encoded in bytes, representing
// a lattice-based problem in the form of a MLWE (Module-Learning With Errors),
// retrieved from the public key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
StringBuffer random_seed_rho_from_public_key_parameters_hexadecimal_buffer = 
    new StringBuffer();


// Initialization of the index for the (pseudo) random seed ρ (rho)
// (i.e. the random seed from which will be expanded and obtained
// the uniformly-random matrix A consisting of polynomials representing
// a lattice algebraic structure for the mathematical equation A x s + e = t),
// encoded in bytes, representing a lattice-based problem in the form of
// a MLWE (Module-Learning With Errors), retrieved from the public key parameters of
// the pair of asymmetric keys to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
int random_seed_rho_from_public_key_parameters_curr_byte_idx = 0;

// For each byte of the (pseudo) random seed ρ (rho) (i.e. the random seed
// from which will be expanded and obtained the uniformly-random matrix A
// consisting of polynomials representing a lattice algebraic structure for
// the mathematical equation A x s + e = t), encoded in bytes, representing
// a lattice-based problem in the form of a MLWE (Module-Learning With Errors),
// retrieved from the public key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
for( byte random_seed_rho_from_public_key_parameters_byte :
        random_seed_rho_from_public_key_parameters_bytes ) {
    
    // Conversion of the current byte of the (pseudo) random seed ρ (rho)
    // (i.e. the random seed from which will be expanded and obtained
    // the uniformly-random matrix A consisting of polynomials representing
    // a lattice algebraic structure for the mathematical equation A x s + e = t),
    // encoded in bytes, representing a lattice-based problem in the form of
    // a MLWE (Module-Learning With Errors), retrieved from the public key parameters of
    // the pair of asymmetric keys to be used by the CRYSTALS-Kyber Public-Key Cryptosystem,
    // to a hexadecimal character.
    random_seed_rho_from_public_key_parameters_hexadecimal_buffer
        .append( Integer.toString( ( random_seed_rho_from_public_key_parameters_byte & 0xff ) +
                                   0x100, NUM_POSSIBLE_BITS_IN_HEXADECIMAL_CHAR ).substring(1) );
    

    // If the current index of the byte of the (pseudo) random seed ρ (rho)
    // (i.e. the random seed from which will be expanded and obtained
    // the uniformly-random matrix A consisting of polynomials representing
    // a lattice algebraic structure for the mathematical equation A x s + e = t),
    // encoded in bytes, representing a lattice-based problem in the form of
    // a MLWE (Module-Learning With Errors), retrieved from the public key parameters of
    // the pair of asymmetric keys to be used by the CRYSTALS-Kyber Public-Key Cryptosystem,
    // is not the last one.
    if( random_seed_rho_from_public_key_parameters_curr_byte_idx < 
        ( random_seed_rho_from_public_key_parameters_bytes.length - 1 ) ) {

        // Concatenation of a 'delimeter' ( ; ) to the String Buffer for the string representation
        // in hexadecimal of the (pseudo) random seed ρ (rho) (i.e. the random seed
        // from which will be expanded and obtained the uniformly-random matrix A
        // consisting of polynomials representing a lattice algebraic structure for
        // the mathematical equation A x s + e = t), encoded in bytes, representing
        // a lattice-based problem in the form of a MLWE (Module-Learning With Errors),
        // retrieved from the public key parameters of the pair of asymmetric keys
        // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        random_seed_rho_from_public_key_parameters_hexadecimal_buffer.append(";");

    }

    
    // Incrementation of the index for the (pseudo) random seed ρ (rho)
    // (i.e. the random seed from which will be expanded and obtained
    // the uniformly-random matrix A consisting of polynomials representing
    // a lattice algebraic structure for the mathematical equation A x s + e = t),
    // encoded in bytes, representing a lattice-based problem in the form of
    // a MLWE (Module-Learning With Errors), retrieved from the public key parameters of
    // the pair of asymmetric keys to be used by the CRYSTALS-Kyber Public-Key Cryptosystem. 
    random_seed_rho_from_public_key_parameters_curr_byte_idx++;
    
}


// Initialization of the string representation in hexadecimal of
// the (pseudo) random seed ρ (rho) (i.e. the random seed from
// which will be expanded and obtained the uniformly-random matrix A
// consisting of polynomials representing a lattice algebraic structure
// for the mathematical equation A x s + e = t), encoded in bytes, representing
// a lattice-based problem in the form of a MLWE (Module-Learning With Errors),
// retrieved from the public key parameters of the pair of asymmetric keys to be
// used by the CRYSTALS-Kyber Public-Key Cryptosystem,
// from the upper-case values of the respective String Buffer.
String random_seed_rho_from_public_key_parameters_hexadecimal_string = 
    random_seed_rho_from_public_key_parameters_hexadecimal_buffer.toString().toUpperCase();

// Initialization of the string array representation of the several
// hexadecimal characters of the (pseudo) random seed ρ (rho)
// (i.e. the random seed from which will be expanded and obtained
// the uniformly-random matrix A consisting of polynomials representing
// a lattice algebraic structure for the mathematical equation A x s + e = t),
// encoded in bytes, representing a lattice-based problem in the form of
// a MLWE (Module-Learning With Errors), retrieved from the public key parameters of
// the pair of asymmetric keys to be used by the CRYSTALS-Kyber Public-Key Cryptosystem,
// from the upper-case values of the respective String Buffer.
String[] random_seed_rho_from_public_key_parameters_hexadecimal_chars = 
    random_seed_rho_from_public_key_parameters_hexadecimal_string
        .split( ";", random_seed_rho_from_public_key_parameters_hexadecimal_string.length() );

// Re-Initialization of the string representation in hexadecimal of
// the (pseudo) random seed ρ (rho) (i.e. the random seed from
// which will be expanded and obtained the uniformly-random matrix A
// consisting of polynomials representing a lattice algebraic structure
// for the mathematical equation A x s + e = t), encoded in bytes, representing
// a lattice-based problem in the form of a MLWE (Module-Learning With Errors),
// retrieved from the public key parameters of the pair of asymmetric keys to be
// used by the CRYSTALS-Kyber Public-Key Cryptosystem.
random_seed_rho_from_public_key_parameters_hexadecimal_string = "";

// Concatenation of one bold line, as a separator, to the string representation
// in hexadecimal of the (pseudo) random seed ρ (rho) (i.e. the random seed from
// which will be expanded and obtained the uniformly-random matrix A
// consisting of polynomials representing a lattice algebraic structure
// for the mathematical equation A x s + e = t), encoded in bytes, representing
// a lattice-based problem in the form of a MLWE (Module-Learning With Errors),
// retrieved from the public key parameters of the pair of asymmetric keys to be
// used by the CRYSTALS-Kyber Public-Key Cryptosystem. 
random_seed_rho_from_public_key_parameters_hexadecimal_string += 
            "=====================================" +
            "=====================================" + 
            "\n          ";


// For each index of the numerical values of the (pseudo) random seed ρ (rho)
// (i.e. the random seed from which will be expanded and obtained the uniformly-random matrix A
// consisting of polynomials representing a lattice algebraic structure for the mathematical
// equation A x s + e = t), encoded in bytes, representing a lattice-based problem in the form of
// a MLWE (Module-Learning With Errors), retrieved from the public key parameters of the pair of
// asymmetric keys to be used by the CRYSTALS-Kyber Public-Key Cryptosystem. 
for( int rho_i = 0; rho_i < random_seed_rho_from_public_key_parameters_hexadecimal_chars.length; rho_i++ ) {
    
    // If the current index of the numerical value is not
    // the first one, and it is not a multiple of four.
    if( ( rho_i != 0 ) && ( rho_i % 4 == 0 ) ) {
        
        // Concatenation of one bold line, as a separator, to the string representation
        // in hexadecimal of the (pseudo) random seed ρ (rho) (i.e. the random seed from
        // which will be expanded and obtained the uniformly-random matrix A
        // consisting of polynomials representing a lattice algebraic structure
        // for the mathematical equation A x s + e = t), encoded in bytes, representing
        // a lattice-based problem in the form of a MLWE (Module-Learning With Errors),
        // retrieved from the public key parameters of the pair of asymmetric keys to be
        // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        random_seed_rho_from_public_key_parameters_hexadecimal_string += 
            "\n          " +
            "=====================================" +
            "=====================================" + 
            "\n          ";
        
    }
    
    // Concatenation of the current hexadecimal characters, to the string representation
    // in hexadecimal of the (pseudo) random seed ρ (rho) (i.e. the random seed from
    // which will be expanded and obtained the uniformly-random matrix A
    // consisting of polynomials representing a lattice algebraic structure
    // for the mathematical equation A x s + e = t), encoded in bytes, representing
    // a lattice-based problem in the form of a MLWE (Module-Learning With Errors),
    // retrieved from the public key parameters of the pair of asymmetric keys to be
    // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    random_seed_rho_from_public_key_parameters_hexadecimal_string += 
        " ρ" + "[" + rho_i + "]" + " = " + 
                random_seed_rho_from_public_key_parameters_hexadecimal_chars[rho_i] + "; ";
    
}

// Concatenation of one bold line, as a separator, to the string representation
// in hexadecimal of the (pseudo) random seed ρ (rho) (i.e. the random seed from
// which will be expanded and obtained the uniformly-random matrix A
// consisting of polynomials representing a lattice algebraic structure
// for the mathematical equation A x s + e = t), encoded in bytes, representing
// a lattice-based problem in the form of a MLWE (Module-Learning With Errors),
// retrieved from the public key parameters of the pair of asymmetric keys to be
// used by the CRYSTALS-Kyber Public-Key Cryptosystem.
random_seed_rho_from_public_key_parameters_hexadecimal_string += 
            "\n          " +
            "=====================================" +
            "=====================================";


// Initialization of the string representation in binary of
// the (pseudo) random seed ρ (rho) (i.e. the random seed from
// which will be expanded and obtained the uniformly-random matrix A
// consisting of polynomials representing a lattice algebraic structure
// for the mathematical equation A x s + e = t), encoded in bytes, representing
// a lattice-based problem in the form of a MLWE (Module-Learning With Errors),
// retrieved from the public key parameters of the pair of asymmetric keys to be
// used by the CRYSTALS-Kyber Public-Key Cryptosystem.
String random_seed_rho_from_public_key_parameters_binary_string = "ρ = {0,1}²⁵⁶ = ";


// Initialization of the index for the (pseudo) random seed ρ (rho)
// (i.e. the random seed from which will be expanded and obtained
// the uniformly-random matrix A consisting of polynomials representing
// a lattice algebraic structure for the mathematical equation A x s + e = t),
// encoded in bytes, representing a lattice-based problem in the form of
// a MLWE (Module-Learning With Errors), retrieved from the public key parameters of
// the pair of asymmetric keys to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
random_seed_rho_from_public_key_parameters_curr_byte_idx = 0;

// For each byte of the (pseudo) random seed ρ (rho) (i.e. the random seed
// from which will be expanded and obtained the uniformly-random matrix A
// consisting of polynomials representing a lattice algebraic structure for
// the mathematical equation A x s + e = t), encoded in bytes, representing
// a lattice-based problem in the form of a MLWE (Module-Learning With Errors),
// retrieved from the public key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
for( byte random_seed_rho_from_public_key_parameters_byte :
        random_seed_rho_from_public_key_parameters_bytes ) {

    // Conversion of the current byte of the (pseudo) random seed ρ (rho)
    // (i.e. the random seed from which will be expanded and obtained
    // the uniformly-random matrix A consisting of polynomials representing
    // a lattice algebraic structure for the mathematical equation A x s + e = t),
    // encoded in bytes, representing a lattice-based problem in the form of
    // a MLWE (Module-Learning With Errors), retrieved from the public key parameters of
    // the pair of asymmetric keys to be used by the CRYSTALS-Kyber Public-Key Cryptosystem,
    // to a binary character (i.e., a bit).
    random_seed_rho_from_public_key_parameters_binary_string += 
        Integer.toBinaryString( random_seed_rho_from_public_key_parameters_byte
                                & 255 | 256 ).substring(1);
    
    // If the current index of the byte of the (pseudo) random seed ρ (rho)
    // (i.e. the random seed from which will be expanded and obtained
    // the uniformly-random matrix A consisting of polynomials representing
    // a lattice algebraic structure for the mathematical equation A x s + e = t),
    // encoded in bytes, representing a lattice-based problem in the form of
    // a MLWE (Module-Learning With Errors), retrieved from the public key parameters of
    // the pair of asymmetric keys to be used by the CRYSTALS-Kyber Public-Key Cryptosystem,
    // is a multiple of six.
    if( ( ( random_seed_rho_from_public_key_parameters_curr_byte_idx + 1 ) % 6 ) == 0 ) {
        
        // Concatenation of a 'blank line' and a 'blank space' to the string representation
        // in binary of the (pseudo) random seed ρ (rho) (i.e. the random seed
        // from which will be expanded and obtained the uniformly-random matrix A
        // consisting of polynomials representing a lattice algebraic structure for
        // the mathematical equation A x s + e = t), encoded in bytes, representing
        // a lattice-based problem in the form of a MLWE (Module-Learning With Errors),
        // retrieved from the public key parameters of the pair of asymmetric keys
        // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        random_seed_rho_from_public_key_parameters_binary_string += 
            "\n                          ";
        
    }
    
    // Incrementation of the index for the (pseudo) random seed ρ (rho)
    // (i.e. the random seed from which will be expanded and obtained
    // the uniformly-random matrix A consisting of polynomials representing
    // a lattice algebraic structure for the mathematical equation A x s + e = t),
    // encoded in bytes, representing a lattice-based problem in the form of
    // a MLWE (Module-Learning With Errors), retrieved from the public key parameters of
    // the pair of asymmetric keys to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    random_seed_rho_from_public_key_parameters_curr_byte_idx++;
    
}


// Print of a header for the information about the (pseudo) random seed ρ (rho)
// (i.e. the random seed from which will be expanded and obtained the uniformly-random matrix A
// consisting of polynomials representing a lattice algebraic structure for
// the mathematical equation A x s + e = t), encoded in bytes, representing
// a lattice-based problem in the form of a MLWE (Module-Learning With Errors),
// retrieved from the public key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
System.out.println( "        " + "+ ρ (uniformly (pseudo) random seed) " + 
                           "[" + random_seed_rho_size_bytes_from_public_key_parameters + " bytes" + " | " + 
                                 random_seed_rho_size_bits_from_public_key_parameters + " bits" + "]:" );

// Print of one blank line, as a separator.
System.out.println("\n");

// Print of the string representation in hexadecimal of
// the (pseudo) random seed ρ (rho) (i.e. the random seed from
// which will be expanded and obtained the uniformly-random matrix A
// consisting of polynomials representing a lattice algebraic structure
// for the mathematical equation A x s + e = t), encoded in bytes, representing
// a lattice-based problem in the form of a MLWE (Module-Learning With Errors),
// retrieved from the public key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
System.out.println( "        " + "  Hexadecimal Values: \n  " +
                    "        " + random_seed_rho_from_public_key_parameters_hexadecimal_string );

// Print of two blank lines, as separators.
System.out.println("");
System.out.println("");

// Print of the string representation in binary of
// the (pseudo) random seed ρ (rho) (i.e. the random seed from
// which will be expanded and obtained the uniformly-random matrix A
// consisting of polynomials representing a lattice algebraic structure
// for the mathematical equation A x s + e = t), encoded in bytes, representing
// a lattice-based problem in the form of a MLWE (Module-Learning With Errors),
// retrieved from the public key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
System.out.println( "        " + "  Binary Values: \n   " +
                    "        " + random_seed_rho_from_public_key_parameters_binary_string );


// Print of one blank line, as a separator.
System.out.println("");


// Print of one bold line, as a separator.
System.out.println( "     ===============================================" +
                         "===============================================\n" );


// Print of a header for the parameters and
// information about the asymmetric private key to be
// used by the CRYSTALS-Kyber Public-Key Cryptosystem.
System.out.println( "       PARAMETERS AND INFORMATION ABOUT THE ASYMMETRIC PRIVATE KEY:\n\n" );


// Retrieval of the size, in terms of bytes, of the random secret s
// (i.e. the secret vector in the lattice algebraic structure), encoded in bytes,
// with the IND-CPA (INDistinguishability under Chosen Plaintext Attack) property,
// retrieved from the private key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
int random_secret_s_size_bytes_from_private_key_parameters =
    crystals_kyber_cryptosystem.get_random_secret_s_size_bytes_from_private_key_parameters();

// Retrieval of the size, in terms of bits, of the random secret s
// (i.e. the secret vector in the lattice algebraic structure), encoded in bytes,
// with the IND-CPA (INDistinguishability under Chosen Plaintext Attack) property,
// retrieved from the private key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
int random_secret_s_size_bits_from_private_key_parameters =
    crystals_kyber_cryptosystem.get_random_secret_s_size_bits_from_private_key_parameters();


// Retrieval of the random secret s (i.e. the secret vector in
// the lattice algebraic structure), encoded in bytes, with the IND-CPA
// (INDistinguishability under Chosen Plaintext Attack) property,
// retrieved from the private key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
byte[] random_secret_s_from_private_key_parameters_bytes = 
    crystals_kyber_cryptosystem.get_random_secret_s_from_private_key_parameters();


// Initialization of a String Buffer for the string representation
// in hexadecimal of the random secret s (i.e. the secret vector in
// the lattice algebraic structure), encoded in bytes, with the IND-CPA
// (INDistinguishability under Chosen Plaintext Attack) property,
// retrieved from the private key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem. 
StringBuffer random_secret_s_from_private_key_parameters_hexadecimal_buffer = new StringBuffer();


// Initialization of the index for the random secret s
// (i.e. the secret vector in the lattice algebraic structure),
// encoded in bytes, with the IND-CPA (INDistinguishability
// under Chosen Plaintext Attack) property, retrieved from
// the private key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.  
int random_secret_s_from_private_key_parameters_curr_byte_idx = 0;

// For each byte of the random secret s (i.e. the secret vector in
// the lattice algebraic structure), encoded in bytes, with the IND-CPA
// (INDistinguishability under Chosen Plaintext Attack) property,
// retrieved from the private key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
for( byte random_secret_s_from_private_key_parameters_byte :
        random_secret_s_from_private_key_parameters_bytes ) {
    
    // Conversion of the current byte of the random secret s
    // (i.e. the secret vector in the lattice algebraic structure),
    // encoded in bytes, with the IND-CPA (INDistinguishability
    // under Chosen Plaintext Attack) property, retrieved from
    // the private key parameters of the pair of asymmetric keys
    // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem,
    // to a hexadecimal character.
    random_secret_s_from_private_key_parameters_hexadecimal_buffer
        .append( Integer.toString( ( random_secret_s_from_private_key_parameters_byte & 0xff ) +
                                   0x100, NUM_POSSIBLE_BITS_IN_HEXADECIMAL_CHAR ).substring(1) );
    
    // If the current index of the byte of the random secret s
    // (i.e. the secret vector in the lattice algebraic structure),
    // encoded in bytes, with the IND-CPA (INDistinguishability
    // under Chosen Plaintext Attack) property, retrieved from
    // the private key parameters of the pair of asymmetric keys
    // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem,
    // is a multiple of two.
    if( ( random_secret_s_from_private_key_parameters_curr_byte_idx % 2 ) == 0 ) {
        
        // Concatenation of a 'blank space' to the String Buffer for the string representation
        // in hexadecimal of the random secret s (i.e. the secret vector in the lattice
        // algebraic structure), encoded in bytes, with the IND-CPA (INDistinguishability
        // under Chosen Plaintext Attack) property, retrieved from the private key parameters of
        // the pair of asymmetric keys to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        random_secret_s_from_private_key_parameters_hexadecimal_buffer.append(" ");
        
    }
    // If the current index of the byte of the random secret s
    // (i.e. the secret vector in the lattice algebraic structure),
    // encoded in bytes, with the IND-CPA (INDistinguishability
    // under Chosen Plaintext Attack) property, retrieved from
    // the private key parameters of the pair of asymmetric keys
    // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem,
    // is not a multiple of two.
    else {
        
        // If the current index of the byte of the random secret s
        // (i.e. the secret vector in the lattice algebraic structure),
        // encoded in bytes, with the IND-CPA (INDistinguishability
        // under Chosen Plaintext Attack) property, retrieved from
        // the private key parameters of the pair of asymmetric keys
        // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem,
        // is not the last one.
        if( random_secret_s_from_private_key_parameters_curr_byte_idx < 
            ( random_secret_s_from_private_key_parameters_bytes.length - 1 ) ) {
        
            // Concatenation of a 'delimeter' ( ; ) to the String Buffer for
            // the string representation in hexadecimal of the random secret s
            // (i.e. the secret vector in the lattice algebraic structure),
            // encoded in bytes, with the IND-CPA (INDistinguishability
            // under Chosen Plaintext Attack) property, retrieved from
            // the private key parameters of the pair of asymmetric keys
            // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
            random_secret_s_from_private_key_parameters_hexadecimal_buffer.append(";");
        
        }
        
    }
    
    // Incrementation of the index for the random secret s
    // (i.e. the secret vector in the lattice algebraic structure),
    // encoded in bytes, with the IND-CPA (INDistinguishability
    // under Chosen Plaintext Attack) property, retrieved from
    // the private key parameters of the pair of asymmetric keys
    // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    random_secret_s_from_private_key_parameters_curr_byte_idx++;
    
}


// Initialization of the string representation in hexadecimal of the random secret s
// (i.e. the secret vector in the lattice algebraic structure), encoded in bytes,
// with the IND-CPA (INDistinguishability under Chosen Plaintext Attack) property,
// retrieved from the private key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem,
// from the upper-case values of the respective String Buffer.
String random_secret_s_from_private_key_parameters_hexadecimal_string = 
    random_secret_s_from_private_key_parameters_hexadecimal_buffer.toString().toUpperCase();

// Initialization of the string array representation of
// the several hexadecimal characters of he random secret s
// (i.e. the secret vector in the lattice algebraic structure),
// encoded in bytes, with the IND-CPA (INDistinguishability
// under Chosen Plaintext Attack) property, retrieved from
// the private key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
String[] random_secret_s_from_private_key_parameters_hexadecimal_chars = 
    random_secret_s_from_private_key_parameters_hexadecimal_string
        .split( ";", random_secret_s_from_private_key_parameters_hexadecimal_string.length() );

// Re-Initialization of the string representation in hexadecimal of the random secret s
// (i.e. the secret vector in the lattice algebraic structure), encoded in bytes,
// with the IND-CPA (INDistinguishability under Chosen Plaintext Attack) property,
// retrieved from the private key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
random_secret_s_from_private_key_parameters_hexadecimal_string = "";

// Concatenation of one bold line, as a separator, to the string representation
// in hexadecimal of the random secret s (i.e. the secret vector in
// the lattice algebraic structure), encoded in bytes, with the IND-CPA
// (INDistinguishability under Chosen Plaintext Attack) property,
// retrieved from the private key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
random_secret_s_from_private_key_parameters_hexadecimal_string += 
            "=====================================" +
            "=====================================" + 
            "\n          ";


// For each index of the numerical values of the random secret s
// (i.e. the secret vector in the lattice algebraic structure), encoded in bytes,
// with the IND-CPA (INDistinguishability under Chosen Plaintext Attack) property,
// retrieved from the private key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem. 
for( int s_i = 0; s_i < random_secret_s_from_private_key_parameters_hexadecimal_chars.length; s_i++ ) {
    
    // If the current index of the numerical value is not
    // the first one, and it is not a multiple of four.
    if( ( s_i != 0 ) && ( s_i % 4 == 0 ) ) {
        
        // Concatenation of one bold line, as a separator, to the string representation
        // in hexadecimal of the random secret s (i.e. the secret vector in
        // the lattice algebraic structure), encoded in bytes, with the IND-CPA
        // (INDistinguishability under Chosen Plaintext Attack) property, retrieved from
        // the private key parameters of the pair of asymmetric keys to be
        // used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        random_secret_s_from_private_key_parameters_hexadecimal_string += 
            "\n          " +
            "=====================================" +
            "=====================================" + 
            "\n          ";
        
    }
    
    // Concatenation of the current hexadecimal characters, to the string representation
    // in hexadecimal of the random secret s (i.e. the secret vector in the lattice
    // algebraic structure), encoded in bytes, with the IND-CPA (INDistinguishability
    // under Chosen Plaintext Attack) property, retrieved from the private key parameters of
    // the pair of asymmetric keys to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    random_secret_s_from_private_key_parameters_hexadecimal_string += 
        " s" + "[" + s_i + "]" + " = " + 
                random_secret_s_from_private_key_parameters_hexadecimal_chars[s_i] + "; ";
    
}

// Concatenation of one bold line, as a separator, to the string representation
// in hexadecimal of the random secret s (i.e. the secret vector in the lattice
// algebraic structure), encoded in bytes, with the IND-CPA (INDistinguishability
// under Chosen Plaintext Attack) property, retrieved from the private key parameters of
// the pair of asymmetric keys to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
random_secret_s_from_private_key_parameters_hexadecimal_string += 
            "\n          " +
            "=====================================" +
            "=====================================";


// Initialization of a (short) integer values array of the random secret s
// (i.e. the secret vector in the lattice algebraic structure), encoded in bytes,
// with the IND-CPA (INDistinguishability under Chosen Plaintext Attack) property,
// retrieved from the private key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
short[] random_secret_s_from_private_key_parameters_values =
    new short[ ( random_secret_s_size_bytes_from_private_key_parameters / 2 ) ];
    
// Conversion of the random secret s (i.e. the secret vector in the lattice
// algebraic structure), encoded in bytes, with the IND-CPA (INDistinguishability
// under Chosen Plaintext Attack) property, retrieved from the private key parameters of
// the pair of asymmetric keys to be used by the CRYSTALS-Kyber Public-Key Cryptosystem,
// from a byte array to a (short) integer array.
ByteBuffer.wrap( random_secret_s_from_private_key_parameters_bytes )
    .order( ByteOrder.LITTLE_ENDIAN ).asShortBuffer()
    .get( random_secret_s_from_private_key_parameters_values );


// Initialization of the string representation in (short)
// integer values of the random secret s (i.e. the secret vector in the lattice
// algebraic structure), encoded in bytes, with the IND-CPA (INDistinguishability
// under Chosen Plaintext Attack) property, retrieved from the private key parameters of
// the pair of asymmetric keys to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
String random_secret_s_from_private_key_parameters_values_string = "";

// Concatenation of one bold line, as a separator, to the string representation
// in (short) integer values of the random secret s (i.e. the secret vector in
// the lattice algebraic structure), encoded in bytes, with the IND-CPA
// (INDistinguishability under Chosen Plaintext Attack) property,
// retrieved from the private key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
random_secret_s_from_private_key_parameters_values_string += 
            "=====================================" +
            "=====================================" + 
            "\n          ";


// For each index of the numerical values of the random secret s
// (i.e. the secret vector in the lattice algebraic structure), encoded in bytes,
// with the IND-CPA (INDistinguishability under Chosen Plaintext Attack) property,
// retrieved from the private key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
for( int s_i = 0; s_i < random_secret_s_from_private_key_parameters_values.length; s_i++ ) {
    
    // If the current index of the numerical value is not
    // the first one, and it is not a multiple of four.
    if( ( s_i != 0 ) && ( s_i % 4 == 0 ) ) {
        
        // Concatenation of one bold line, as a separator, to the string representation
        // in (short) integer values of the random secret s (i.e. the secret vector in
        // the lattice algebraic structure), encoded in bytes, with the IND-CPA
        // (INDistinguishability under Chosen Plaintext Attack) property,
        // retrieved from the private key parameters of the pair of asymmetric keys
        // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        random_secret_s_from_private_key_parameters_values_string += 
            "\n          " +
            "=====================================" +
            "=====================================" + 
            "\n          ";
        
    }
    
    // Concatenation of the current numerical value, to the string representation
    // in (short) integer values of the random secret s (i.e. the secret vector in
    // the lattice algebraic structure), encoded in bytes, with the IND-CPA
    // (INDistinguishability under Chosen Plaintext Attack) property,
    // retrieved from the private key parameters of the pair of asymmetric keys
    // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    random_secret_s_from_private_key_parameters_values_string += 
        " s" + "[" + s_i + "]" + " = " + 
                random_secret_s_from_private_key_parameters_values[s_i] + "; ";
    
}

// Concatenation of one bold line, as a separator, to the string representation
// in (short) integer values of the random secret s (i.e. the secret vector in
// the lattice algebraic structure), encoded in bytes, with the IND-CPA
// (INDistinguishability under Chosen Plaintext Attack) property,
// retrieved from the private key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
random_secret_s_from_private_key_parameters_values_string += 
            "\n          " +
            "=====================================" +
            "=====================================";


// Print of a header for the information about the random secret s
// (i.e. the secret vector in the lattice algebraic structure), encoded in bytes,
// with the IND-CPA (INDistinguishability under Chosen Plaintext Attack) property,
// retrieved from the private key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
System.out.println( "        " + "+ s (Secret Vector) " + 
                           "[" +   random_secret_s_size_bytes_from_private_key_parameters + " bytes" + " | " + 
                                 ( random_secret_s_size_bytes_from_private_key_parameters / 2 ) + " short integers" + " | " + 
                                   random_secret_s_size_bits_from_private_key_parameters + " bits" + "]:" );

// Print of one blank line, as a separator.
System.out.println("\n");

// Print of the string representation in hexadecimal of the random secret s
// (i.e. the secret vector in the lattice algebraic structure), encoded in bytes,
// with the IND-CPA (INDistinguishability under Chosen Plaintext Attack) property,
// retrieved from the private key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
System.out.println( "        " + "  Hexadecimal Values: \n  " +
                    "        " + random_secret_s_from_private_key_parameters_hexadecimal_string );

// Print of two blank lines, as separators.
System.out.println("");
System.out.println("");

// Print of the string representation in (short) integer values of the random secret s
// (i.e. the secret vector in the lattice algebraic structure), encoded in bytes,
// with the IND-CPA (INDistinguishability under Chosen Plaintext Attack) property,
// retrieved from the private key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
System.out.println( "        " + "  (Short) Integer Values: \n  " +
                    "        " + random_secret_s_from_private_key_parameters_values_string );


// Print of one blank line, as a separator.
System.out.println("");

// Print of one bold line, as a separator.
System.out.println( "       " + "^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^"
                              + "^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^" );

// Print of one blank line, as a separator.
System.out.println("");


// Retrieval of the size, in terms of bytes, of the hashed public key,
// denoted as H(pub_key), encoded in bytes, with the IND-CPA (INDistinguishability
// under Chosen Plaintext Attack) property, retrieved from the private key parameters of
// the pair of asymmetric keys to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
int hashed_public_key_h_pub_k_size_bytes_from_private_key_parameters =
    crystals_kyber_cryptosystem.get_hashed_public_key_h_pub_k_size_bytes_from_private_key_parameters();

// Retrieval of the size, in terms of bits, of the hashed public key,
// denoted as H(pub_key), encoded in bytes, with the IND-CPA (INDistinguishability
// under Chosen Plaintext Attack) property, retrieved from the private key parameters of
// the pair of asymmetric keys to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
int hashed_public_key_h_pub_k_size_bits_from_private_key_parameters =
    crystals_kyber_cryptosystem.get_hashed_public_key_h_pub_k_size_bits_from_private_key_parameters();


// Retrieval of the hashed public key, denoted as H(pub_key), encoded in bytes,
// with the IND-CPA (INDistinguishability under Chosen Plaintext Attack) property,
// retrieved from the private key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
byte[] hashed_public_key_h_pub_k_from_private_key_parameters_bytes = 
    crystals_kyber_cryptosystem.get_hashed_public_key_h_pub_k_from_private_key_parameters();


// Initialization of a String Buffer for the string representation
// in hexadecimal of the hashed public key, denoted as H(pub_key), encoded in bytes,
// with the IND-CPA (INDistinguishability under Chosen Plaintext Attack) property,
// retrieved from the private key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
StringBuffer hashed_public_key_h_pub_k_from_private_key_parameters_hexadecimal_buffer = 
    new StringBuffer();


// Initialization of the index for the hashed public key,
// denoted as H(pub_key), encoded in bytes, with the IND-CPA
// (INDistinguishability under Chosen Plaintext Attack) property,
// retrieved from the private key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
int hashed_public_key_h_pub_k_from_private_key_parameters_curr_byte_idx = 0;

// For each byte of the hashed public key, denoted as H(pub_key),
// encoded in bytes, with the IND-CPA (INDistinguishability
// under Chosen Plaintext Attack) property, retrieved from
// the private key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
for( byte hashed_public_key_h_pub_k_from_private_key_parameters_byte :
        hashed_public_key_h_pub_k_from_private_key_parameters_bytes ) {
    
    // Conversion of the current byte of the hashed public key,
    // denoted as H(pub_key), encoded in bytes, with the IND-CPA
    // (INDistinguishability under Chosen Plaintext Attack) property,
    // retrieved from the private key parameters of the pair of asymmetric keys
    // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem,
    // to a hexadecimal character.
    hashed_public_key_h_pub_k_from_private_key_parameters_hexadecimal_buffer
        .append( Integer.toString( ( hashed_public_key_h_pub_k_from_private_key_parameters_byte & 0xff ) +
                                   0x100, NUM_POSSIBLE_BITS_IN_HEXADECIMAL_CHAR ).substring(1) );
  
        
    // If the current index of the byte of the hashed public key,
    // denoted as H(pub_key), encoded in bytes, with the IND-CPA
    // (INDistinguishability under Chosen Plaintext Attack) property,
    // retrieved from the private key parameters of the pair of asymmetric keys
    // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem,
    // is not the last one.
    if( hashed_public_key_h_pub_k_from_private_key_parameters_curr_byte_idx < 
        ( hashed_public_key_h_pub_k_from_private_key_parameters_bytes.length - 1 ) ) {

        // Concatenation of a 'delimeter' ( ; ) to the String Buffer for the string representation
        // in hexadecimal of the hashed public key, denoted as H(pub_key), encoded in bytes,
        // with the IND-CPA (INDistinguishability under Chosen Plaintext Attack) property,
        // retrieved from the private key parameters of the pair of asymmetric keys
        // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        hashed_public_key_h_pub_k_from_private_key_parameters_hexadecimal_buffer.append(";");

    }
  
    
    // Incrementation of the index for the hashed public key,
    // denoted as H(pub_key), encoded in bytes, with the IND-CPA
    // (INDistinguishability under Chosen Plaintext Attack) property,
    // retrieved from the private key parameters of the pair of asymmetric keys
    // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem. 
    hashed_public_key_h_pub_k_from_private_key_parameters_curr_byte_idx++;
    
}


// Initialization of the string representation in hexadecimal of
// the hashed public key, denoted as H(pub_key), encoded in bytes,
// with the IND-CPA (INDistinguishability under Chosen Plaintext Attack) property,
// retrieved from the private key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem,
// from the upper-case values of the respective String Buffer.
String hashed_public_key_h_pub_k_from_private_key_parameters_hexadecimal_string = 
    hashed_public_key_h_pub_k_from_private_key_parameters_hexadecimal_buffer.toString().toUpperCase();

// Initialization of the string array representation of the several hexadecimal
// characters of the hashed public key, denoted as H(pub_key), encoded in bytes,
// with the IND-CPA (INDistinguishability under Chosen Plaintext Attack) property,
// retrieved from the private key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem,
// from the upper-case values of the respective String Buffer.
String[] hashed_public_key_h_pub_k_from_private_key_parameters_hexadecimal_chars = 
    hashed_public_key_h_pub_k_from_private_key_parameters_hexadecimal_string
        .split( ";", hashed_public_key_h_pub_k_from_private_key_parameters_hexadecimal_string.length() );

// Re-Initialization of the string representation in hexadecimal of
// the hashed public key, denoted as H(pub_key), encoded in bytes,
// with the IND-CPA (INDistinguishability under Chosen Plaintext Attack) property,
// retrieved from the private key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
hashed_public_key_h_pub_k_from_private_key_parameters_hexadecimal_string = "";

// Concatenation of one bold line, as a separator, to the string representation
// in hexadecimal of the hashed public key, denoted as H(pub_key), encoded in bytes,
// with the IND-CPA (INDistinguishability under Chosen Plaintext Attack) property,
// retrieved from the private key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
hashed_public_key_h_pub_k_from_private_key_parameters_hexadecimal_string += 
            "=====================================" +
            "=====================================" + 
            "\n          ";


// For each index of the numerical values of the hashed public key,
// denoted as H(pub_key), encoded in bytes, with the IND-CPA
// (INDistinguishability under Chosen Plaintext Attack) property,
// retrieved from the private key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
for( int h_pub_k_i = 0; h_pub_k_i < hashed_public_key_h_pub_k_from_private_key_parameters_hexadecimal_chars.length; h_pub_k_i++ ) {
    
    // If the current index of the numerical value is not
    // the first one, and it is not a multiple of three.
    if( ( h_pub_k_i != 0 ) && ( h_pub_k_i % 3 == 0 ) ) {
        
        // Concatenation of one bold line, as a separator, to the string representation
        // in hexadecimal of the hashed public key, denoted as H(pub_key), encoded in bytes,
        // with the IND-CPA (INDistinguishability under Chosen Plaintext Attack) property,
        // retrieved from the private key parameters of the pair of asymmetric keys
        // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        hashed_public_key_h_pub_k_from_private_key_parameters_hexadecimal_string += 
            "\n          " +
            "=====================================" +
            "=====================================" + 
            "\n          ";
        
    }
    
    // Concatenation of the current hexadecimal characters, to the string representation
    // in hexadecimal of the hashed public key, denoted as H(pub_key), encoded in bytes,
    // with the IND-CPA (INDistinguishability under Chosen Plaintext Attack) property,
    // retrieved from the private key parameters of the pair of asymmetric keys
    // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    hashed_public_key_h_pub_k_from_private_key_parameters_hexadecimal_string += 
        " H(Kₚ)" + "[" + h_pub_k_i + "]" + " = " + 
                hashed_public_key_h_pub_k_from_private_key_parameters_hexadecimal_chars[h_pub_k_i] + "; ";
    
}

// Concatenation of one bold line, as a separator, to the string representation
// in hexadecimal of the hashed public key, denoted as H(pub_key), encoded in bytes,
// with the IND-CPA (INDistinguishability under Chosen Plaintext Attack) property,
// retrieved from the private key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
hashed_public_key_h_pub_k_from_private_key_parameters_hexadecimal_string += 
            "\n          " +
            "=====================================" +
            "=====================================";


// Initialization of the string representation in binary of
// the hashed public key, denoted as H(pub_key), encoded in bytes,
// with the IND-CPA (INDistinguishability under Chosen Plaintext Attack) property,
// retrieved from the private key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
String hashed_public_key_h_pub_k_from_private_key_parameters_binary_string = "H(Kₚ) = {0,1}²⁵⁶ = ";


// Initialization of the index for the hashed public key,
// denoted as H(pub_key), encoded in bytes, with the IND-CPA
// (INDistinguishability under Chosen Plaintext Attack) property,
// retrieved from the private key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
hashed_public_key_h_pub_k_from_private_key_parameters_curr_byte_idx = 0;

// For each byte of the hashed public key, denoted as H(pub_key), encoded in bytes,
// with the IND-CPA (INDistinguishability under Chosen Plaintext Attack) property,
// retrieved from the private key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
for( byte hashed_public_key_h_pub_k_from_private_key_parameters_byte :
        hashed_public_key_h_pub_k_from_private_key_parameters_bytes ) {

    // Conversion of the current byte of the hashed public key,
    // denoted as H(pub_key), encoded in bytes, with the IND-CPA
    // (INDistinguishability under Chosen Plaintext Attack) property,
    // retrieved from the private key parameters of the pair of asymmetric keys
    // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem,
    // to a binary character (i.e., a bit).
    hashed_public_key_h_pub_k_from_private_key_parameters_binary_string += 
        Integer.toBinaryString( hashed_public_key_h_pub_k_from_private_key_parameters_byte
                                & 255 | 256 ).substring(1);
    
    // If the current index of the byte of the hashed public key,
    // denoted as H(pub_key), encoded in bytes, with the IND-CPA
    // (INDistinguishability under Chosen Plaintext Attack) property,
    // retrieved from the private key parameters of the pair of asymmetric keys
    // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem,
    // is a multiple of six.
    if( ( ( hashed_public_key_h_pub_k_from_private_key_parameters_curr_byte_idx + 1 ) % 6 ) == 0 ) {
        
        // Concatenation of a 'blank line' and a 'blank space' to the string representation
        // in binary of the hashed public key, denoted as H(pub_key), encoded in bytes,
        // with the IND-CPA (INDistinguishability under Chosen Plaintext Attack) property,
        // retrieved from the private key parameters of the pair of asymmetric keys
        // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        hashed_public_key_h_pub_k_from_private_key_parameters_binary_string += 
            "\n                              ";
        
    }
    
    // Incrementation of the index for the hashed public key,
    // denoted as H(pub_key), encoded in bytes, with the IND-CPA (INDistinguishability
    // under Chosen Plaintext Attack) property, retrieved from the private key parameters of
    // the pair of asymmetric keys to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    hashed_public_key_h_pub_k_from_private_key_parameters_curr_byte_idx++;
    
}


// Print of a header for the information about the hashed public key,
// denoted as H(pub_key), encoded in bytes, with the IND-CPA
// (INDistinguishability under Chosen Plaintext Attack) property,
// retrieved from the private key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
System.out.println( "        " + "+ H(Kₚ) (hashed public key) " + 
                           "[" + hashed_public_key_h_pub_k_size_bytes_from_private_key_parameters + " bytes" + " | " + 
                                 hashed_public_key_h_pub_k_size_bits_from_private_key_parameters + " bits" + "]:" );

// Print of one blank line, as a separator.
System.out.println("\n");

// Print of the string representation in hexadecimal of
// the hashed public key, denoted as H(pub_key), encoded in bytes,
// with the IND-CPA (INDistinguishability under Chosen Plaintext Attack) property,
// retrieved from the private key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
System.out.println( "        " + "  Hexadecimal Values: \n  " +
                    "        " + hashed_public_key_h_pub_k_from_private_key_parameters_hexadecimal_string );

// Print of two blank lines, as separators.
System.out.println("");
System.out.println("");

// Print of the string representation in binary of
// the hashed public key, denoted as H(pub_key), encoded in bytes,
// with the IND-CPA (INDistinguishability under Chosen Plaintext Attack) property,
// retrieved from the private key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
System.out.println( "        " + "  Binary Values: \n   " +
                    "        " + hashed_public_key_h_pub_k_from_private_key_parameters_binary_string );


// Print of one blank line, as a separator.
System.out.println("");

// Print of one bold line, as a separator.
System.out.println( "       " + "^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^"
                              + "^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^" );

// Print of one blank line, as a separator.
System.out.println("");


// Retrieval of the size, in terms of bytes, of the random secret nonce z
// (i.e., to avoid practical fault attacks), encoded in bytes, with
// the IND-CPA (INDistinguishability under Chosen Plaintext Attack) property
// for the Asymmetric Encryption and with the (INDistinguishability under
// Chosen Ciphertext Attack) property for the KEM (Key Encapsulation Method/Mechanism),
// retrieved from the private key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
int random_secret_nonce_z_size_bytes_from_private_key_parameters =
    crystals_kyber_cryptosystem.get_random_secret_nonce_z_size_bytes_from_private_key_parameters();

// Retrieval of the size, in terms of bits, of the random secret nonce z
// (i.e., to avoid practical fault attacks), encoded in bytes, with
// the IND-CPA (INDistinguishability under Chosen Plaintext Attack) property
// for the Asymmetric Encryption and with the (INDistinguishability under
// Chosen Ciphertext Attack) property for the KEM (Key Encapsulation Method/Mechanism),
// retrieved from the private key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
int random_secret_nonce_z_size_bits_from_private_key_parameters =
    crystals_kyber_cryptosystem.get_random_secret_nonce_z_size_bits_from_private_key_parameters();


// Retrieval of the random secret nonce z (i.e., to avoid practical fault attacks),
// encoded in bytes, with the IND-CPA (INDistinguishability under Chosen Plaintext Attack)
// property for the Asymmetric Encryption and with the (INDistinguishability under
// Chosen Ciphertext Attack) property for the KEM (Key Encapsulation Method/Mechanism),
// retrieved from the private key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
byte[] random_secret_nonce_z_from_private_key_parameters_bytes = 
    crystals_kyber_cryptosystem.get_random_secret_nonce_z_from_private_key_parameters();


// Initialization of a String Buffer for the string representation
// in hexadecimal of the random secret nonce z (i.e., to avoid practical fault attacks),
// encoded in bytes, with the IND-CPA (INDistinguishability under Chosen Plaintext Attack)
// property for the Asymmetric Encryption and with the (INDistinguishability under
// Chosen Ciphertext Attack) property for the KEM (Key Encapsulation Method/Mechanism),
// retrieved from the private key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
StringBuffer random_secret_nonce_z_from_private_key_parameters_hexadecimal_buffer = 
    new StringBuffer();


// Initialization of the index for the random secret nonce z
// (i.e., to avoid practical fault attacks), encoded in bytes, with the IND-CPA
// (INDistinguishability under Chosen Plaintext Attack) property for the Asymmetric Encryption
// and with the (INDistinguishability under Chosen Ciphertext Attack) property for
// the KEM (Key Encapsulation Method/Mechanism), retrieved from the private key parameters of
// the pair of asymmetric keys to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
int random_secret_nonce_z_from_private_key_parameters_curr_byte_idx = 0;

// For each byte of the random secret nonce z
// (i.e., to avoid practical fault attacks), encoded in bytes,
// with the IND-CPA (INDistinguishability under Chosen Plaintext Attack) property for
// the Asymmetric Encryption and with the (INDistinguishability under Chosen Ciphertext Attack)
// property for the KEM (Key Encapsulation Method/Mechanism), retrieved from the private key
// parameters of the pair of asymmetric keys to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
for( byte random_secret_nonce_z_from_private_key_parameters_byte :
        random_secret_nonce_z_from_private_key_parameters_bytes ) {
    
    // Conversion of the current byte of the random secret nonce z
    // (i.e., to avoid practical fault attacks), encoded in bytes, with the IND-CPA
    // (INDistinguishability under Chosen Plaintext Attack) property for the Asymmetric Encryption
    // and with the (INDistinguishability under Chosen Ciphertext Attack) property for
    // the KEM (Key Encapsulation Method/Mechanism), retrieved from the private key parameters of
    // the pair of asymmetric keys to be used by the CRYSTALS-Kyber Public-Key Cryptosystem,
    // to a hexadecimal character.
    random_secret_nonce_z_from_private_key_parameters_hexadecimal_buffer
        .append( Integer.toString( ( random_secret_nonce_z_from_private_key_parameters_byte & 0xff ) +
                                   0x100, NUM_POSSIBLE_BITS_IN_HEXADECIMAL_CHAR ).substring(1) );
    
  
    // If the current index of the byte of the random secret nonce z
    // (i.e., to avoid practical fault attacks), encoded in bytes, with the IND-CPA
    // (INDistinguishability under Chosen Plaintext Attack) property for the Asymmetric Encryption
    // and with the (INDistinguishability under Chosen Ciphertext Attack) property for
    // the KEM (Key Encapsulation Method/Mechanism), retrieved from the private key
    // parameters of the pair of asymmetric keys to be used by the CRYSTALS-Kyber
    // Public-Key Cryptosystem, is not the last one.
    if( random_secret_nonce_z_from_private_key_parameters_curr_byte_idx < 
        ( random_secret_nonce_z_from_private_key_parameters_bytes.length - 1 ) ) {

        // Concatenation of a 'delimeter' ( ; ) to the String Buffer for the string representation
        // in hexadecimal of the random secret nonce z (i.e., to avoid practical fault attacks),
        // encoded in bytes, with the IND-CPA (INDistinguishability under Chosen Plaintext Attack)
        // property for the Asymmetric Encryption and with the (INDistinguishability under
        // Chosen Ciphertext Attack) property for the KEM (Key Encapsulation Method/Mechanism),
        // retrieved from the private key parameters of the pair of asymmetric keys
        // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        random_secret_nonce_z_from_private_key_parameters_hexadecimal_buffer.append(";");

    }
    
    
    // Incrementation of the index for the random secret nonce z
    // (i.e., to avoid practical fault attacks), encoded in bytes, with the IND-CPA
    // (INDistinguishability under Chosen Plaintext Attack) property for the Asymmetric Encryption
    // and with the (INDistinguishability under Chosen Ciphertext Attack) property for
    // the KEM (Key Encapsulation Method/Mechanism), retrieved from the private key parameters of
    // the pair of asymmetric keys to be used by the CRYSTALS-Kyber Public-Key Cryptosystem. 
    random_secret_nonce_z_from_private_key_parameters_curr_byte_idx++;
    
}


// Initialization of the string representation in hexadecimal of the random secret nonce z
// (i.e., to avoid practical fault attacks), encoded in bytes, with the IND-CPA
// (INDistinguishability under Chosen Plaintext Attack) property for the Asymmetric Encryption
// and with the (INDistinguishability under Chosen Ciphertext Attack) property for
// the KEM (Key Encapsulation Method/Mechanism), retrieved from the private key parameters of
// the pair of asymmetric keys to be used by the CRYSTALS-Kyber Public-Key Cryptosystem,
// from the upper-case values of the respective String Buffer.
String random_secret_nonce_z_from_private_key_parameters_hexadecimal_string = 
    random_secret_nonce_z_from_private_key_parameters_hexadecimal_buffer.toString().toUpperCase();

// Initialization of the string array representation of the several 
// hexadecimal characters of the random secret nonce z (i.e., to avoid practical fault attacks),
// encoded in bytes, with the IND-CPA (INDistinguishability under Chosen Plaintext Attack) property
// for the Asymmetric Encryption and with the (INDistinguishability under Chosen Ciphertext Attack)
// property for the KEM (Key Encapsulation Method/Mechanism), retrieved from the private key
// parameters of the pair of asymmetric keys to be used by the CRYSTALS-Kyber Public-Key Cryptosystem,
// from the upper-case values of the respective String Buffer.
String[] random_secret_nonce_z_from_private_key_parameters_hexadecimal_chars = 
    random_secret_nonce_z_from_private_key_parameters_hexadecimal_string
        .split( ";", random_secret_nonce_z_from_private_key_parameters_hexadecimal_string.length() );

// Re-Initialization of the string representation in hexadecimal of
// the random secret nonce z (i.e., to avoid practical fault attacks),
// encoded in bytes, with the IND-CPA (INDistinguishability under Chosen Plaintext Attack) property
// for the Asymmetric Encryption and with the (INDistinguishability under Chosen Ciphertext Attack)
// property for the KEM (Key Encapsulation Method/Mechanism), retrieved from the private key
// parameters of the pair of asymmetric keys to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
random_secret_nonce_z_from_private_key_parameters_hexadecimal_string = "";

// Concatenation of one bold line, as a separator, to the string representation
// in hexadecimal of the random secret nonce z (i.e., to avoid practical fault attacks),
// encoded in bytes, with the IND-CPA (INDistinguishability under Chosen Plaintext Attack) property
// for the Asymmetric Encryption and with the (INDistinguishability under Chosen Ciphertext Attack)
// property for the KEM (Key Encapsulation Method/Mechanism), retrieved from the private key
// parameters of the pair of asymmetric keys to be used by the CRYSTALS-Kyber Public-Key Cryptosystem. 
random_secret_nonce_z_from_private_key_parameters_hexadecimal_string += 
            "=====================================" +
            "=====================================" + 
            "\n          ";


// For each index of the numerical values of the random secret nonce z
// (i.e., to avoid practical fault attacks), encoded in bytes, with the IND-CPA
// (INDistinguishability under Chosen Plaintext Attack) property for the Asymmetric Encryption
// and with the (INDistinguishability under Chosen Ciphertext Attack) property for
// the KEM (Key Encapsulation Method/Mechanism), retrieved from the private key
// parameters of the pair of asymmetric keys to be used by the CRYSTALS-Kyber Public-Key Cryptosystem. 
for( int z_i = 0; z_i < random_secret_nonce_z_from_private_key_parameters_hexadecimal_chars.length; z_i++ ) {
    
    // If the current index of the numerical value is not
    // the first one, and it is not a multiple of four.
    if( ( z_i != 0 ) && ( z_i % 4 == 0 ) ) {
        
        // Concatenation of one bold line, as a separator, to the string representation
        // in hexadecimal of the random secret nonce z (i.e., to avoid practical fault attacks),
        // encoded in bytes, with the IND-CPA (INDistinguishability under Chosen Plaintext Attack)
        // property for the Asymmetric Encryption and with the (INDistinguishability under
        // Chosen Ciphertext Attack) property for the KEM (Key Encapsulation Method/Mechanism),
        // retrieved from the private key parameters of the pair of asymmetric keys
        // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        random_secret_nonce_z_from_private_key_parameters_hexadecimal_string += 
            "\n          " +
            "=====================================" +
            "=====================================" + 
            "\n          ";
        
    }
    
    // Concatenation of the current hexadecimal characters, to the string representation
    // in hexadecimal of the random secret nonce z (i.e., to avoid practical fault attacks),
    // encoded in bytes, with the IND-CPA (INDistinguishability under Chosen Plaintext Attack)
    // property for the Asymmetric Encryption and with the (INDistinguishability under
    // Chosen Ciphertext Attack) property for the KEM (Key Encapsulation Method/Mechanism),
    // retrieved from the private key parameters of the pair of asymmetric keys
    // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    random_secret_nonce_z_from_private_key_parameters_hexadecimal_string += 
        " z" + "[" + z_i + "]" + " = " + 
                random_secret_nonce_z_from_private_key_parameters_hexadecimal_chars[z_i] + "; ";
    
}

// Concatenation of one bold line, as a separator, to the string representation
// in hexadecimal of the random secret nonce z (i.e., to avoid practical fault attacks),
// encoded in bytes, with the IND-CPA (INDistinguishability under Chosen Plaintext Attack)
// property for the Asymmetric Encryption and with the (INDistinguishability under
// Chosen Ciphertext Attack) property for the KEM (Key Encapsulation Method/Mechanism),
// retrieved from the private key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
random_secret_nonce_z_from_private_key_parameters_hexadecimal_string += 
            "\n          " +
            "=====================================" +
            "=====================================";


// Initialization of the string representation in binary of
// the random secret nonce z (i.e., to avoid practical fault attacks),
// encoded in bytes, with the IND-CPA (INDistinguishability under Chosen Plaintext Attack)
// property for the Asymmetric Encryption and with the (INDistinguishability under
// Chosen Ciphertext Attack) property for the KEM (Key Encapsulation Method/Mechanism),
// retrieved from the private key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
String random_secret_nonce_z_from_private_key_parameters_binary_string = "z = {0,1}²⁵⁶ = ";


// Initialization of the index for the random secret nonce z
// (i.e., to avoid practical fault attacks), encoded in bytes, with the IND-CPA
// (INDistinguishability under Chosen Plaintext Attack) property for the Asymmetric Encryption
// and with the (INDistinguishability under Chosen Ciphertext Attack) property for
// the KEM (Key Encapsulation Method/Mechanism), retrieved from the private key parameters of
// the pair of asymmetric keys to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
random_secret_nonce_z_from_private_key_parameters_curr_byte_idx = 0;

// For each byte of the random secret nonce z (i.e., to avoid practical fault attacks),
// encoded in bytes, with the IND-CPA (INDistinguishability under Chosen Plaintext Attack)
// property for the Asymmetric Encryption and with the (INDistinguishability under
// Chosen Ciphertext Attack) property for the KEM (Key Encapsulation Method/Mechanism),
// retrieved from the private key parameters of the pair of asymmetric keys
// to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
for( byte random_secret_nonce_z_from_private_key_parameters_byte :
        random_secret_nonce_z_from_private_key_parameters_bytes ) {

    // Conversion of the current byte of the random secret nonce z
    // (i.e., to avoid practical fault attacks), encoded in bytes, with the IND-CPA
    // (INDistinguishability under Chosen Plaintext Attack) property for the Asymmetric Encryption
    // and with the (INDistinguishability under Chosen Ciphertext Attack) property for
    // the KEM (Key Encapsulation Method/Mechanism), retrieved from the private key parameters of
    // the pair of asymmetric keys to be used by the CRYSTALS-Kyber Public-Key Cryptosystem,
    // to a binary character (i.e., a bit).
    random_secret_nonce_z_from_private_key_parameters_binary_string += 
        Integer.toBinaryString( random_secret_nonce_z_from_private_key_parameters_byte
                                & 255 | 256 ).substring(1);
    
    // If the current index of the byte of the random secret nonce z
    // (i.e., to avoid practical fault attacks), encoded in bytes, with the IND-CPA
    // (INDistinguishability under Chosen Plaintext Attack) property for the Asymmetric Encryption
    // and with the (INDistinguishability under Chosen Ciphertext Attack) property for
    // the KEM (Key Encapsulation Method/Mechanism), retrieved from the private key parameters of
    // the pair of asymmetric keys to be used by the CRYSTALS-Kyber Public-Key Cryptosystem,
    // is a multiple of six.
    if( ( ( random_secret_nonce_z_from_private_key_parameters_curr_byte_idx + 1 ) % 6 ) == 0 ) {
        
        // Concatenation of a 'blank line' and a 'blank space' to the string representation
        // in binary of the random secret nonce z (i.e., to avoid practical fault attacks),
        // encoded in bytes, with the IND-CPA (INDistinguishability under Chosen Plaintext Attack)
        // property for the Asymmetric Encryption and with the (INDistinguishability under
        // Chosen Ciphertext Attack) property for the KEM (Key Encapsulation Method/Mechanism),
        // retrieved from the private key parameters of the pair of asymmetric keys
        // to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
        random_secret_nonce_z_from_private_key_parameters_binary_string += 
            "\n                          ";
        
    }
    
    // Incrementation of the index for the random secret nonce z
    // (i.e., to avoid practical fault attacks), encoded in bytes, with the IND-CPA
    // (INDistinguishability under Chosen Plaintext Attack) property for the Asymmetric Encryption
    // and with the (INDistinguishability under Chosen Ciphertext Attack) property for
    // the KEM (Key Encapsulation Method/Mechanism), retrieved from the private key parameters of
    // the pair of asymmetric keys to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
    random_secret_nonce_z_from_private_key_parameters_curr_byte_idx++;
    
}


// Print of a header for the information about the random secret nonce z
// (i.e., to avoid practical fault attacks), encoded in bytes, with the IND-CPA
// (INDistinguishability under Chosen Plaintext Attack) property for the Asymmetric Encryption
// and with the (INDistinguishability under Chosen Ciphertext Attack) property for
// the KEM (Key Encapsulation Method/Mechanism), retrieved from the private key parameters of
// the pair of asymmetric keys to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
System.out.println( "        " + "+ z (random secret nonce) " + 
                           "[" + random_secret_nonce_z_size_bytes_from_private_key_parameters + " bytes" + " | " + 
                                 random_secret_nonce_z_size_bits_from_private_key_parameters + " bits" + "]:" );

// Print of one blank line, as a separator.
System.out.println("\n");

// Print of the string representation in hexadecimal of the random secret nonce z
// (i.e., to avoid practical fault attacks), encoded in bytes, with the IND-CPA
// (INDistinguishability under Chosen Plaintext Attack) property for the Asymmetric Encryption
// and with the (INDistinguishability under Chosen Ciphertext Attack) property for
// the KEM (Key Encapsulation Method/Mechanism), retrieved from the private key parameters of
// the pair of asymmetric keys to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
System.out.println( "        " + "  Hexadecimal Values: \n  " +
                    "        " + random_secret_nonce_z_from_private_key_parameters_hexadecimal_string );

// Print of two blank lines, as separators.
System.out.println("");
System.out.println("");

// Print of the string representation in binary of the random secret nonce z
// (i.e., to avoid practical fault attacks), encoded in bytes, with the IND-CPA
// (INDistinguishability under Chosen Plaintext Attack) property for the Asymmetric Encryption
// and with the (INDistinguishability under Chosen Ciphertext Attack) property for
// the KEM (Key Encapsulation Method/Mechanism), retrieved from the private key parameters of
// the pair of asymmetric keys to be used by the CRYSTALS-Kyber Public-Key Cryptosystem.
System.out.println( "        " + "  Binary Values: \n   " +
                    "        " + random_secret_nonce_z_from_private_key_parameters_binary_string );



 ---------------- EXAMPLE OF AN EXECUTION OF THE CRYSTALS-KYBER PUBLIC-KEY CRYPTOSYSTEM ---------------- 


  NAME, DESCRIPTION, AND EXTRACTION OF THE EXECUTION AND SECURITY PARAMETERS ADOPTED:

   => Kyber512: Security Parameters Set with a factor of k = 2 ( k x n = 2 x 256 = 512 )
                for the size of the lattice algebraic structure, which provides
                a (post-quantum) security strength of 64 bits, designed for
                the distribution of symmetric session keys of 128 bits.

                NOTE: The Kyber512 Cryptosystem has a NIST (Classical)
                      Post-Quantum Security Level of 1 (≈ exhaustive search on AES-128).

     * Size of the Symmetric Session Secret Key to be generated and exchanged:
       - 16 bytes | 128 bits

     * Security Parameter Set to be adopted:
       - k = 2 | n = 256 | q = 3329 | q⁻¹ = 62209 | η₁ = 3 | η₂ = 2
       - dₜ = 12 | dᵤ = 10 | dᵥ = 4

     * Size of the Symmetric Hashes, (Pseudo) Random Seeds and
  

           t[200] = 5B AA;  t[201] = BC 43;  t[202] = 7E BD;  t[203] = 8B 3B; 
           t[204] = 98 18;  t[205] = 19 00;  t[206] = 4D 1D;  t[207] = 79 58; 
           t[208] = 57 60;  t[209] = 51 CF;  t[210] = 0D 39;  t[211] = 1B 82; 
           t[212] = 50 C9;  t[213] = 51 01;  t[214] = 9D 0D;  t[215] = 71 82; 
           t[216] = E3 16;  t[217] = 6D DF;  t[218] = 33 C9;  t[219] = 68 9C; 
           t[220] = 99 BE;  t[221] = 75 3A;  t[222] = FF 17;  t[223] = 73 5C; 
           t[224] = EA A6;  t[225] = D4 E4;  t[226] = 15 D1;  t[227] = 0B 91; 
           t[228] = 0D 15;  t[229] = BF 57;  t[230] = 50 37;  t[231] = B5 16; 
           t[232] = 2B E0;  t[233] = F8 99;  t[234] = 6A 75;  t[235] = 62 4E; 
           t[236] = 63 80;  t[237] = D7 B5;  t[238] = 5D 61;  t[239] = 79 29; 
           t[240] = 1A 1A;  t[241] = 04 53;  t[242] = 35 48;  t[243] = 96 F6; 
           t[244] = 9E 50;  t[245] = 53 C6;  t[246] = A8 21;  t[247] = 0A 95; 
           t[248] = E3 3C;  t[249] = 44 A5;  t[250] 

           t[200] = -21925;  t[201] = 17340;  t[202] = -17026;  t[203] = 15243; 
           t[204] = 6296;  t[205] = 25;  t[206] = 7501;  t[207] = 22649; 
           t[208] = 24663;  t[209] = -12463;  t[210] = 14605;  t[211] = -32229; 
           t[212] = -14000;  t[213] = 337;  t[214] = 3485;  t[215] = -32143; 
           t[216] = 5859;  t[217] = -8339;  t[218] = -14029;  t[219] = -25496; 
           t[220] = -16743;  t[221] = 14965;  t[222] = 6143;  t[223] = 23667; 
           t[224] = -22806;  t[225] = -6956;  t[226] = -12011;  t[227] = -28405; 
           t[228] = 5389;  t[229] = 22463;  t[230] = 14160;  t[231] = 5813; 
           t[232] = -8149;  t[233] = -26120;  t[234] = 30058;  t[235] = 20066; 
           t[236] = -32669;  t[237] = -18985;  t[238] = 24925;  t[239] = 10617; 
           t[240] = 6682;  t[241] = 21252;  t[242] = 18485;  t[243] = -2410; 
           t[244] = 20638;  t[245] = -14765;  t[246] = 8616;  t[247] = -27382; 
           t[248] = 15587;  t[249] = -23228;  t[2



          Binary Values: 
           ρ = {0,1}²⁵⁶ = 001110110101100100000110001100100001001101010000
                          001111100000000100011101000010110100110001100011
                          101000001011010001110111001101011011011001001000
                          100110011000111001111001010101001110000111111011
                          011011100000101000011001110010100001111010111111
                          1100011001111110


       PARAMETERS AND INFORMATION ABOUT THE ASYMMETRIC PRIVATE KEY:


        + s (Secret Vector) [768 bytes | 384 short integers | 6144 bits]:


          Hexadecimal Values: 
           s[0] = BF 87;  s[1] = 70 9F;  s[2] = A5 1B;  s[3] = 45 D2; 
           s[4] = 8E 44;  s[5] = 47 2F;  s[6] = E6 98;  s[7] = 6C 65; 
           s[8] = 0C B7;  s[9] = FF D4;  s[10] = 8E D7;  s[11] = B8 88; 
           s[12] = 90 EB;  s[13] = 9A A5;  s[14] = 37 11;  s[15] = 72 B3; 
           s[16] = 95 E8;  s[17] = 59 B5;  s[18] = 61 58;  s[19] = A0 E6; 
          

           s[200] = E6 6A;  s[201] = E2 CB;  s[202] = 56 76;  s[203] = B2 A2; 
           s[204] = E1 E4;  s[205] = 5D 15;  s[206] = E4 75;  s[207] = F1 D5; 
           s[208] = 7F 07;  s[209] = 88 29;  s[210] = 83 56;  s[211] = 06 F4; 
           s[212] = B1 0F;  s[213] = 75 35;  s[214] = 84 40;  s[215] = 19 13; 
           s[216] = F3 63;  s[217] = 61 B8;  s[218] = 33 3A;  s[219] = CF CB; 
           s[220] = 8E 39;  s[221] = 44 7A;  s[222] = 49 D0;  s[223] = B4 F9; 
           s[224] = 08 3E;  s[225] = 70 56;  s[226] = 64 4F;  s[227] = B5 AC; 
           s[228] = 48 C7;  s[229] = 28 E1;  s[230] = 91 67;  s[231] = A9 62; 
           s[232] = 9C E2;  s[233] = 3A 6F;  s[234] = 47 7B;  s[235] = AB C2; 
           s[236] = C6 7C;  s[237] = AC 9C;  s[238] = C6 71;  s[239] = F4 7A; 
           s[240] = 1E 07;  s[241] = B1 25;  s[242] = 84 69;  s[243] = 02 79; 
           s[244] = 9E 7B;  s[245] = 62 0B;  s[246] = 0F 54;  s[247] = BD D5; 
           s[248] = 50 0D;  s[249] = 1F 42;  s[250] 

           s[200] = 27366;  s[201] = -13342;  s[202] = 30294;  s[203] = -23886; 
           s[204] = -6943;  s[205] = 5469;  s[206] = 30180;  s[207] = -10767; 
           s[208] = 1919;  s[209] = 10632;  s[210] = 22147;  s[211] = -3066; 
           s[212] = 4017;  s[213] = 13685;  s[214] = 16516;  s[215] = 4889; 
           s[216] = 25587;  s[217] = -18335;  s[218] = 14899;  s[219] = -13361; 
           s[220] = 14734;  s[221] = 31300;  s[222] = -12215;  s[223] = -1612; 
           s[224] = 15880;  s[225] = 22128;  s[226] = 20324;  s[227] = -21323; 
           s[228] = -14520;  s[229] = -7896;  s[230] = 26513;  s[231] = 25257; 
           s[232] = -7524;  s[233] = 28474;  s[234] = 31559;  s[235] = -15701; 
           s[236] = 31942;  s[237] = -25428;  s[238] = 29126;  s[239] = 31476; 
           s[240] = 1822;  s[241] = 9649;  s[242] = 27012;  s[243] = 30978; 
           s[244] = 31646;  s[245] = 2914;  s[246] = 21519;  s[247] = -10819; 
           s[248] = 3408;  s[249] = 16927;  s[25



          Binary Values: 
           H(Kₚ) = {0,1}²⁵⁶ = 011010110100100101010110010111010101001100110100
                              011101100010110000000111101010111001101000110010
                              010100000100001010010110000011010011001000101000
                              111010111001000011011010101110110000011110010011
                              010101010110111000000100010111101011110010101000
                              1110010101110000

       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

        + z (random secret nonce) [32 bytes | 256 bits]:


          Hexadecimal Values: 
           z[0] = 1F;  z[1] = 58;  z[2] = 26;  z[3] = 88; 
           z[4] = 22;  z[5] = 1A;  z[6] = F9;  z[7] = DB; 
           z[8] = 44;  z[9] = B8;  z[10] = C1;  z[11] = F9; 
           z[12] = 64;  z[13] = 71;  z[14] = 9E;  z[15] = 0B; 
           z[16] = C0;  z[17] = 7D;  z[18] = 4E;  z[19] = FB; 
           z[20] = B6;  z[21] = 4F;  z[22] = 

null

***

### Demonstration for the IND-CCA2 KEM<br>using the CRYSTALS-Kyber Cryptosystem

***

## Conclusion
***

***In development...***

***